# BERT senetence similarty analyser 
by using BERT senetence similarty analyser we can begin to get a better understanding of if a question is close enoough to a target intent to use the corasponding answer.


## Step 1
lets start by reading in the current data we have set inside a target json file

In [1]:
import json
  
# Opening JSON file
f = open('intents.json',)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
for i in data['intents']:
    print(i)
  
# Closing file
f.close()

{'tag': 'Q0', 'patterns': ['A CSR in my class cannot access their CCO Learning ', 'A CSR in my class cannot get into their CCO Learning. ', 'A CSR in my class cannot log into their CCO Learning ', 'A participant in my class cannot access their CCO Learning ', 'A participant in my class cannot get into their CCO Learning. ', 'A participant in my class cannot log into their CCO Learning ', 'A trainee in my class cannot access their CCO Learning. ', 'A trainee in my class cannot get into their CCO Learning.  ', 'A trainee in my class cannot log into their CCO Learning. ', 'How do I submit a ticket to report my trainee in class cannot get into CCO Learning? ', 'Who do I reach out to if a trainee cannot access CCO Learning?'], 'responses': ['Refer to [IT Service Desk](https://itservicedesk.maximus.com/CherwellPortal/IT?_=422bffd2) complete and submit a ticket reporting CCO Learning issue. '], 'context_set': ''}
{'tag': 'Q1', 'patterns': ['A CSR in my class locked themselves out of their app

## Step 2
Now that we have our data lets orginize it so that we have all our intent questions and answers alined 

In [2]:
questions =[]
answers = []

for i in data['intents']:
    questions.append(i['patterns'])
    answers.append(i['responses'][0])
    
print(questions)
print('')
print('')
print(answers)

[['A CSR in my class cannot access their CCO Learning ', 'A CSR in my class cannot get into their CCO Learning. ', 'A CSR in my class cannot log into their CCO Learning ', 'A participant in my class cannot access their CCO Learning ', 'A participant in my class cannot get into their CCO Learning. ', 'A participant in my class cannot log into their CCO Learning ', 'A trainee in my class cannot access their CCO Learning. ', 'A trainee in my class cannot get into their CCO Learning.  ', 'A trainee in my class cannot log into their CCO Learning. ', 'How do I submit a ticket to report my trainee in class cannot get into CCO Learning? ', 'Who do I reach out to if a trainee cannot access CCO Learning?'], ['A CSR in my class locked themselves out of their application.  How will they complete the daily health assessment? ', 'How to handle the clear2work app if a trainee gets locked out of the app?', 'One of my new hires is locked out of the Health Assessment application. Who should I notify to 

## Step 3
Now that we have our data set in two simple lists we can run comparisons on a question type versus intents and return the answer that makes the most sence given BERTS understanding of our target question.

First we will set up the BERT text similarity tracker - 

In [3]:
#use this command to install sentence transformers if you need them 
#!pip install sentence-transformers

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('bert-base-nli-mean-tokens')


## Step 4
Now that we have that set up build a method that can take in the data and map comparisons 

In [10]:
def get_similaritys(text, comparison_list, index):
    
    sentences = [text] + comparison_list    
    sentence_embeddings = model.encode(sentences)
    sentence_embeddings.shape
    
    #Let's calculate cosine similarity for sentence 0:
    result=cosine_similarity([sentence_embeddings[0]],sentence_embeddings[1:])
    
    Max = max(result[0])
    
    if(Max > 0.95):
        return ['Bingo',index]  
    return [Max, index]
        

In [11]:
# test the method 
targets =['this is a good dog.', 'the cat is free.', 'who could have predicted this?','time is older than us']
text='i think dogs are cool'

result = get_similaritys(text, targets, 0)

print(result)

[0.89871323, 0]


In [21]:
questions1 = [['this is a good dog.', 'the dog is great'],['this cat is free','a free cat walks free'],['who predicted this?'],['time is old','time is older than us']]
answers1 =['indeed a good dog','free cats for all','not me thats for sure ', 'yes and we all should feel time is the master']


## Step 5
Now that we have a comparison method we can build a central model input method that will respond to a question with a matching answer or if none seem likely from the question a simple sorry please repeat that I do not understand.

In [12]:
def qa_model_input(text, questions1, answers1):
    index =-1
    targets =[]
    for x in questions1:
        index +=1
        result = get_similaritys(text, x, 0)
        if(result[0]=='Bingo'):
            return answers1[index]
        else:
            targets.append(result)
    max_value = max(targets) 
    if(max_value[0] < 0.9):
        return("Sorry I do not understand the question, can you please try asking again.")
    max_index = targets.index(max_value)
    return answers1[index] 

In [7]:
# Test run
run = True
while run:
    val = input("Hey Im a chat bot what can I help you with today : ")
    if(val =='/exit' or val == 'no'):
        print('good bye lets talk again soon!')
        break
    
    result = qa_model_input(val, questions1, answers1)
    print('Question : \n')
    print(val)
    print('Answer : \n')
    print(result)
    print('\n')
    print('Great question care to ask another ?')
    print('\n')
    

Hey Im a chat bot what can I help you with today : /exit
good bye lets talk again soon!


In [8]:
## max wait time is 4 to 6 seconds currently
run = True
while run:
    val = input("Hello Im the Maxmimus help desk chat bot what can I help you with today : ")
    if(val =='/exit' or val == 'no'):
        print('good bye lets talk again soon!')
        break
    
    result = qa_model_input(val, questions, answers)
    print('Question : \n')
    print(val)
    print('Answer : \n')
    print(result)
    print('\n')
    print('Great question care to ask another ?')
    print('\n')

Hello Im the Maxmimus help desk chat bot what can I help you with today : /exit
good bye lets talk again soon!


In [6]:
# result = qa_model_input(val, questions, answers)

# set up parrot to do parra phrases 
# Import libraries
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")
print('parrot loaded ready to work')


parrot loaded ready to work


In [7]:
##Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

In [9]:
#build compresed method 
def get_para(phrase, num, length):
    try:
        texts = []
        para_phrases = parrot.augment(input_phrase=phrase, use_gpu=False, diversity_ranker='levenshtein', do_diverse=False,
                                    max_return_phrases=num, max_length=length, adequacy_threshold=0.9, fluency_threshold=0.9)
        for para_phrase in para_phrases:
            text = para_phrase[0]
            texts.append(text)

        return texts
    except:
        return[phrase]
        

text = 'Whats the most delicious papayas?'
texts = get_para(text, 10, len(text)+5)
print(texts)

['tell me the best and best papaya i have ever eaten?', 'tell me the best papaya to eat?', "what's the best papaya to eat?", "what's the best papaya?", 'which is the best tasting papaya?', 'which is the most delicious papaya ever?', 'show some of the most delicious papayas?', 'can you list the most delicious papayas?']


In [29]:
targets_set =[["a csr in my class can't access their cco learning account", "a csr in my class can't access their cco learning stream", "a csr in my class can't access their cco learning", 'A CSR in my class cannot access their CCO Learning '],
['A CSR in my class cannot get into their CCO Learning. ', 'A CSR in my class cannot get into their CCO Learning. '],
['a csr in my class cannot log into their cco learning oriented courses', "a csr in my class can't log into their cco learning - system", "a csr in my class can't log into their cco learning account", 'a csr in my class cannot log into their learning cct', 'a csr in my class cannot log in to their cco learning', 'a csr in my class can not log into their cco learning', 'A CSR in my class cannot log into their CCO Learning ']	,
['a participant in my class cannot access their cco learning - resources', 'a participant in my class cannot access their cco learning - content', 'a participant in my class cannot access their cco learning materials', 'a participant in my class cannot access the cco learning app', 'a participant in my class cannot access his cco learning', "a participant in my class can't access their cco learning -", "a participant in my class can't access their cco learning", 'A participant in my class cannot access their CCO Learning ']	,
["in my class there's no way for a participant to enter cco learning", "a participant in my class can't get into cco learning", 'a participant in my class cannot access their cco learning', 'a participant in my class cannot get into cco learning', 'a participant in my class cannot join their cco learning', 'a participant in my class cannot get into my cco learning', 'A participant in my class cannot get into their CCO Learning. ']	,
["a participant in my class can't log into their cco learning registrar", 'a participant in my class can not log into their cco learning program', 'a participant in my class cannot log in to their cco learning account', 'a participant in my class cannot login to cco learning', 'a participant in my class cannot log into their cco learning account', "a participant in my class can't log into their cco learning", 'A participant in my class cannot log into their CCO Learning ']	,
['a student in my class cannot access their cco learning', 'a trainee in my class cannot access their cco learning', 'a trainee in my class cannot access their cco learning ', 'A trainee in my class cannot access their CCO Learning. ']	,
['the student in my class cannot get into cco learning', 'a trainee in my class cannot go to cco learning', 'a trainee in my class can not enter cco learning', "a trainee in my class can't get into cco learning", 'a trainee in my class cannot get into cco learning', 'a trainee in my class cannot get into my cco learning', 'A trainee in my class cannot get into their CCO Learning.  ']	,
["a trainee in my class can't log into cco learning", 'a trainee in my class can not log into their cco learning account', 'a trainee in my class cannot log into their cco learning program', 'a trainee in my class cannot log into cco learning', 'a trainee in my class cannot log into their cco learning system', "a trainee in my class can't log into their cco learning", 'A trainee in my class cannot log into their CCO Learning. ']	,
['how can i file a ticket when my trainee fails to get cco learning?', 'How do I submit a ticket to report my trainee in class cannot get into CCO Learning? ']	,
['who can i contact if a trainee has difficulty with cco learning?', 'who can i contact if i cannot find cco learning?', "who can i call if i can't access cco learning?", "how do i contact someone if i can't access cco learning?", 'who can i reach out to if i am not connected with cco learning?', "who do you contact if a trainee can't access cco learning?", "who should i reach out to if i can't access cco learning?", 'Who do I reach out to if a trainee cannot access CCO Learning?']	,
['what will be the daily health assessment?', 'how do i get csrs to participate in daily health assessment?', 'how do i get the student to complete their health assessment?', 'A CSR in my class locked themselves out of their application.  How will they complete the daily health assessment? ']	,
['how do i get out of clear2work without an app lock?', 'how do i find out a trainee is locked out of the clear2work app?', 'how do i get clear2work out of the app?', 'How to handle the clear2work app if a trainee gets locked out of the app?']	,
['what should i do if one of my employees is locked out of the application for health assessment?', 'one of my new recruits is locked out of the health assessment application how can i get him back?', 'one of my new hires is locked out of my health assessment application who should i notify?', 'one of my new hires is currently locked out of my health assessment application who should i notify to get them unlocked?', 'one of my new hires was locked out of the health assessment application what should i notify to get them unlocked?', 'One of my new hires is locked out of the Health Assessment application. Who should I notify to get them unlocked? ']	,
['what should i do when i see that one of my participants has been locked out of my health assessment application?', 'one of my participants is locked out of the health assessment application who do i notify?', 'a participant is locked out of a health assessment application who should i inform to get them removed?', 'one of my participants is locked out of the health assessment application and what should i notify them about?', 'one of my participants has been locked out of the health assessment application who should i notify to get them unlocked? ', 'One of my participants is locked out of the Health Assessment application. Who should I notify to get them unlocked? ']	,
['why should i tell a trainee who is locked out of the health assessment application?', 'i have another trainee that is locked out of the health assessment application who should i contact to get them unlocked?', 'One of my trainees is locked out of the Health Assessment application. Who should I notify to get them unlocked? ']	,
['is there any way to notify a trainee that he is locked out of my training application?', 'how do i notify someone who lost their password in an app?', 'how do i notify my trainees when they leave my app?', 'how do i contact a trainee if he is locked out of the app?', 'how do i notify people who are locked out of my app?', 'can anyone contact me if one of my trainees is locked out of my app?', 'Who do I notify if one of my trainees is locked out of their app.?']	,
["how do i find out if my employee's time card was wrong?", 'if a csr has a timecard that is incorrect how can i correct it?', 'i quit the company what should i do to get my timecard corrected?', 'A CSR quit but their timecard is wrong.  How can I get their timecard corrected? ']	,
['how can we repair time cards?', 'how do i get my csr timecard corrected?', 'how can i get a csr to change their timecard?', 'A CSR resigned but their timecard is wrong.  How can I get their timecard corrected? ']	,
['how do i change a timecard on a csr?', 'how do i get a csr to correct a time card?', 'how do i get a csr with an incorrect timecard?', 'A CSR resigned but their timecard is wrong.  How can I get their timecard fixed? ']	,
["how can i fix my company's time card?", 'how can i fix an old timecard for a csr?', 'A CSR rquit but their timecard is wrong.  How can I get their timecard fixed? ']	,
['how do i rectify timecards?', "how can i get my new employee's timecard fixed?", 'how do i ask a new hire to correct his timecard?', 'A New Hire quit, but their timecard is wrong.   How can I get their timecard corrected? ']	,
['is there any way for a new hire to correct their time card?', 'can someone with an irregular timecard get that timecard back?', 'A New Hire quit, but their timecard is wrong.  How can I get their timecard fixed? ']	,
['is it possible to take a new hire off with a wrong time card?', 'i signed up for a company and they had time stamp errors what can i do for a company to fix this?', 'how do i correct a time card when a new hire resigned from the job?', 'how can i get a timecard corrected?', 'A New Hire resigned, but their timecard is wrong.   How can I get their timecard corrected? ']	,
['is it possible to get your time card correct?', 'is there a way to get a replacement timecard on my new job?', 'how do i get a timecard fixed?', 'how do i find a person with timecard problems?', "how do i get my new colleague's timecard fixed?", 'i left an employee with a timecard that was wrong how can i fix it?', 'A New Hire resigned, but their timecard is wrong.  How can I get their timecard fixed? ']	,
["what should i do if i'm tired and have a wrong time card?", 'how do you have the timecard corrected?', 'can i get someone to correct my timecard if i quit because their time card is wrong?', 'if a participant quits but the time card is wrong how can i fix it?', 'A participant quit, but their timecard is wrong.   How can I get their timecard corrected? ']	,
['how do i fix my timecard?', 'is there any way to get my timecard replaced?', 'how do you get your timecard fixed?', 'A participant quit, but their timecard is wrong.  How can I get their timecard fixed? ']	,
['how do i correct timecard errors?', 'how should i get the wrong timecard when a participant resigned?', 'A participant resigned, but their timecard is wrong.   How can I get their timecard corrected? ']	,
['can you give me some suggestions to help me get my timecard corrected?', "how do i get a participant's timecard fixed?", 'i have resigned and my timecard is wrong how can i get my timecard fixed?', 'A participant resigned, but their timecard is wrong.  How can I get their timecard fixed? ']	,
['can you check that your timecard is correct?', 'a trainee quit but his timecard is wrong how can i rectify that?', 'when a trainee quits with an incorrect timecard how can i get his timecard corrected?', 'A trainee quit but their timecard is wrong. How can I get their timecard corrected? ']	,
['how can i fix timecards?', 'can i have a failed timecard?', 'A trainee quit,  but their timecard is wrong. How can I get their timecard fixed? ']	,
['how can i rectify a timecard that was incorrect at the training?', 'can you have a timecard that is incorrect?', 'how do i get a trainee to correct a wrong time card?', 'how can a trainee be notified if his timecard is wrong?', 'A trainee resigned but their timecard is wrong. How can I get their timecard corrected? ']	,
['what should i do if my time card is missing or a trainee left my company after working and resigned?', 'what can i do if my trainee timecard is wrong?', 'how can i fix a time card for a trainee who resigned?', 'how can i fix timecard problems in a trainee job?', 'A trainee resigned but their timecard is wrong. How can I get their timecard fixed? ']	,
['how does a time card add time on an absent trainee?', "how can i get additional time added to a student's timecard?", "how can i get time on a absent trainee's timecard?", "how can i get the time on an absent trainee's timecard?", "How can I get time added on an absent trainee's timecard?"]	,
['what steps do i take when i submit an approval from deltek?', 'tell me the best way to submit a deltek approval request?', 'how do i submit an applicant to deltek approvals?', 'can i apply for a deltek approval?', 'how do i submit a deltek approved product?', 'How do I submit a Deltek Approval?']	,
['how do i get my timecard signed in to the proxy?', 'how can i get a timecard proxy signed?', 'how can i get my timecard proxy signed?', 'how do i get my timecard proxy signed?', 'i want to sign my timecard how do i do that?', "I need my trainee's timecard proxy signed.  How do I do that?"]	,
['the step-by-step guideline on how to fill in specific sections of the daily training report daily', 'this article offers a step-by-step guideline on how to fill in specific sections of the daily training report on the daily basis', 'A step-by-step guideline on how to fill in specific sections of the daily training report  on the daily basis.']	,
['a step-by-step guideline on how to fill in specific sections of the dtr everyday', 'a step-by-step guideline on how to fill in specific sections of the dtr daily', 'the step-by-step guideline on how to fill in specific sections of the dtr daily', 'a step-by-step guide on how to fill in specific sections of the dt on the daily basis', 'a step-by-step guideline on how to fill in specific sections of the dtrr on a daily basis', 'a step-by-step guideline on how to fill in specific sections of the dtr on the daily basis', 'A step-by-step guideline on how to fill in specific sections of the dtr on the daily basis.']	,
['how can i fill out a daily training report for training where information about the training portal is unavailable?', 'how do i fill out the daily training report when the data on the recruitment and onboarding portal are unavailable?', 'how do i fill out a daily training report when the recruitment and onboarding portal data is unavailable?', 'how do you fill out a daily training report when the candidate recruitment and onboarding portal data is not available?', 'how do you fill out a daily training report when the candidate recruitment and onboarding portal data is unavailable?', 'how do i fill out a daily training report when candidate recruitment and onboarding portal data is unavailable?', 'How do you correctly fill out a Daily Training Report when Candidate Recruitment and Onboarding Portal data is unavailable? ']	,
['how can i fill out a daily training report when there is no crop data available?', 'tell me the best way to fill out a daily training report when crop data is not available?', 'how do i fill out a daily training report when no crop data is available?', 'how do i fill out a daily training report when crop data is not available?', 'how can you fill out a daily training report when crop data is unavailable?', 'how do i fill out a daily training report when crop data is unavailable?', 'how do you fill out a daily training report when crop data is unavailable?', 'How do you correctly fill out a Daily Training Report when CROP data is unavailable? ']	,
['how should i fill out a training report?', 'how can i fill out a daiy training report?', 'how does one fill out a daiy training report?', 'how do you fill out a daiy training report?', 'How do you correctly fill out a Daiy Training Report? ']	,
['how do i fill out a dtr when there is no user data on the application portal?', 'how do i submit a dtr application when the data from the website is unavailable?', 'how can i fill out a dtr if the data from the candidate recruitment portal is not available?', 'how do i fill out a dtr when the candidate recruitment portal is unavailable?', 'how do you complete a dtr if the data in the candidate recruitment and onboarding portal is not available?', 'how do you fill out a dtr correctly when candidate recruitment and onboarding portal data is unavailable?', 'How do you correctly fill out a DTR when Candidate Recruitment and Onboarding Portal data is unavailable? ']	,
['is it possible to fill out crop data without the corresponding crop data?', "how can i fill a dtr form when the source isn't available?", 'how do you fill out a crop report when it is not available?', 'how do i fill out a dtr while my crop data is not available?', 'what does it take to fill out dtr when crip data is unavailable?', 'how do you fill out a dtr when crop data is not available?', 'how do you fill out a dtr when the crop data is unavailable?', 'how do i fill out a dtr when crop data is unavailable?', 'How do you correctly fill out a DTR when CROP data is unavailable? ']	,
['how can i fill out a dtr correctly?', 'how should i fill in a dtr?', 'how should i fill out a dtr?', 'How do you correctly fill out a DTR? ']	,
['how do i fill a daily training report?', 'how do i complete a training report when our data is not available in a candidate recruiting and operations portal?', 'how do i complete a training report when all data from the portal has not been given to us?', 'how do i fill out my daily training reports if the data from the candidate recruiting and operations portal is not provided?', 'How to correctly fill out a Daily Training Report when Candidate Recruiting and Operations Portal  data has not been given to us?']	,
["what should i do if i don't receive all the data and instructions for daily training reports?", 'how can i fill out a daily training report?', 'how do i fill out a training report with our crop data if the information is not provided to us?', 'how can i fill a daily training report without giving us any crop data?', 'how do i fill out a daily training report for a training program that has not yet been given to us?', 'how can i fill a daily training report if crop data is not available to us?', 'how do i fill in a daily training report when crop data is not given to us?', 'how do i fill out a daily training report when crop data has not been given to us?', 'How to correctly fill out a Daily Training Report when CROP data has not been given to us?']	,
['how can i fill out dtr?', 'how do you fill a dtr when crop data does not exist?', 'How to correctly fill out a DTR when CROP data has not been given to us?']	,
['a trainee has questions about how to fill out the tax forms', 'the trainee has questions about filling the form', 'a trainee has questions about filling out tax forms', 'a trainee has questions about completing tax forms', 'a trainee has questions about completing the tax forms', 'A trainee has questions about completing the tax forms. ']	,
['a trainee has questions about the completion of the form', 'a student asked questions about filling out the tax form', 'a trainee has questions about filling the tax form', 'a trainee has questions about filling out tax forms', 'a trainee has questions about filling out a tax form', 'a trainee has questions about filling out the tax form', 'the trainee has questions about filling out the tax form', 'A trainee has questions about filling out the tax form. ']	,
['how can we help our new hire who fills out a tax form?', 'what support can we give to a new hire to fill out a tax return?', 'what can i do for a new hire who is filling out the tax form?', 'what assistance can i give to a new hire filling the tax form?', 'what assistance will you give a new hire that is filling out the tax form?', 'what assistance can i give a new hire in filling out the tax form?', 'what assistance can we give a new hire who is filling out a tax form?', 'What assistance can we give to a new hire who is filling out the tax form? ']	,
['how do i help a tax returner on their tax return?', 'can we help a taxpayer to fill out a tax form?', "how can i help someone who's filling out a tax form?", 'how can we help with filling out the tax form?', 'what assist can you give to a taxpayer when they fill out tax return?', "what assistance do you give a participant who's filling out the tax form?", 'what assistance can we give to a participant who is completing the tax form?', 'What assistance can we give to a participant who is filling out the tax form? ']	,
['tell me the support we give to the tax-form filler?', 'can we help a trainee who fills out tax return forms?', 'how can we help trainees by filling up the tax form?', 'what support can we give to students who are filling out the tax form?', 'what help can we give to a trainee who is filling out tax forms?', 'what assistance can we give a trainee who is filling out the tax form?', 'What assistance can we give to a trainee who is filling out the tax form? ']	,
['how do i get a trainee to fill a tax withholding form?', 'can we help our trainees to fill out the tax withholding form?', 'what help can i give a trainee who is looking for help to fill out their tax withholding form?', 'what advice can i offer to an hrtax trainee in filling out the tax withholding form?', 'what help can we offer trainees to fill out the tax withholding form?', 'which help can we offer a trainee if he has questions about the tax withholding form?', 'What help can we offer to a trainee who has questions about filling out the tax withholding form? ']	,
['tell me the best resource to fill out maxnet tax forms?', 'how can i help a training student fill out the tax form on maxnet?', 'can you list the best tips to fill out tax forms on maxnet?', 'show some useful resources to help you fill out tax forms on maxnet?', 'how can we help our students fill out tax forms on maxnet?', 'tell me the best way to assist a trainee in filling out a tax form for maxnet?', 'what resources can be used to assist in filling out tax forms on maxnet?', 'What resource can we use to assist trainees in filling out tax forms on MaxNet?']	,
['what should i do if my cell phone went off in class?', 'how do i stop a student who went off their cell phone while in class?', 'what should a trainee do now when a cell phone goes off in class?', 'a trainee in my class had his cell phone off in class what do you think?', 'A trainee in my class had their cell phone go off in class, what now?']	,
['how do we know if a trainee has their cell phone vibrated?', 'a student in my class rang my cell phone and their phone vibrated', 'what should i do if my cell phone vibrates in class?', 'when the trainee had his cell phone ring in class what would he do now?', 'A trainee in my class had their cell phone ring/vibrate in class, what now? ']	,
['can i have my cell phone in the training room?', 'do students have to have a cell phone in the training room?', 'can a trainee have their cell phone in the training room?', "What if a trainee doesn't have a locker, can they have their cell phone in the training room?"]	,
['what is a secure floor policy? what is the definition of security floor policies?', 'what is a secure floor policy? what are the details?', 'show the key aspects of an effective floor plan?', 'tell me the security floor policy?', 'what is safe floor policy?', 'tell me the secure floor policy?', 'what is secure floor policy?', 'what is a secure floor policy?', 'What is the Secure Floor Policy?']	,
['do i get the right training daily?', 'how can i correct my daily training report?', 'how to complete a daily training report?', 'are i completing my daily training reports?', 'have you done the daily training report correctly?', 'am i completing my daily training report correctly?', 'Am I completing Daily Training Report correctly?']	,
['do i need to read a daily training report if it is done right?', 'are i taking a daily training report?', 'do i have to complete the daily training report?', 'Am I completing Daily Training Report right?']	,
['is dtr compliant?', 'is completing dtr right?', 'am i completing dtr correctly?', 'Am I completing DTR correctly?']	,
["i'm completing dtr right?", 'Am I completing DTR right? ']	,
['how can i get more benefits from daily training reports?', 'do i make daily training reports?', 'is my daily training report good?', 'do i write daily training reports correctly?', 'do i do daily training reports correctly?', 'am i doing my daily training reports correctly?', 'Am I doing Daily Training Report correctly?']	,
['is my daily training report accurate?', 'do i do a daily training report correctly?', 'is there a daily training report?', 'do i do my daily training report?', 'am i completing a daily training report right?', 'am i doing the daily training report right?', 'Am I doing Daily Training Report right?']	,
['tell me the correct way to do dtr?', 'can i follow dtr correctly?', 'do i do the dtr correctly?', 'do i do dtr correctly?', 'Am I doing DTR correctly?']	,
['is it all right to do a drv with dtr?', 'is it right to do dtr?', 'Am I doing DTR right? ']	,
['does daily training report format work?', 'do i fill in the daily training report correctly?', 'are we filling the daily training report correctly?', 'am i filling in the daily training report correctly?', 'are i filling out the daily training report correctly?', 'Am I filling out the Daily Training Report correctly?']	,
['do i need to fill out my training report?', 'can i fill out the daily training report?', 'does that mean i fill out a daily training report right?', 'am i filling the daily training report right?', 'do i have to fill out the daily training report right?', 'am i filling out the daily training report right?', 'Am I filling out the Daily Training Report right?']	,
['is it true that i have correctly completed my dtr?', 'how can i answer questions on my dtr form?', 'do i fill in a dtr correctly?', 'how should i fill out the dtr properly?', 'can i fill the dtr correctly?', 'how do i fill out the dtr correctly?', 'Am I filling out the DTR correctly? ']	,
["what's the process for completing the dtr?", 'do i need to complete dtr?', 'Am I filling out the DTR right?  ']	,
['show the best ways to fill out dtr?', 'how can i fill the dtr?', 'how should i fill up the dtr?', 'How do I fill out the DTR?']	,
['how do i prepare a daily training report?', 'how do i fill out my daily training report?', 'tell me the best way to fill out the daily training report?', 'how should i fill out the daily training report?', 'how do i fill out the training report correctly?', 'I need to fill out the Daily Training Report. How do I do that correctly?']	,
['how can i fill out dtr?', 'what should i do to fill out dtr?', 'how do i fill out the dtr?', 'how should i fill out the dtr form?', 'how can i fill out dtr correctly?', 'I need to fill out the DTR.  How do I do that correctly?']	,
['when i train for a job do i have to take calls and be on the phone during class?', 'should i take calls during training?', 'is it compulsory to take calls during training?', 'is it required for me to take calls during training?', 'is it true that you have to take calls while working on classes?', 'are i required to make a call during training?', 'do i have to take calls when i train for a class?', "how are you required to take calls while you're in class?", 'am i required to take calls while training a class?', 'Am I required to take calls while I train a class?']	,
['how many hours can i take each month?', 'how many hours of calls can i take in a month?', 'how many hours do i have to take each month?', 'how many hours of calls do i need to take each month?', 'how many hours of calls do i have to take every month?', 'how many hours of call do i have to take each month?', 'how many hours of calls do i have to take each month?', 'How many hours of calls do I have to take each month? ']	,
['what should be the priority on day 1?', 'what is your priority in day 1 of nesting?', 'what are the priorities of day 1 of nesting?', 'what is my priority on day 1 of nesting?', 'what are your priorities for day 1 nesting?', 'And what is priority for Day 1 Nesting?']	,
['how can i access the nesting agenda?', 'How do we access the Nesting Agenda? ']	,
['what will be done on the day of nisting?', "what's the work that will be completed by nesting?", "what's the procedure for day 1 of nesting?", 'what should i do on day one of nesting?', 'what should we complete during day one of nesting?', 'what is to be done from day one of nesting?', 'what tasks are to be completed on day 1 of nesting?', 'what needs to be done on day one of nesting?', 'What needs to be completed on day one of Nesting?']	,
['where is the nesting agenda located?', 'where is the nesting agenda?', 'how can i get the nesting agenda?', 'where can i get my nesting agenda?', 'where can i find a nesting agenda?', 'where can i find the nesting agenda?', 'Where can I find the Nesting Agenda?']	,
['can i bring cco learning courses into my classes?', 'list the steps to upload my cco learning courses to my training classes?', 'what steps should i follow to load cco learning courses to my training class?', 'is there a way to load cco learning courses to my training class?', 'Are there a list of steps to load the courses in CCO Learning to my training class? ']	,
['how should one load cco learning material for a group session?', 'show some steps i should follow when bringing the course materials to a cco learning course?', 'how do you load cco learning materials into my online training classes?', 'can i load cco learning materials for my training class?', 'how do i load cco learning materials to my training class?', 'Are there a list of steps to load the materials in CCO Learning to my training class?']	,
['how can i assign courses to my classes at cco learning?', 'how can i assign courses to my class through cco learning?', 'how do i assign courses to my classes through cco learning?', 'how do i assign cco learning courses?', 'how do i assign courses to my classes?', 'how can i assign classes through cco learning to my classes?', 'how can i assign courses through cco learning to my class?', 'How do I assign courses through CCO Learning to my class? ']	,
['can i assign cco learning material?', 'how do i assign cco learning material to the class?', 'how do i assign cco-learned content to a class?', 'how do i assign material to my class through cco learning?', 'how do i assign material to the class?', 'how do i assign the material to a cco learning class?', 'how do you assign material through cco learning to your class?', 'How do I assign material through CCO Learning to my class?']	,
['how do i assign the curriculum for the cco learning classes?', 'how can i assign the curriculum to my cco learning class?', 'tell me the best way to assign the curriculum to a class in cco learning?', 'how do i assign a curriculum to my class at cco learning?', 'How do I assign the curriculum to my class in CCO Learning?']	,
['how do i know that a class has a curriculum for her class?', 'how do you get the curriculum for your classroom?', "how do i get my school's curriculum assigned?", 'how do i receive curriculum assignments for my class?', 'tell me the best way to get the curriculum assigned to my class?', 'how can i get a curriculum assigned to my class?', 'how do you get curriculum assigned to a class?', 'how do i get the curriculum assigned to my class?', 'How do I get curriculum assigned to my class? ']	,
['how do i make cco learning more efficient?', 'how do i introduce curriculum to cco learning?', 'how can i push the curriculum of cco learning?', 'How do I push curriculum in CCO Learning?']	,
['how can i reach the cco learning library for my lesson?', 'is it possible to push out a lesson in my class using cco learning?', 'how do i reach more people through cco learning?', 'what is a way to bring your lessons to a class by cco learning?', 'how do i push lectures out to my class through cco learning?', 'How do I push lessons out to my class through CCO Learning? ']	,
['tell me the best way to push a course in cco learning?', 'what should i do to push out training courses in cco learning?', 'how can i push out courses in cco learning?', 'How do I push out courses in CCO Learning?']	,
['how do you get the materials out to your students?', 'how do i bring material to the class?', 'how do i get materials out in classes?', 'how do i get my material to my class?', 'how do i make sure i get these materials out to my class?', 'how can i push materials out to my class?', 'How do I push the materials out to my class?']	,
['how do you load the curriculum for cco learning?', 'How do you load the curriculum for the class in CCO Learning?']	,
['how can you push a cco course once the online registration is completed?', 'how do you push cco courses once you have access?', 'How push courses on CCO once there is access?']	,
['how should i assign new hires to cco learning?', 'how do i assign new hires to cco learning?', 'how do you assign new hires on cco learning?', 'how do i assign new hires in cco learning?', 'how do i assign cco learning for my newly recruited staff?', 'how do i assign cco learning for my new hires?', 'How to assign on CCO Learning for your assigned new hires? ']	,
['how do i assign a participant to cco learning?', 'how do i assign a participle to cco learning?', 'how do i assign participants on cco learning to their assigned team?', 'tell me the best way to assign cco learning to your assigned participer?', 'How to assign on CCO Learning for your assigned particpants? ']	,
['how do you assign trainees to cco learning?', 'how do i assign cco training trainees for cco teaching?', 'how do you assign cco learning trainees to them?', 'how do i assign a cco learning course for a trainee?', 'How to assign on CCO Learning for your assigned trainees? ']	,
['how do i assign cco learning to the students assigned to me?', 'what should i do when attempting to assign and push cco learning to my assigned participants?', 'how do i assign cco learning to participants?', 'How to assign/push on CCO Learning for your assigned participants? ']	,
['how do you get new employees to push out for cco learning?', 'what i should do for cco learning?', 'tell me the best way to promote cco learning for your new hires?', 'how can i push cco learning to your new hires?', 'How to push out on CCO Learning for your assigned new hires?']	,
['what should i do if i have an instructor push me on cco learning for my assigned participants?', 'tell me the best way to push out cco learning for your assigned participants?', 'How to push out on CCO Learning for your assigned participants?']	,
['tell me the best way to push out a trainee?', 'is it possible to push out on cco learning to your assigned trainees?', 'How to push out on CCO Learning for your assigned trainees?']	,
['is there any way to pull out or assign a trainee to cco learning?', 'how do you assign your trainees on cco learning?', 'how do you assign your trainees to cco learning?', 'how do you push out your trainees?', 'How to push out/assign on CCO Learning for your assigned trainees?']	,
['how do i get materials to the school?', 'how can i get new material for a class?', 'how can i get my materials out of class?', 'how do i get materials out of my classes?', 'how do you get the materials to my class?', 'how do i get the materials to my class?', 'how can i get the materials out of class?', 'I need to push the materials out to my class.  How do I do that?']	,
['can you list the best ways to get materials to students?', 'what should i do to get materials in class?', 'tell me the best way to push materials into my class?', 'how do i move the material to a class?', "show me how i'm putting my materials out to the class?", 'how do i pass the materials to my class?', "show me how i've pushed materials out in a class?", 'show me how i push the materials in the class?', 'show me how to push the material out to my class?', 'show me how i push the material out to my class?', 'Show me how I push the materials out to my class?']	,
['what steps must i take to assign classes?', 'what steps must i follow in order to assign materials to classes?', 'how do i assign materials for a class?', 'how can we assign materials to our class?', 'how do you assign materials to a class?', 'how should i assign materials in my class?', 'how can i assign material to my class?', 'how do i assign materials to my class?', 'What are the steps to assign the materials to my class?']	,
['what should i do when i am creating a lesson plan?', 'how do you assign materials in a classroom?', 'tell me the best way to allocate the material to classes?', 'how do i assign material in a class?', 'what should i do to assign materials to my class?', 'how do i assign the materials for my class?', 'list the best steps to assign the materials for my class?', 'what steps can i take to assign the material for my class?', 'What steps do I take to assign the materials for my class? ']	,
['how can i improve the cco learning experience?', 'how can i push a cco course for a class?', 'show some ways you can push a student to cco learning?', 'how do i push cco courses into my class?', 'how do you push your courses on cco learning into the classroom?', 'what steps should i take to push cco learning into my class?', 'what steps should i take to introduce cco learning to my class?', 'What steps do I take to push cources on CCO Learning to my class?']	,
['tell me the difference between the deltek charge codes on days 1 and 2?', 'tell me the best deltek charge codes we should use on days 1 and 2?', 'what are some different deceltek charge codes to use on days 1 and 2?', 'list the different deltek charge codes we should use on days 1 and 2?', "list the different deltek charge codes we're supposed to use at day 1 and day 2?", 'are there different deltek charge codes we should use on days 1 and 2?', 'are there different deltek charge codes we are supposed to use on days 1 and 2?', 'Are there different deltek charge codes we are supposed to use on days 1 and 2? ']	,
['do the deltek project codes apply on days 1 and 2?', 'tell me the different deltek project codes we should use on day 1 and day 2?', 'recommend some different deltek project codes we should use on days 1 and 2?', 'can you suggest the different deltek project code we are supposed to use on days 1 and 2?', 'are there different deltek project codes we are supposed to use on days 1 and 2?', 'Are there different Deltek project codes we are supposed to use on days 1 and 2? ']	,
['list the best deltek charge codes?', 'What charge codes in Deltek should we be using on day 1 and day 2? ']	,
['What charge codes in Deltek should we be using on day one and day two? ', 'What charge codes in Deltek should we be using on day one and day two? ']	,
['how do i find the appropriate deltek code for new employees for day 1 and day 2?', 'where can i find the proper deltek id codes for new hires for the first and second day?', 'how do i find the correct deltek codes for new hires for day 1 and day 2?', 'how can i find the correct deltek codes for new hires on day 1 and day 2?', 'where can i find the correct deltek codes for new hires for day 1 and day 2?', 'Where can I locate the proper Deltek codes for new hires for Day 1 and Day 2?']	,
['where can i find the training charge codes?', 'how do you find charging codes for training?', 'where do you find charging codes used in training?', 'where can i find charging codes used for training?', 'how can you find the charge codes used in training?', 'how do i find the charge codes used for training?', 'where can i find the charge codes used for training?', 'Where do we find the charge codes used for training? ']	,
['how do i finish a class daily?', 'can i submit any daily requirments for class?', 'what are the daily requirements for my classes?', 'is there anything i can do every day in a class?', 'is there any requirement i must do every day in my classes?', 'which are the daily requirements in order to complete a class?', 'what are requirements i must complete daily when i have a class?', 'are there daily requirements i need to complete when i have a class?', 'Are there requirements I need to complete daily when I have a class? ']	,
['is there daily work to do in a class?', 'list the responsibilities in a class that i need to complete daily?', 'can i be expected to complete daily responsibilities when i have a class?', 'what responsibility do i have to complete every day for a class?', 'show the responsibilities i need to complete daily during a class?', 'what responsibilities i have to complete daily when i have a class?', 'Are there responsibilities I need to complete daily when I have a class? ']	,
['do i have to complete everyday things for my class?', 'how many things do i have to do daily after a lecture?', 'is there anything i have to do daily to be in a class?', 'what things are required to do every day in a specialized class?', 'recommend some of the things that i have to do on a daily basis when i have a class?', "what are things i have to complete daily when i'm in classes?", 'list the things that i have to complete daily when i have a class?', 'list the things i have to complete daily when i have a class?', 'Are there things I am required to complete daily when I have a class?']	,
['how can i work on something for a class when i have to do something daily?', 'what things do i need to do daily?', "how do i keep my hands off things i should complete every day when i'm in a class?", 'what should i do everyday when i am in class?', 'what are things i have to do on a daily basis?', 'show the things i have to complete every day?', 'what should i do daily when i have a class?', 'list the things i have to do daily when i am in a class?', 'can i do anything i need to do daily when i have a class?', 'Are there things I need to complete daily when I have a class?']	,
['is there a daily check list for teaching?', 'tell me the routine that should be done daily for a teacher?', 'list the daily tasks i have to complete to teach a class?', 'do i have to complete daily tasks to teach a class?', 'can you list the things we need to do daily when we teach?', 'what list do i need to complete daily when i teach a class?', 'is there a check list of things to do daily when teaching a class?', 'is there a check list of things i have to complete daily in order to teach a class?', 'is there a check list i need to complete daily when i teach a class?', 'Is there a check list of things I need to complete daily when I am teaching a class?']	,
['list some of the tasks i have to do during a class?', 'list the responsibilities of a student while in school?', 'list the duties i must fulfill for a class?', 'tell me the responsibility of life while teaching classes?', "what are my everyday responsibilities while i'm in the classroom?", 'how can you give me daily responsibilities while you are taking a class?', 'what are my daily responsibilities while doing my class?', 'What are my daily responsibilities while I have a class?']	,
['what should i be focusing on daily in a class?', 'what should i be doing every day to make my class?', 'what things must i do everyday to teach?', 'tell me the thing i should keep doing in class everyday?', 'What are the things I need to make sure to do everyday for my class everyday?']	,
['what should i do daily when i have a class in session? a good way to solve this problem?', 'what do i do daily when i am taking a class?', 'what should i do daily to complete classes?', 'what should i complete daily in a class?', 'what i should do daily when i have a class in session?', 'what should i complete daily when i have a class in session?', 'What should I complete daily when I have a class in session? ']	,
['can you list the rules for the training room?', 'list the rules i can post on my training room?', 'is there any specific rule i should use in my training room?', 'is there any rule that i should post in my training room?', 'what rules i should post in my training room?', 'list the rules i should post in my training room?', 'Are there rules I should post in my training room? ']	,
['can you list the road rules for training?', 'what should i take when going to training class?', 'show the rules for a training course?', 'what rules are needed for a training class?', 'tell me the rules of the road for training classes?', 'Are there rules of the road for training class? ']	,
['list some road rules i need to post in my training room?', 'Are there rules of the road I should post in my training room? ']	,
['how can i learn the fundamental rules of a fitness training class?', 'is there a specific ground rule in a class training?', 'tell me the ground rules for training?', 'are there any clear ground rules for training?', 'is there a set ground rules for training?', 'tell me the ground rules for training a class?', 'is there any set ground rules for the training class?', 'Are there set ground rules for training class? ']	,
['is there a standard rule that i need to follow for training and what exactly are the bases for this?', 'are there ground rules that should be posted in training rooms?', 'is there a set of rules that i should post to my training room?', 'list the ground rules i should post in my training room?', 'are there any special ground rules i should post in my training room?', 'is there any ground rules i should post in my training room?', 'Are there set ground rules I should post in my training room? ']	,
['give me some rules', 'give me some basic rules', 'give me some ground rules', 'Give me some ground rules. ']	,
['what ground rules should we include in everything we do?', 'list the rules for the ground game?', 'recommend some ground rules to follow?', 'which ground rules should i always follow?', 'can you list the ground rules i should follow?', 'list some ground rules i should always follow?', "what's the best ground rule that you should always incorporate?", 'What are some ground rules that I should always include?']	,
['list the basic rules for classes?', 'show the basic rules of class?', 'list the ground rules of class?', 'show the ground rules for a class?', 'list the ground rules for a class?', 'tell me the ground rules of a class?', 'tell me the ground rules for a class?', 'show the ground rules for class?', 'tell me the ground rule for class?', 'What are the ground rules for class? ']	,
['how can i create a daily training report for a nesting poc?', 'how do i create a daily training report for nesting pocs?', 'how do i create a nesting poc?', 'is it possible to create the daily nesting training reports?', 'tell me the best way to produce a daily nesting training report?', 'how do i create a daily nesting training report?', 'how do i create the daily training report?', 'how can i create a nesting daily training report?', 'how do i create a nesting daily training report?', 'As a Nesting POC, how do I create the Nesting Daily Training Report?']	,
['is there a way to create a nesting dtr?', 'how do i create a nesting dtr?', 'how do i create the nesting dtr?', "what's the best way to create the nesting dtr?", 'As a Nesting POC, how do I create the Nesting DTR?']	,
["what's the best way to create a daily training report for nesting as a nesting point of contact?", 'what do i do as my nesting point of contact?', 'how do i create a daily nesting training report?', 'how can i create a daily nesting training report?', 'how do i create daily nesting training reports?', 'how do i create the daily nesting training report?', 'how do i create a nesting training report? ', 'how do i create a nesting daily training report?', 'how do i create the nesting daily training report?', 'As a Nesting Point of Contact, how do I create the Nesting Daily Training Report? ']	,
['how do i create a dtr for nesting?', 'how do i create a dtr?', 'how do you create a nesting dtr?', 'how do i create a nesting point of contact?', 'how do i use a nesting point of contact to create the nesting dtr?', 'As A Nesting Point of Contact, how do I create the Nesting DTR? ']	,
['how do you build a daily nesting training report?', 'how do i create a daily nesting training report?', 'how do i generate a nesting training report?', 'how do i create a daily training report?', 'how do you write a nesting daily training report?', 'how do i create a nesting daily training report?', 'How do I create a Nesting Daily Training Report?']	,
['is it possible to create an encoding for a nesting dtr?', 'tell me the process of creating a nesting dtr?', 'tell me the best way to create a nesting dtr?', 'what do you do when creating a nesting dtr?', 'how can i create a nesting dtr?', 'how do you create a nesting dtr?', 'how do i create a nesting dtr?', 'How do I create a Nesting DTR?']	,
['how do i fill out a daily nesting training report?', 'how do you fill in a nesting daily training report?', 'how can i fill out a nesting daily training report?', 'how do i fill out a nesting daily training report?', 'How do I fill out a Nesting Daily Training Report? ']	,
['how do i fill out the ndtr for nesting?', 'what do i do if i fill out a nesting dtr?', 'how do i fill out a nesting dtr?', 'How do I fill out a Nesting DTR? ']	,
['is there any information in a daily nesting report?', 'how do i include the information in a daily nesting training report?', 'what information should i include in a daily nesting training report?', 'what should i include in my nipping daily training report?', 'which information is included in a nesting training report?', 'what information should i include in a nesting daily training report?', 'What information do I include in a Nesting Daily Training Report?']	,
['can i include the following information in my nesting dtr?', 'show the different types of information in a nesting dtr?', 'what should we include in a nesting dtr?', 'what can i include in my nesting dtr?', 'what information should be included in a nesting dt?', 'what information should i include in the nesting dtr?', 'what information do i include in the nesting dtr?', 'What information do I include in a Nesting DTR?']	,
['what information should i include in my dtr?', 'what information is included in nesting dtr?', 'which information should be included in nesting dtr?', 'what information should i include in my nesting dtr?', 'what information can i include in my nesting dtr?', 'tell me the information i need to include in my nesting dtr?', 'What information needs to be included in my Nesting DTR?']	,
['should i take courses at cco learning?', 'how do i assign the appropriate courses to cco learning?', 'Assigning the proper courses in CCO Learning? ']	,
['is the curriculum provided by cco learning the correct one?', 'how do i ensure that i am given the appropriate curriculum in cco learning?', 'list the necessary curriculum for cco learning?', 'how do i assign the curriculum for cco learning?', 'Assigning the proper curriculum in CCO Learning? ']	,
['how should i allocate learning materials in cco?', 'is it appropriate to assign appropriate learning materials in cco learning?', 'which materials are appropriate for cco learning?', 'are there any resources available in cco learning?', 'how should i provide the correct materials in cco learning?', 'how do i assign proper material for cco learning?', 'how can i assign the appropriate materials in cco learning?', 'Assigning the proper materials in CCO Learning? ']	,
['how can i push a course?', 'how can i push courses?', 'tell me the best way to go about deploying a cco?', 'how does one push a course online?', 'what courses should be completed once an individual applies to a cco?', 'what should i do for cco once the access is granted?', 'tell me the best way to push courses on cco?', 'list the right courses for cco once access is available?', 'How push courses and what are the proper courses to push on CCO once there is access?']	,
['how do i assign courses on cco learning once i have access to it?', 'how do i assign courses for cco learning once i get access?', 'tell me the best way to assign courses on cco learning once there is access?', 'how can i assign courses on cco learning once there is access?', 'How to assign the proper courses on CCO Learning once there is access? ']	,
['Can a New Hire have their Fit Bit in class? ', 'Can a New Hire have their Fit Bit in class? ']	,
['is it possible to have a new hire have their fit bit?', 'Can a New Hire have their Fit Bit in trainng? ']	,
['how can new hires track their fitness during class?', 'can a new hire wear a fitness tracker in class?', 'can a new hire have their fitness tracker in class?', 'can a new hire have their fitness tracker in class? ', 'Can a New Hire have their fitness tracker in class? ']	,
['can new hires have fitness trackers in training?', 'can a new hire have a fitness tracker in training?', 'Can a New Hire have their fitness tracker in training? ']	,
['Can a participant have their Fit Bit in class? ', 'Can a participant have their Fit Bit in class? ']	,
['what is a fit bit? how can a participant achieve it in a workout?', 'can a participant exercise their fit bit in training?', 'Can a participant have their Fit Bit in training? ']	,
['how can a participant track his fitness status and fitness in class?', 'can i have my fitness tracker in class?', 'can a participant have his or her fitness tracker in class?', 'can participants have their own fitness tracker in class?', 'can a participant have a fitness tracker in class?', 'can a participant have his fitness tracker in class?', 'can a participant have their fitness tracker in class?', 'Can a participant have their fitness tracker in class?  ']	,
['can someone have a fitness tracking device in their training session?', 'how can i track my fitness during workouts?', 'is it possible to use a fitness tracking device in training?', 'can a participant have a fitness tracker while he or she is doing training?', 'can you use a fitness tracker during training?', 'can a participant have their own fitness tracker in training?', 'Can a participant have their fitness tracker in training?  ']	,
['can a trainee exercise on a regular basis in class?', 'can you fit a trainee in class?', 'Can a trainee have their Fit Bit in class ?']	,
['can a trainee do it in class?', 'can you track your fitness tracker in class?', 'does a trainee have their personal fitness tracker in class?', 'can trainees have fitness trackers in class?', 'can a trainee have his fitness tracker in class?', 'Can a trainee have their fitness tracker in class? ']	,
['how is it possible for training staff to have a fitness tracker?', 'will fitness trackers help a trainee?', 'how can i track my fitness tracker while i train?', 'is it possible for a trainee to use their fitness tracker while in training?', 'can a trainee have a fitness tracker in training?', 'Can a trainee have their fitness tracker in training? ']	,
['how many persons in a class room are not allowed according to the secure floor policy?', "what can't be done in a class room according to the secure floor policy?", 'what is not allowed in a classroom according to secure floor policy?', "what isn't allowed in a class room according to the secure floor policy?", 'what is not allowed in the class room according to the secure floor policy?', 'what is not allowed in a classroom according to the secure floor policy?', 'what is not allowed in a class room according to the secure floor policy?', 'What is not allowed to have in a Class Room according the the Secure Floor Policy?']	,
['what is not allowed in a training room according to the security floor policy?', 'what is not allowed in a training room according to the secure floor policy?', 'what is not allowed to be in the training room according to the secure floor policy?', 'What is not allowed to have in a Training Room according the the Secure Floor Policy?']	,
['is there an easy way to update an it ticket?', 'how do i correct a ticket?', 'is there a way to correct an it ticket?', 'how can i correct an it ticket that i submitted?', 'can i correct an it ticket after it was submitted?', 'Can I correct an IT ticket after it was submitted?']	,
['how do you fix an it ticket?', 'what do i do to fix an it ticket?', 'how can i fix an it ticket?', 'can we fix a problem after it is submitted?', "is there a way to fix my it ticket after it's submitted?", 'can i fix an it ticket after it has been submitted?', 'how do you fix an iit ticket after it was submitted?', 'Can I fix an IT ticket after it was submitted? ']	,
['how do i fix an email that is submitted incorrectly by an it person?', 'how can i correct a ticket submitted to it?', 'can i correct a ticket i submitted to it?', 'can i correct an it ticket that was submitted incorrectly?', 'I submitted a ticket to IT incorrectly. Can I correct it?']	,
['how do i correct a ticket?', 'how do i correct my ticket after submitting it?', 'how do i correct a ticket after sending it?', 'is it possible to correct a ticket after the ticket has been submitted?', 'is there a way to correct a ticket?', 'how do i correct a ticket that was submitted?', 'is there a way to correct a ticket on my ticket?', "how do i correct a ticket after it's submitted?", 'is there a way to correct a ticket after it has been submitted?', 'Is there a way to correct a ticket after it was submitted?']	,
['what will happen to the ticket if i submit the wrong ticket?', 'what happens to a ticket i submit incorrectly?', 'what will happen if i do not submit a ticket correctly?', 'what should i do if i submit a ticket incorrectly?', 'what happens when i submit a ticket incorrectly?', 'what happens if i submit a ticket incorrectly?', 'What happens if I submit a ticket incorrectly?']	,
['Can IT set me up on the printer?', 'Can IT set me up on the printer?']	,
['how do i get the printer setup?', 'tell me the best way to get a printer?', 'how can i set up my printer?', 'how do i set up my printer?', 'How do I get set up to use the printer?']	,
['how do i get printer credentials for training?', 'How do I set up a printer credentials to be able to print items needed while training?']	,
['tell me the best way to get my printer set up?', 'how do you setup a printer?', 'how do i set up a printer?', 'how do i setup the printer?', 'How do I set up the printer?']	,
['how do i set up my printer?', 'how do i set up a printer?', 'is there any way to get a printer installed?', 'what can i do to setup my printer?', 'how can i get my printer set up?', 'I am not set up on the printer. How do I get set up? ']	,
["i need to print a milestone but my printer isn't set up", 'i need to print a milestone but the printer is not equipped', "i need to print a milestone but i don't have a printer", "i need to print a milestone but i don't have the printer", "i need to print a milestone but i'm not setting up the printer", 'I need to print a milestone but am not set up on the printer. ']	,
['how do i get my printer access?', 'how do i set up printer access?', 'who do i ask to get me printer access?', 'who do i ask to get me to set up printer access?', 'who do i ask to set me up with printer access?', 'Who do I ask to set me up with printer access?']	,
['do instructors have to use the calls that are on the instructor resource list for handiwork?', 'can we only use the calls that are in the instructor resource list for hands on?', 'Can we only use the calls that are on the Instructor Resource List for Hands on?']	,
['Can we only use the calls that are on the Training Resource for Hands on?', 'Can we only use the calls that are on the Training Resource for Hands on?']	,
['are there calls to play other than those in the lessons for hands-on activities?', 'Can we play calls other than those in the lessons for hands on activities? ']	,
['can we ask different things in the material?', 'Can we pull calls other than those in the materials for hands on activity? ']	,
['do we use calls only during lessons?', 'can we use the calls only in lessons?', 'do we use calls in the lesson?', 'do we only use the calls in the lessons?', 'Do we only use the calls in the lessons? ']	,
['how do you use different calls and not just ones in the instructor resource list?', 'Can we use different calls to listen during class not only the ones on the instructor resources list?']	,
['I am excellent on calls.  Can I play my own calls for my class? ', 'I am excellent on calls.  Can I play my own calls for my class? ']	,
['I found some great calls to play for class!', 'I found some great calls to play for class!']	,
['is there a way i can just play my calls?', 'if the materials are not as good as mine can i just play my calls?', 'The calls in the materials are not as good as mine.  Can I just play my calls? ']	,
['Class is over, but I still need to fill out my DTR. How long can I stay over? ', 'Class is over, but I still need to fill out my DTR. How long can I stay over? ']	,
['how early can i get in and out of a building before the school class starts?', 'tell me the best way to get to school early?', 'how early should i start the class?', 'how early should i come to class?', 'how early can i go to class before school starts?', 'how early can i arrive before class starts?', 'How early can I arrive before class starts?']	,
['how late should i be to get my homework done after class?', 'is it too late to have work done after class?', 'how late can i stay after class?', 'how late can i stay after classes?', 'how late can i stay after classes to finish my work?', 'how late can i stay after class to get my work done?', 'How late can I stay after class to get my work completed?']	,
['how long can i stay in my office after class?', 'how long should i stay after class for administrative work?', 'how long can i stay after class to finish administrative work?', 'how long can i stay after class in order to do administrative work?', 'how long can i stay after class to finish my administrative work?', 'how long do i have to stay after class to do my administrative work?', 'how long do i have to stay after a class to complete administrative work?', 'how long can i stay after class to complete my administrative work?', 'how long am i allowed to stay after class to complete administrative work?', 'How long am I allowed to stay after class to complete my administrative work? ']	,
['how long can i go to work before a class starts?', 'how much time does it take to come to work?', 'how long before the class starts can i go in?', 'how long before the class start can i go to work?', 'how long before the class starts can i go to work?', 'how long before the class starts can i come to work?', 'how long before class starts can i go to work?', 'how long before class starts can i come to work?', 'how long before class starts can i come in to work?', 'How long before class starts can I come in to work?']	,
['how many minutes do i need to make the first class?', 'how many minutes can i arrive before class?', 'how many minutes should i come in before class?', 'how many minutes can i come in before class?', 'how many minutes can i come in before the class starts?', 'how many minutes can i come in before class starts?', 'How many minutes can I come in before class starts?']	,
['can i come in early?', 'are i allowed to come in before classes?', 'how can i get in early for class?', 'does a person have to come early for a class?', 'is there any time i can come to school early?', 'is there any time for students to come in early for classes?', 'is there a time frame for coming in early for class?', 'is there any time i can come in early for class?', 'Is there an amount of time I can come in early for class?']	,
['will we be able to complete training every day?', 'do i need to complete the training report on a daily basis?', 'how can i complete the training report daily?', 'can i do a daily training report every day?', 'Do I complete the Daily Training Report every day? ']	,
['what should i do every day to complete the dtr?', 'will i ever complete the dtr every day?', 'Do I complete the DTR every day? ']	,
['do you fill out the daily training report every day in a classroom?', 'do i fill out the daily training report every day for classes?', 'can i fill out the daily training report every day we have a class?', 'do i fill out the daily training report every day we have a class?', 'Do I fill out the Daily Training Report every day we have class? ']	,
['do i have to submit a dtr every day for class?', 'do i have to submit a dtr every day we have class?', 'do i have to fill out the dtr every day we have classes?', 'do i fill out the dtr every day we have class?', 'Do I fill out the DTR every day we have class? ']	,
['do co-instructors need to fill in the daily training report?', 'do your co-instructors sign your daily training report?', 'how do i get my co-instructor to fill out the daily training report?', 'can my co-instructor fill out the daily training report?', 'does my co-instructor fill out the daily training report?', 'Does my co-instructor fill out the Daily Training Report? ']	,
['how does a co-instructor fill out a dtr form?', 'can my co-instructor fill out dtr?', 'does my co-instructor fill out dtr?', "does my co-instructor fill out the dtr? '", 'does my co-instructor fill out the dtr?', 'Does my co-instructor fill out the DTR? ']	,
['what do my coworkers do in their daily training reports?', 'do co-trainers fill out the daily training report?', 'does my coworker fill out the daily training report?', 'will my co-trainer fill out the daily training report?', 'does my co trainer fill out the daily training report?', 'Does my cotrainer fill out the Daily Training Report? ']	,
['is the co-trainer able to fill out the dtr for me?', 'do cotrainers fill out the dtr?', 'does my co-trainer fill in the dtr?', 'does my co-trainer fill out the dtr?', 'Does my cotrainer fill out the DTR? ']	,
['how often is the training report updated?', 'how often is it wise to update the training daily report?', 'how often is it recommended to update the training report?', 'how often can we update the daily training report?', 'how often should i update the daily training report?', 'how often is it necessary to update the daily training report?', 'how often are we supposed to update the training report?', 'How often are we supposed to update the Daily Training Report? ']	,
['how often do we update dtrs?', 'how often are we supposed to update the dtr?', 'How often are we supposed to update the DTR? ']	,
['why should i fill out dtr every day?', 'who is supposed to fill in dtr every day?', 'who is supposed to fill out dtr every day?', 'who is supposed to fill the dtr every day?', 'Who is supposed to fill out the DTR every day? ']	,
['should i fill out all the call cert forms?', 'do you need to fill out the entire call cert form?', 'do i need to fill out the whole call cert form?', 'Do I need to fill out all of the call cert form? ']	,
['do i have to fill out all the certification form in order to call the company?', 'do you have to fill in all of the call certification forms?', 'do i have to fill out all the call certification forms?', 'do i need to fill out the call certification form?', 'do i need to fill out all the call certification forms?', 'Do I need to fill out all of the call certification form? ']	,
['how do i fill a cert form?', 'how do i fill out cert forms?', 'how do i fill out a cert form?', 'How do I fill out a cert form? ']	,
['how do i fill out csr certification forms?', 'how do you fill out a csr certification form?', 'how do i fill out a csr certification form?', 'How do I fill out a CSR Certification Form? ']	,
['what should i do to fill out the cert cert?', 'how can i fill a call cert form?', 'how do i fill out a call cert form?', 'how do i fill out the call cert form?', 'How do I fill out the call cert form? ']	,
['how can i fill the call certification form for iicrc in a simple way?', 'how do i fill a call certification form?', 'how do i fill out the certification form?', 'how do i fill in the call certification form?', 'how do i fill out the call certification form?', 'How do I fill out the call certification form?']	,
['how can you fill a certfication form?', 'how can i fill out a certfication form?', 'how can i fill out the certification form?', 'How do you fill out the certfication form?']	,
['how can i fill a certification form for various certifications?', 'how can i fill out an application form for certification?', 'how do i complete the form of certification?', 'how can i fill up the forms of certifications?', 'how can i fill out a certification form?', 'how do i fill out the certification forms?', 'how do i fill out my certifications form?', 'how do i fill out a certification form?', 'How to fill out Certifications forms?']	,
['how do i complete an application for a certification?', 'how do you apply for the certification?', 'tell me the best way to prepare and mail a call certification form?', 'how can i complete the call certification form?', 'how do you complete the call certification form?', 'how do i complete the call certification form?', 'tell me the best way to complete the call certification form?', 'what are steps to complete the call certification form?', 'What steps do I take to complete the Call Certification Form? ']	,
['what should i do everyday as part of my job as an assistant teacher?', 'how can i manage my tasks outside of classroom?', 'how do you keep track of things you do outside of the classroom?', 'how can i plan my daily responsibilities outside the classroom?', 'do we have daily checklists to do outside of the classroom?', 'is there a daily checklist of responsibilities outside of school?', 'do we have a daily checklist of responsibilities outside of classroom as part of my job?', 'do i have a daily checklist of responsibilities outside of the classroom as part of my job?', 'do you have a daily list of responsibilities i have outside of the classroom as part of my job?', 'do we have a daily checklist of responsibilities we have outside of the classroom as part of my job?', 'Do we have a daily checklist of responsibilities I have outside of the classroom as part of my job?']	,
['what are your daily responsibilities?', 'how can you write a daily list of responsibilities?', 'tell me the best way to keep a daily responsibilities list?', 'how do i get a daily responsibilities list?', 'recommend some ways to keep your daily work responsibilities list?', 'how can i keep a daily work responsibilities list?', 'how do you keep a daily responsibilities list?', 'how do you keep a daily work-based responsibilities list?', 'how do you keep a daily work-based responsibilities list?']	,
['how do you know what you are supposed to be doing outside of school?', 'how do you know if you have responsibilities other than what you have in your classroom?', 'how do i understand what responsibilities i have outside of school?', 'how do i know if i have any responsibilities outside of the classroom?', 'how do i know what responsibilities i have inside and outside the classroom?', 'how do i know what my responsibilities are outside of a classroom?', 'how do i know what responsibilities i have outside the classroom?', 'how can i know what responsibilities i have outside of the classroom?', 'how do i know what responsibilities i have outside of the classroom?']	,
['does someone have a daily schedule outside the classroom?', 'do you have a daily checklist outside the classroom?', 'do you have a daily task list outside of the classroom?', 'do you have daily responsibilities outside of the classroom?', 'do you have a daily list of responsibilities beyond the classroom?', 'do you have a daily list of responsibilities outside of class?', 'do you have a daily list of responsibilities outside the classroom?', 'do you have a daily list of responsibilities outside of the classroom?', 'do you have a daily list of responsibilities outside of the classroom?']	,
['do we have daily tasks outside the classroom?', 'are we expected to have daily responsibilities outside of school for my job?', 'do we have daily responsibilities outside the classroom?', 'do we have daily lists of responsibilities outside of school?', 'do we need daily assignments outside of the classroom as part of our job?', 'do we have daily assignments outside the classroom as part of our job?', 'do we have daily lists of responsibility outside of class as part of our job?', 'do we have daily lists of responsibilities outside the classroom as part of our job?']	,
['does your job involve daily responsibilities outside of school?', 'do you have daily job responsibilities besides teaching the lesson?', 'do you have a daily list of responsibilities and responsibilities outside of the classroom?', 'have you given me a daily list of responsibilities and tasks i have outside the classroom?', 'do you have a daily list of responsibilities that i have outside my classroom?', 'do you have a daily list of responsibilities outside of the classroom?', 'do you have a daily list of things to do outside of the classroom as part of my job?', 'do you have any daily responsibilities i have outside my classroom as part of my job?', 'do you have a daily list of responsibilities outside of the classroom as part of my job?', 'do you have a daily list of responsibilities i have outside of classroom as part of my job?', 'do you have a daily list of responsibilities i have outside of the classroom as part of my job?']	,
['do new hires get time off?', 'how can new hires earn time off?', 'do new hires get paid time off?', 'do people in training get paid time off?', 'Does new hires in training get Earned Time Off? ']	,
['why does a new hire with limited experience get eto?', 'can trainees start with eto?', 'will new hires get eto?', 'does an new hire get an eto?', 'how are new hires covered in eto?', 'is new hires in training eligible for eto?', 'Does new hires in training get ETO?']	,
['how do you get the percentage off of a trainee?', "how do i know my trainee's earned time off during training?", "How do I figure my trainee's Earned Time Off while in training?"]	,
['tell me the logical way to calculate eto?', 'how does a trainee know their eto?', 'how do i learn eto for a trainee?', 'tell me the best way to know your eto while training?', "how can i figure out my trainee's eto while in training?", "How do I figure my trainee's ETO while in training?"]	,
['How do I submit Earned Time Off? ', 'How do I submit Earned Time Off? ']	,
['list some of the processes for submitting an eto?', 'what should i do to submit the eto?', 'How do I submit ETO?']	,
['what requirements are given to new employees to receive earned time off for perfect attendance?', 'tell me the minimum requirement for new hires to receive earned time off for perfect attendance?', 'which are the requirements for new hires to receive earned time off for perfect attendance?', 'What are the requirements for New Hires to receive Earned Time Off for perfect attendance? ']	,
['how do new hires receive eto?', 'how does a new hire get to receive eto?', 'can you describe the eligibility criteria for a new hire to receive eto?', 'which are the eto requirements of new hires?', 'tell me the eto requirement for new employees for perfect attendance?', 'can you list the requirements for new hires to receive eto for perfect attendance?', 'What are the requirements for New Hires to receive ETO for perfect attendance? ']	,
['who can i submit my time off to?', "who do i submit my 'time off' to?", 'who do i submit my earned time off to?', 'Who do I submit Earned Time Off to? ']	,
['who are the authorities responsible for this eto application?', 'what are your options in submitting an eto?', 'who does the eto submission go to?', 'tell me the best way to submit an eto?', 'who should submit eto?', 'who do i submit for the eto?', 'Who do I submit ETO to?']	,
['have you received feedback from me so far?', 'is there feedback for a site?', 'do you have feedback?', 'has someone received any feedback?', 'Have I received any feedback?']	,
['where is my score?', 'My coinstructor just completed an audit on me, where is my score?']	,
['My coinstructor just completed an observation on me, where is my score?', 'My coinstructor just completed an observation on me, where is my score?']	,
['My cotrainer just completed an audit on me, where is my score?', 'My cotrainer just completed an audit on me, where is my score?']	,
['my trainer just had a brief observation', 'my colleague just completed a little observation', 'my coach just did an observation on me', 'my co-trainer just finished an observation on me', 'my co-trainer just completed an observation about me', 'my trainer just completed an observation on me', 'My cotrainer just completed an observation on me. ']	,
['how does a training leader find his score?', "how can i find my training score after i've completed a class audit?", "if a training lead just completed an audit what's the best way to find out?", 'my training lead just completed a class audit where can i find my score?', 'My Training Lead just completed a class audit, where can I find my score?']	,
['i just finished my training where can i find my score?', 'my trainer just completed a observation what can i do to find my score?', 'My Training Lead just completed a observation, where can I find my score?']	,
['can i see my performance after completing a class audit?', "i've just completed an audit for a course how can i find my score?", 'my training manager just completed a class audit and was confused by my score where can i find it?', 'my training manager just completed a class audit where can i find my score?', 'My Training Manager just completed a class audit, where can I find my score?']	,
['how do i find my score?', 'how can i view a given score online for a particular training team?', 'how can i find my score when my training manager just finished a observation?', 'i am a trainer who just completed a visit where can i find my score?', 'my training manager just completed a survey where can i find my score?', 'My Training Manager just completed a observation, where can I find my score?']	,
['how do i review feedback from my training lead?', 'how do i review feedback that my trainer gave me about my training program?', 'what should i do about the feedback my training lead gave me during her last observation?', 'how do i review feedback my training lead gave me during my last observation?', 'how can i review the feedback my training lead gave me on her last observations?', 'how can i review feedback my training lead gave me during her last observation?', 'how do i review the feedback my training lead gave me during her last observation?', 'Where can I review the feedback my Training Lead gave me during her last observation? ']	,
['what feedback did my training manager give me during her last observation?', 'how can i review the feedback my training manager gave me during the last observation?', 'how can i review the feedback my training manager gave me during her last observation?', 'where can i review the feedback my training manager gave me during her last observation?', 'Where can I review the feedback my Training Manager gave me during her last observation? ']	,
['what is a way to see where you went in your observations?', 'where can i see what has gone well in my observations?', 'how can i see the progress of my observations?', 'how can i see how things went?', 'where can i see my observations?', 'how do i see how my observations went?', 'what should i do to see how my observations have gone?', 'how can i see how my observations have gone?', 'where can i see how my observations went?', 'where can i see how my observations have gone?', 'Where can I see how my observations have went? ']	,
["what's the best way to get my class to come up to me?", 'How can I get my class to roll up to me in CCO Learning? ']	,
['i need my class moved to cco learning', 'i need my class to be moved to cco learning', 'I need my class to be moved over to me in CCO Learning. ']	,
["i need to assign materials to my class but they're not showing up for me at cco learning", 'I need to assign materials to my class, but they are not rolling up to me in CCO Learning.']	,
['i am not in the cco learning class what can i do? how can i fix this?', 'how can i get my class back into cco learning?', 'what should i do if my cco learning class is not under me?', 'My class is not under me in CCO Learning. How can I get that fixed? ']	,
['what can i do if i want my class to be moved in cco learning so that i can assign the curriculum to them?', 'What do I do if I need my class moved to me in CCO Learning so I can assign the curriculum to them? ']	,
['tell me the best way to make new hires at cco learning more effective?', 'is there any way to get some new hires in cco learning?', "what should i do if i don't find a new hire for cco learning?", 'how do i find a new hire who does not fit my need in cco learning?', "what should i do if the new hires don't come to me from cco learning?", 'what do i do if the new hires are not rolling up to me at cco learning?', 'What do I do if the new hires are not rolling up to me in CCO Learning?']	,
['what should i do when a participant does not come up to me for cco learning?', 'What do I do if the participants are not rolling up to me in CCO Learning?']	,
['how do i contact trainees?', "what should i do if the trainees don't attend cco learning?", 'what should i do if trainees have not signed on to cco learning?', 'What do I do if the trainees are not rolling up to me in CCO Learning?']	,
['how do i get a teacher to assign a curriculum to me?', 'What ticket should I submit to get me class moved over to me to assign curriculum to them? ']	,
['How can I play Verint Calls for my class? ', 'How can I play Verint Calls for my class? ']	,
['is there any way to prepare for class calls?', 'How do I play calls for class?']	,
['how can i pull calls in a class?', 'How do I pull calls to play for CSRs in class?']	,
['how can i play the csr calls?', 'how can we play calls for csr?', "How to play calls for the csr's?"]	,
['How to play calls for the New hires?', 'How to play calls for the New hires?']	,
['How to play calls for the participants?', 'How to play calls for the participants?']	,
['How to play calls for the trainees?', 'How to play calls for the trainees?']	,
['how can i play classes at verint?', 'I need to play calls for class. How do I play them through Verint?']	,
['I need to play Verint calls for my class', 'I need to play Verint calls for my class']	,
['tell me the easiest way to play a class call?', 'can i play calls in class?', 'what steps do i take to play calls in a class?', 'What are the steps to play calls in my class?']	,
['how do i listen for calls?', 'how do i listen to calls in virtual classes?', 'how can i listen to calls in video training?', 'how do you listen to calls in a virtual training class?', 'How can we listen to call when we have a virtual training class?']	,
['how do you listen to calls?', 'what is the best way to listen to a webex class?', 'how do you listen to a call during webex training?', 'tell me the best way to listen to a call in a webex training class?', "how do we listen to calls when i'm in a webex training course?", 'How can we listen to call when we have a Webex training class?']	,
['how can i create a webex virtual training session?', 'how do you create a virtual training session using webex?', 'show the best ways to create a virtual training session with webex?', 'how can i create a virtual training session with webex?', 'how do i create virtual training sessions with webex?', 'how do i create a virtual training session with webex?', 'How do I create a virtual training session with WebEx?']	,
['how do i do webex meetings?', 'how can i create webex meetings?', 'how do i create a webex meeting?', 'how do i create webex meetings for a class?', 'how do i create webex meetings for my class?', 'how do i create a webex meeting for my class?', 'How do I create a WebEx meeting for my class?']	,
['i need to create a virtual course with webex', 'i need to create a virtual training course with webex', 'i have to create a virtual training class with webex', 'i need to create a virtual training class with webex', 'I need to create a virtual training class with WebEx. ']	,
['i need a simple guide how to use webex and microsoft teams in my classrooms what does it take to improve my knowledge?', 'how do i learn to use microsoft teams and webex?', 'is there a simple step by step guide to use microsoft teams and webex when training a class?', 'is there a quick step by step guide to use microsoft teams and webex in a class?', 'is there a quick step by step guide to use microsoft teams and webex when training a class?', 'Is there a quick step by step guide to use Microsoft Teams and WebEx when I train a class?']	,
['how do i create a training session using webex?', 'how do i do webex training?', 'list the steps for creating a webex training session?', 'how do i create webex training sessions?', 'how can i create a webex training session?', 'how do you create a webex training session?', 'how do i create a webex training session?', 'What do I need to do to create a WebEx training session?']	,
['how do i fill out a confirmation letter for my class?', 'how can i fill a confirmation letter?', 'how do i fill out the confirmation of a class?', 'can you fill out a confirmation form?', 'how do i fill out confirmation?', 'how do i fill out a class confirmation letter?', 'how do i fill in a class confirmation?', 'how do i fill out a class confirmation?', 'How do a fill out a class confirmation? ']	,
['how can i get confirmation of class?', 'what should i do to confirm the class?', 'how can i do class confirmation?', 'how do i do class confirmation?', 'How do I do the class confirmation?']	,
['what steps do you take to send a confirmation of class?', 'What are the steps to prepare and send a class confirmation?']	,
['how do i send a class confirmation form to an employee?', 'What do I do to complete and send a class confirmation? ']	,
['what is class confirmation?', 'what is a class confirmation?', 'What is a Class Confirmation?']	,
['how can you deal with a disruptive student?', 'how do i deal with disruptive trainees?', 'how do you deal with a disruptive trainee?', 'how do i deal with a disruptive trainee?', 'how do you handle disruptive trainees?', 'how do i handle disruptive trainees?', 'How do a handle a disruptive trainee?']	,
['how do i manage behavior issues?', 'how do i handle a behavior problem?', 'how do i approach behavior issue trainees?', 'what do you do with a behavioral issue trainee?', 'how do i approach a behavior issue trainee?', 'how do i help a behavior issue trainee?', 'how do i handle a behavior issue trainee?', 'How do I handle a behavior issue trainee? ']	,
['how do i handle behavior in a classroom?', 'how do i deal with behavior issues in class?', 'how can i solve behavior issues in class?', 'how do i resolve behavior issues in class?', 'how do i handle behavior issues in my class?', 'how can i handle behavior issues in class?', 'how do i handle behavior issues in a class?', 'how do i handle behavior issues in class?', 'How do I handle behavior issues in class?']	,
['how should i respond to disruptive participants?', 'show the best ways to address disruptive participants?', 'how do you deal with disruptive participants in training?', 'tell me the best way to address a disruptive participant of training?', 'how can i professionally and personally address a disruptive participant in a training?', 'how can i effectively address a disruptive participant in my training?', 'what should i do to address a disruptive participant in training?', 'how do you address a disruptive participant in training?', 'How do you professionally address a disruptive participant in training? ']	,
['how can i address behavioral issues in a classroom?', 'how do you address behavioral problem csr in your classroom?', 'how do i address behavioral issues as a csr professional?', 'how can you address an csr behavioral issue in your classroom in a professional manner?', 'what should i do to address a behavioral issue in csr in my classroom in the most professional manner?', 'how do you address behavioral issues in your classroom in the most professional manner?', 'How do you properly address a behavioral issue CSR in your classroom in the most professional manner?']	,
['how should i handle a disruptive csr in my class?', 'how can you deal with disruptive csr in a classroom?', 'how can you address disruptive csr in your classroom?', 'show some ways to properly address disruptive csr in the classroom?', 'how do you appropriately address a disruptive csr in your classroom?', 'how can you address disruptive csr in your classroom with the most professional manner?', 'how can you properly address disruptive csr in your classroom in the most professional manner?', 'How do you properly address a disruptive CSR in your classroom in the most professional manner?']	,
['how can i correct behavior in classes?', 'how do i resolve behavior problems in my classes?', 'how can i address behavior issues in a classroom?', 'how do i address behaviour issues in my classes?', 'how should i resolve behavior issues in my class?', 'how should i address behavior issues in a school?', 'how do i address behavior issues in my classes?', 'what should i do to address behavior issues in my class?', 'how can i address behavior issues in my class?', 'how should i address behavior issues in my classes?', 'How should I address behavior issues in my class? ']	,
['how should i be responsible for my class disruption?', 'how can i deal with disruptive behaviour in a class?', 'how should i deal with disruptive behavior in classes?', 'how should i deal with interruptions during classes?', 'how can i resolve a disturbance in my class?', 'how do i handle a disruption of class?', 'how can i deal with disruption in my class?', 'how do i deal with disruption in my class?', 'how do i address disruptions in the school?', 'how can i address my disruptions in class?', 'How should I address disruptions in my class? ']	,
['how do i handle someone who thinks a training session is not necessary?', "how do you handle a company employee who doesn't think training is needed?", "how do you deal with an employee who doesn't think your training is needed?", "how would you deal with an employee who isn't sure training session is necessary?", "what do you do when an employee doesn't think your training session is needed?", "how would you handle an employee who didn't think training was necessary?", "how do you deal with an employee who doesn't think a training session is necessary?", 'how do you handle an employee who does not think your training is necessary?', "how do you handle an employee who doesn't think your training session is necessary?", "How would you handle an employee who doesn't think your training session is necessary? "]	,
["how do i train a new employee who doesn't have this training?", "how can i get a student who's rehired who doesn't think they need the training?", "why does the new hire in the classroom think they don't need that training? how do i deal with it?", "i've recently been hired by a graduate who says that they don't need this training how should i deal with this?", "I have a rehire in class who doesn't think they need this training. How should I handle this? "]	,
['how do i tackle a behavioral issue?', 'how do you address a behavioral problem in class?', 'what should i do if i am in a behavioral issue classroom?', 'what should i do if i have a behavioral problem in my class?', 'what is a good way to deal with behavioral issues?', 'how can i deal professionally with behavioral issues?', 'how can i professionally address a behavioral issue?', 'list the best ways to professionally address a behavioral issue?', 'if there is a behavioral issue trainee in class what is the best way to deal with the behavioral problem?', 'if there is a behavioral issue trainee in class what is the proper way to professionally address the behavioral issue??', 'If there is a behavioral issue trainee in class, what is the proper way to professionally address the behavioral issue? ']	,
['how can i handle a distuptive trainee?', 'tell me the best way to address a distuptive trainee in class?', 'how do i handle a disruptive trainee in class?', 'how do i deal with a distuptive trainee in class?', 'how do i deal with a distuptive trainee who is in the middle of class?', 'if there is a distuptive trainee in class what is the appropriate way to professionally address the disruption?', 'if there is a distuptive trainee in class what is the correct way to professionally address the disruption?', 'If there is a distuptive trainee in class, what is the proper way to professionally address the disruption? ']	,
['tell me the best way to deal with a person who thinks training is unnecessary?', 'tell me the right way to cope with a person who thinks training is unnecessary?', "what can i do when i'm a student who feels this training is unnecessary?", 'what should i do when a participant thinks the training is unnecessary?', 'how do i deal with a participant who thinks that this training is unnecessary?', 'is there a person in a class who thinks this training is unnecessary?', 'i think my training is unnecessary how do i deal with this situation?', 'my partner has found a program that is unnecessary how should i deal with this situation?', 'i have a classmate that thinks this training is unnecessary how should i handle this situation?', 'There is a participant in class that thinks this training is unnecessary. How should I deal with this situation? ']	,
['what should i do if i have behavior problem?', 'how do i deal with behavior problems in a class?', 'what should i do when a behavior issue trainee is in class?', 'what can be done when you have a behavior issue trainee in class?', 'What can be done when you have a behavior issue  trainee in class? ']	,
['when a disruptive trainee is present in the class what can they do?', 'what should i do if my fellow trainee becomes disruptive in class?', 'what should i do if i want to disrupt a trainee?', 'what should you do if you have an upset trainee?', 'what should i do if i have a disruptive student in class?', 'what should i do when i have a disruptive student in my class?', 'what do you do when you have a disruptive trainee?', 'what can be done when you have a disruptive trainee in class?', 'What can be done when you have a disruptive trainee in class? ']	,
['how do i handle a trainee who does not think that course is important?', "how do i deal with a student who doesn't think this training is important?", 'how do i handle a person in class who does not think that this training is important?', "tell me the best way to deal with someone who doesn't think their training is important?", "what do i do to handle a trainee in class who doesn't think it's important?", "show the best ways to handle a beginner who doesn't think this training is important?", "how can i handle a trainee in class who doesn't think this training is important?", "tell me the best way to handle a trainee in the class who doesn't think this training is important?", "tell me the best way to handle a trainee in class that doesn't think this training is important?", "What is the best way to handle a trainee in class who doesn't think this training is important? "]	,
['i am having behavior issues in class what should i do?', 'how can i handle my behavior issues?', 'how do i handle behavior problems in my class?', 'how can i address behavior issues in a class?', 'how should i manage behavior issues in class?', 'how can i handle behavior issues in class?', 'can you list the best ways to deal with behavior issues in a classroom?', 'how do i handle behavior issues in my class?', "what's the best way to tackle behaviour issues in a classroom?", 'What is the best way to handle behavior issues in my class? ']	,
['how can i deal with school issues?', 'how do you deal with classroom issues?', 'how can i deal with classroom issues?', 'list the best ways to deal with classroom problems?', 'tell me the best approach to dealing with classroom issues?', "what's the best way to deal with a classroom issue?", 'recommend some effective steps to deal with classroom issues?', 'what should be the first step in dealing with a classroom problem?', 'What is the first steps in dealing with classroom issues?']	,
['tell me the best way to deal with behavior issues?', 'tell me the best way to address behavior in class?', 'how can you deal with your behavior problems in a classroom?', 'what can be done to deal with behavior in a class?', 'how do you control behavior issues in your class?', 'how can i solve my behavior issues in my class?', 'how do i deal with behavior issues in my class?', 'what should i do to resolve my behavior issue in my class?', 'what should be done about behavior issues in a classroom?', 'what should i do about behavior issues in my class?', 'What should I do about behavior issues in my class? ']	,
['tell me the easiest way to train virtual classes?', 'how can i train virtual classes?', 'how do i train virtual classes?', 'how do you train a virtual class?', 'how should i train a virtual class?', 'how do i train a virtual class?', 'How do a train a virtual class? ']	,
['how do i train for webex?', 'tell me the best way to train a webex class?', 'how do i train webex classes?', 'how do i train my webex class?', 'how do i train a webex class?', 'How do a train a Webex class? ']	,
['how can i conduct training on webex?', 'tell me the best way to prepare for a webex training?', 'how can i initiate a webex training?', 'how can i conduct webex training?', 'how can i conduct a webex training?', 'how do i conduct webex training?', 'how do i conduct a webex training?', 'How do I conduct a Webex Training? ']	,
['how do i handle a webex training session?', 'how can i conduct a webex training?', 'how do i conduct a webex training?', 'how can i facilitate a webex training?', 'how do i facilitate webex training?', 'How do I facilitate a WebEx Training? ']	,
['how do i start creating webex virtual classes?', 'how do i setup a webex virtual class?', 'list some tips for setting up a virtual webex class?', 'how do i get a virtual webex class?', 'how do i set up virtual webex classes?', 'how can i set up a virtual webex class?', 'how do i setup a virtual webex class?', 'how do i set up a virtual webex class?', 'How do I set up a Virtual WebEx Class?']	,
['i am assigned a webmaster training program with webex what is your plan?', 'I am assigned to train using WebEx. What now? ']	,
['is there any way to get help for virtual training?', 'how do i find help when training online?', 'can i find some guidance if i want to train online?', 'where can i find guidance when preparing for a virtual training?', 'where can i find guidance when preparing for virtual class?', 'where can i find guidance on how to train virtually?', 'Where do I find guidance when I am assigned to train virtually? ']	,
['How do go about uploading my DTR?', 'How do go about uploading my DTR?']	,
['how can i correct my daily training report errors?', 'how can i rectify a mistake in my daily training report?', 'how can i correct an error in my training report?', 'how do i correct an error in my training report?', 'how do i correct errors on training reports?', 'how can i correct an error in my daily training report?', 'how do i correct errors in a training report?', 'how can i correct an error on my daily training report?', 'how do i correct errors on my training report?', 'how do i correct an error in my daily training report?', 'How do I correct errors on my Daily Training Report? ']	,
['how do i correct my dtr error?', 'how do i correct dtr errors?', 'how can i correct a wrong dtr?', 'how do i fix an error in dtr?', 'how do i fix an error on a dtr?', 'what should i do to correct any errors in a dtr?', 'how do i rectify errors on my dtr?', 'how do i correct errors on drtr?', 'How do I correct errors on my DTR? ']	,
['How do I go about uploading my daily training report? ', 'How do I go about uploading my daily training report? ']	,
['how do i prepare a daily training report for a class?', 'how do i prepare a daily training report for class?', 'how do i write a daily training report?', 'how can i prepare the daily training report?', 'how do i prepare my daily training report?', 'how do i prepare the daily training report?', 'How do I prepare the class Daily Training Report? ']	,
['tell me the best way to prepare for the dtr?', 'how do i prepare for dtr class?', 'how should i prepare for dtr?', 'how do i prepare for the dtr?', 'how do i prepare for the class dtr?', 'How do I prepare the class DTR?']	,
['how can i record training information on the daily report?', 'how can i take a log of the information we gather in the daily training report?', 'can i record my daily training report?', 'how can we record a daily training report?', 'is there a way to record the information on a training report?', 'how should i record the data in daily training reports?', 'how can i record information on daily training reports?', 'how do i record information in a training report?', 'How do I record information in the Daily Training Report?']	,
['how can i record and record dtr data?', 'how do i record the data in the dtr?', 'how do you record information in drt?', 'how can i record information on dtr?', 'How do I record information in the DTR?']	,
['how can i upload daily training reports?', 'how can i upload a daily training report?', 'How do I upload a Daily Training Report?']	,
['how can i upload a dtr?', 'How do I upload a DTR?']	,
['how do i upload dtrs to a class?', 'how can i upload a dtr for class?', 'How do I upload the DTR for class? ']	,
['How do I Upload the DTR for either Class Training or Nesting?', 'How do I Upload the DTR for either Class Training or Nesting?']	,
['how much data should one keep in training reports?', 'what information can be recorded in a daily training report?', 'what information should i record on the training reports?', 'what information should be in my daily training report?', 'what amount of information should i record in my training reports?', 'how much information should be recorded in a daily training report?', 'how much information should i log in my daily training reports?', 'how much information should i record in my daily training reports?', 'How much information should I record in my Daily Training Report?']	,
['how many records should be in a dtr?', 'how much info do i need to record on my dtr?', 'how much information should be recorded in dtr?', 'which information should i record in my dtr?', 'how much information should i record in dtr?', 'how much information should i record in my dtr?', 'How much information should I record in my DTR?']	,
['how can i fix a problem during daily training?', 'how can i fix an error in a daily training report?', 'how can i fix an error on my daily training report?', 'how can i fix the errors on my daily training report?', 'how do i fix an error on a daily training report?', 'how do you fix an error on a daily training report?', 'how to fix an error in a daily training report?', 'how do i fix errors in my daily training report?', 'How to fix errors on Daily Training Report']	,
['how can i fix a dtr error?', 'how do you fix a dtr error?', 'how do i fix a dtr error?', 'how can i fix the error on dtr?', 'how can i fix an error on dtr?', 'how do i fix an error on dtr?', 'How to fix errors on DTR']	,
['can you correct me for a messed up training report?', 'how do i fix a linocut error in my daily training report?', 'how can i fix a glitch on my training report?', 'how do i fix a problem with my daily training report?', 'how can i fix my daily training report?', 'i messed up my training report how should i fix it?', 'I messed up my Daily Training Report.  How do I fix it?']	,
['how do you fix a dtr?', 'how do i fix an issue in my dtr?', 'how do i fix a problem with my dtr?', 'how do i fix a dtr problem?', 'how do i fix a dtr?', 'what should i do if my dtr does not work?', 'I messed up my DTR.  How do I fix it?']	,
['i need to fix mistakes in my training report daily', 'i need to fix the error in my daily training report', 'i need to fix the errors in my daily training report', 'i need to fix an error on my daily training report', 'i need to fix errors in my daily training report', 'i need to fix errors on my daily training report', 'I need to fix errors on my Daily Training Report. ']	,
['i need to fix some errors on my dtr', 'i need to fix the error on my dtr', 'i need to fix the errors in my dtr', 'i need to fix a error in my dtr', 'i need to fix errors on my dtr', 'I need to fix errors on my DTR. ']	,
['i have to upload my training report daily', 'i have to upload my daily training report', 'i need to upload my daily training report', 'I need to upload my Daily Training Report.']	,
['I need to upload my DTR.', 'I need to upload my DTR.']	,
['how do i make a daily training report and use it in my class?', 'list some checklists for daily training reports?', 'how do i create a daily training report for a class?', 'what should i use when creating the daily training report?', 'is there a checklist i need to follow in creating my daily training reports?', 'tell me the checklist to use in creating daily training reports?', 'which is the best checklist to use for creating daily training reports?', "what's the checklist to follow when creating my daily training report for a class?", 'is there a checklist for creating my daily training report for class?', 'is there a checklist to use when creating a daily training report for a class?', 'Is there a checklist to use when creating my Daily Training Report for class?']	,
['should i use a dtr for class checklist?', 'list some important things to do when creating a class dtr?', 'how can i start creating a dtr for my class?', 'what checklist should be used to prepare a dtr for class?', 'is there a checklist i can follow to start creating my dtrs for my class?', 'what should i do when creating a dtr for a class?', 'tell me the checklist you use for creating your dtr for your class?', 'what checklists should i use when creating my dtr for class?', 'is there a checklist to use when creating dtr for a class?', 'is there a checklist to use when creating a drtr for a class?', 'Is there a checklist to use when creating my DTR for class?']	,
['how do i upload training reports to my website?', 'tell me the most efficient way to upload training news?', 'how can i post my training report?', 'how can i upload daily training reports?', 'how can i upload a daily training report?', 'What are the steps to upload my Daily Training Report?']	,
['tell me the process of uploading a dtr?', 'can you list the steps to upload a dtr?', 'What are the steps to uploading my DTR?']	,
['what should i name my daily training report for?', 'what should i call my daily training report?', 'is that what i call my daily training report?', 'What do I name my Daily Training Report?']	,
['what should we call a dtr?', 'how do you name the dtr?', 'how do you name a dtr?', 'What do I name my DTR?']	,
['how is class dtr defined? what information do i need?', 'how can i include a list of information in the class dtr?', 'list the information to include in the dtr class?', 'what information should be listed in the class dtr?', 'what information should i include in the dtr class?', 'what information should i include in the class dtr?', 'what information should i include on the class dtr?', 'What information should I include on the class DTR?']	,
['what should be recorded in a daily training report?', 'what will be recorded in the daily training report?', 'what should be recorded on the daily training report?', 'what should be recorded in the daily training report?', 'what items are expected to be recorded in the training report?', 'what would be expected to be reported in the daily training report?', 'what is expected to be recorded in the daily training report?', 'What is expected to be recorded in the Daily Training Report?']	,
['what is expected in the dtr?', 'what information should be recorded in the dtr?', 'what will be recorded in the dtr?', 'what should be recorded in the dtr?', 'what are expected to be recorded in the dtr?', "what's expected to be recorded in the dtr?", 'what is expected to be recorded in the dtr?', 'What is expected to be recorded in the DTR?']	,
['show the names of the daily training reports?', 'what is naming convention for daily training reports?', "what's the naming convention for my daily training report?", 'What is the naming convention for my class Daily Training Report?']	,
['who can fill out the training daily report?', 'who can fill out a daily training report?', 'who is supposed to fill out training daily?', 'who is supposed to fill out the training report every day?', 'who is supposed to write the daily training report?', 'who is supposed to fill in the daily training report?', 'who is supposed to fill out the daily training report?', 'who is supposed to fill out the daily training report daily?', 'Who is supposed to fill out the Daily Training Report daily?']	,
['who can fill dtr daily?', 'who should fill the dtr daily?', 'who is supposed to fill out daily dtr?', 'who is supposed to fill dtr daily?', 'who is supposed to fill out dtr daily?', 'Who is supposed to fill out the DTR daily?']	,
['how can i add additional folders to my dtr assessment?', 'what is your system for adding additional folders in the dtr attendance assessment?', 'How do I add additional folders in the DTR (attendance, assessments etc)?']	,
['tell me the best way to create the metadata folder for the daily trainng reports?', 'how do you create a metadata folder for your daily trainng report?', 'how can i create a metadata folder for my daily trainng report?', 'how do i create a metadata folder for the daily trainng report?', 'how do i create a metadata folder for a daily trainng report?', 'how do i create a metadata folder for my daily trainng report?', 'How do I create a Metadata folder for my Daily Trainng Report?']	,
['how do i create a folder for my metadata dtr?', 'how do i build a metadata folder for my dtr?', 'i have a dtr and how do i create a metadata folder for it?', 'how do you create a metadata folder for a dtr?', 'how do i create a metadata folder for my dtr?', 'How do I create a metadata folder for my DTR?']	,
['how do you create a file folder for a daily training report?', 'how do i create a folder for my daily training report?', 'how do you create a metadata folder for your daily training reports?', 'can i create a new metadata folder for my daily training report?', 'how do i create a metadata folder for my training report?', 'how do you make a metadata folder for the daily training report?', 'how do i create a metadata folder for a daily training report?', 'how do i create a metadata folder for my daily training report?', 'how do i make a metadata folder for my daily training report?', 'How do I make a metadata folder for my Daily Training Report?']	,
['how can you create metadata for your dtr?', 'how can i create metadata folders for dtr?', 'how can i create a metadata folder for dtr?', 'how do i create a metadata folder on a dtr?', 'how do i create a metadata folder for a dtr?', 'how do i create a metadata folder for my dstr?', 'how do i create a metadata folder for my dtr?', 'how do i make a metadata folder for a dtr?', 'How do I make a metadata folder for my DTR?']	,
['how do you create a folder on sharepoint for training reports?', 'how do you create a folder for training reports?', 'what do i do if i need a folder for my daily training report?', 'tell me the best way to create a folder for my daily training report on sharepoint?', 'how do i create a folder for my daily training reports on sharepoint?', 'can i create a folder for my daily training report in sharepoint?', 'how do i create a folder for my daily training report in sharepoint?', 'i need to create a folder for my daily training report in sharepoint how can i start with this?', 'i need to create a folder for my daily training report on the sharepoint site how do i do that?', 'I need to create a folder for my Daily Training Report on the SharePoint site. How do I do that?']	,
['how do i create a folder to manage my dtr on sharepoint?', 'how can i get a folder for my dtr in sharepoint?', 'how do i create a folder for my dtr in sharepoint?', 'how do i create a folder for my dtr on sharepoint?', 'i need to create a folder for my dtr on sharepoint how can i do that?', 'i need to create a folder for my dtr on sharepoint how do i do this?', 'i need to create a folder for my dtr in the sharepoint site how do i do this?', 'I need to create a folder for my DTR on the SharePoint site.  How do I do that? ']	,
['how do you get a meta data folder for a database?', 'how do i get a metadata folder in the database?', 'how do i get a folder with meta data to the database?', 'how do i get a meta data folder into the database?', 'What is a meta data folder and how do I get it into the database?']	,
['how should i get folders with metadata?', 'what folders should i put metadata into?', 'what file should an metadata folder have?', 'tell me the proper file structure for a metadata folder?', 'what information should be used in a metadata folder?', 'What should be included in a Metadata folder?']	,
['where can i find my daily training report?', 'is there a way to store my class training report?', 'where can i store my daily training report?', 'where do i store my daily training report?', 'Where do I store my class Daily Training Report?']	,
['how can i save the dtr class for later use?', 'how can i get the class dtrs stored?', 'how can i keep my class dtr data?', 'where can i store a dtr class?', 'where should i store my class dtr data?', 'Where do I store my class DTR?']	,
['how can i upload daily training reports?', 'where can i put up my daily training report?', 'where can i upload daily training reports?', 'where can i upload my daily training reports?', 'where can i upload my daily training report?', 'Where do I upload my Daily Training Report?']	,
['how can i upload a dtr file?', 'how can i upload a dtr?', 'Where do I upload my DTR?']	,
["what is a csr? what's the best way to certify?", 'how do i get certified for a csr?', 'how do i go about certifying a csr?', 'tell me the best way to certify a csr?', 'how can i certify my csr?', 'list the ways to certify a csr?', 'how do i certify a csr?', 'How do I certify a CSR?']	,
['how does a csr certification process work?', 'what are csr certification processes? what are them?', 'show the steps for csr certification?', 'can you list the steps for csr certification?', 'tell me the process of csr certification?', 'tell me the process for csr certification?', 'what should be the process for csr certification?', 'What is the process for CSR Certification?']	,
['how do i communicate with a certified caller?', 'tell me the best way to communicate if a class is called certificated?', 'how can i tell that a class is call certified?', 'How do I communicate my class has call certified?']	,
['how do i get the class confirmation form?', 'how do i complete my class confirmation test?', 'how do i get a class confirmation?', 'How do I complete a class confirmation? ']	,
['How do I complete and send a Class Confimation?', 'How do I complete and send a Class Confimation?']	,
['how can we submit a confirmation of classes?', 'How do I submit a class confirmation? ']	,
['what are best ways to submit for a confirmation in a class?', 'how can i submit a confirmation to a class?', 'how do you send a class confirmation email?', 'How do I submit and send a class confirmation? ']	,
["what's the best way to send a confirmation to a class?", 'can i send confirmation of class to my teacher?', 'tell me the procedure for sending confirmation?', 'How to send in a class confirmation?']	,
['how can i complete a confirmation for the class?', 'i need to complete a class confirmation how can i do this?', 'I have to complete a class confirmation. How do I do that? ']	,
['how do i send a confirmation email to my class?', 'tell me the best way to send a confirmation of classes?', 'tell me the process of sending class confirmations?', 'What are the steps to sending a class confirmation? ']	,
['how can i send a confirmation to a class?', 'tell me the process for submitting a confirmation for a class?', 'what are steps for sending a class confirmation?', 'tell me the process of submitting a class confirmation?', 'what is the process for submitting a class confirmation?', 'What is the process for submitting a class confirmation?']	,
['how can i do peer audit?', 'how do i do a peer audit?', 'how do i complete peer auditing?', 'how do i complete a peer audit?', 'How do I complete peer audit?']	,
['what is instructor observation?', 'What is an Instructor Observation?']	,
['tell me the best way to prepare for the training report?', 'how can i complete daily training reports as per my requirements?', 'how do i get a daily training report?', 'how can i complete a daily training report?', 'how can i complete the daily training report?', 'How do I complete the Daily Training Report properly? ']	,
['how can you prepare for dtr?', 'is there any way to complete dtr?', 'how can i complete the dtr?', 'how do i complete the dtr?', 'how do i complete the dtr successfully?', 'How do I complete the DTR properly? ']	,
['how do i fill out a daily training report?', 'how do i fill out the daily training report?', 'tell me the best way to fill out the daily training report?', 'how do i fill out the daily training report? where can i find the information to assist me in filling out this report?', 'how do we fill out the daily training report and where do we get this information to assist us in filling out the daily training report?', 'How to fill out the Daily Training Report and where would we get this information to assist us in filling out the Daily Training Report?\xa0']	,
['how should i fill out the dtr and how should i do it?', 'how do i fill out the dtr? where can i get an explanation of how to do this?', 'How to fill out the DTR and where would we get this information to assist us in filling out the DTR?\xa0']	,
['how can i complete my daily training report without any problems?', 'show the best ways of getting a daily training report?', 'how do i write my daily training reports?', 'how do i fill the daily training report?', 'How to properly complete the Daily Training Report?']	,
['how do i complete a dtr test?', 'How to properly complete the DTR ']	,
['how can i become certified in csr?', 'which is the best way to do csr certification?', 'how can we do csr certification?', 'how can i do csr certification?', 'how can we conduct csr certification?', 'how can i conduct csr certification?', 'how do i conduct csr certification?', 'How do I conduct CSR Certification?']	,
['can you list the steps needed for csrf csr certification?', 'how do i get to csre?', 'how do i get my certification?', 'how do i get csr certification?', 'how can i get csr certification?', 'how do i do csr certification?', 'how can i do csr certification?']	,
['what should i do for certification in the csrs examination?', 'what should i do if i want to become a csr?', 'what is the best way to become certified in csr?', 'what can i do to get a certificate in csr?', 'how can i prepare for csr certification?', 'what do i have to do for csr certification?', 'what should i do for csr certification?', 'what do i do for csr certification?']	,
['list the best ways of registering for the certification process?', 'how do i pass an csr?', 'how do i prepare for csr?', 'how can i pass csr?', 'can you list the best ways to do csr certification?', 'how can i get an csr certificate?', 'how do i do csr certification?', 'how can i obtain csr certification?', 'how can i conduct csr certification?']	,
['how should i prepare for the csr exam?', 'how can i get an csr?', 'how should i conduct certifications in csr?', 'how can i do my csr certification?', 'how can i successfully conduct csr certification?', 'how does one do csr certification?', 'how do i complete csr certification?', 'how should i conduct csr certification?', 'how do i conduct csr certification?']	,
['How do I correctly input new hire attendance hours using the ATT? ', 'How do I correctly input new hire attendance hours using the ATT? ']	,
['how do i enter attendance hours from newly hired personnel using the attendance tracking tool?', 'how do i enter the attendance hours of new hires using the attendance tracking tool?', 'tell me the easiest way to track new hire attendance using the attendance tracking tool?', 'how do i properly enter new hire attendance hours using the attendance tracking tool?', 'how do i correctly enter new hire attendance hours using the attendance tracking tool?', 'How do I correctly input new hire attendance hours using the Attendance Tracking Tool? ']	,
['How do I correctly track new hire attendance using ATT?', 'How do I correctly track new hire attendance using ATT?']	,
['How do I correctly track new hire attendance using Attendance Tracking ToolT?', 'How do I correctly track new hire attendance using Attendance Tracking ToolT?']	,
['How do I input time using ATT correctly for new hires in the training class? ', 'How do I input time using ATT correctly for new hires in the training class? ']	,
['How do I input time using Attendance Tracking Tool correctly for new hires in the training class? ', 'How do I input time using Attendance Tracking Tool correctly for new hires in the training class? ']	,
['how do you find time on time?', 'How do I use ATT and put time in correctly?']	,
['how does attendance tracking tool work?', 'how do i set an attendance tracking tool?', 'How do I use the Attendance Tracking Tool and put time in correctly?']	,
['How is time missed during new hire training tracked in ATT?', 'How is time missed during new hire training tracked in ATT?']	,
['tell me the best way to track the time lost during new hire training?', 'how does attendance tracking work in new hire training?', 'How is time missed during new hire training tracked in the Attendance Tracking Tool?']	,
['how can i create a training report each day?', 'tell me the best way to create a daily training report?', 'how do i generate a daily training report?', 'how do i create daily training reports?', 'how do i create a daily training report?', 'How do I create a Daily Training Report?']	,
['how can i use dtr?', 'how do you create a dtr?', 'how do i create dtrs?', 'how can i create a dtr?', 'how do i create a dtr?', 'How do I create a DTR? ']	,
['how can i name my training reports?', 'How do I name my Daily Training Report?']	,
['can you document your training?', 'how can i document what my training delivery milestones were?', 'how can i document the training milestone?', 'how do i document a training milestone?', 'how can i document that i have delivered a training milestone?', 'How do I document that I delivered a Training Milestone?']	,
['how do i coach a trainee who missed training?', "i need to coach a trainee who missed training when they didn't have time to train? what form do i fill out?", 'i need to coach a trainee who missed the training how do i do that?', 'i am coaching a trainee who missed training what form do i fill out?', 'i need to coach a trainee who missed training what form should i fill out?', 'i need to coach a trainee who missed his training what form should i fill out?', 'i want to coach a trainee who missed their training what form do i fill out?', 'i need to coach a trainee who missed training what form do i fill out?', 'I need to coach a trainee who has missed during training.  What form do I fill out?']	,
['can you list the templates i need to deliver my training milestones?', 'tell me the template for delivering a training milestone?', 'where can i find a template for a training milestone?', 'if you need to track a milestone in a training course where can you find a template?', 'i need a template for a training milestone where do i start?', 'i want to deliver a training milestone where can i find a template?', 'i need to deliver a training milestone where can i find a template?', 'I need to deliver a training milestone.  Where can I find a template?']	,
['which template is best to find for a training milestone?', 'how do i create my training milestone?', "what's the template for training milestones?", 'is there a template for training milestones?', 'where can i find a template for my training metric?', 'where can i get a template for a training milestone?', 'Where do I find a template for a Training Milestone?']	,
['tell me the best way to fill out an ecl online?', 'tell me the procedure for filling an ecl?', 'what should i do to fill ecl form?', 'how can i fill out ecl?', 'how do i fill in an ecl?', 'how do i fill out the ecl?', 'How do I fill out an eCL?']	,
['show me the best ways to fill an ecl?', 'tell me the best way to get an ecl?', 'show me the easiest way to fill in an ecl?', 'tell me the best way to fill out an ecl?', 'tell me the best way to fill in the ecl?', 'tell me the best way to fill up an ecl?', 'can you tell me the best way to fill an ecl?', 'tell me the best way to fill in an ecl?', 'tell me the best way to fill in an ecl?']	,
['can you list the ways to make an ecl?', 'how do i fill an ecl?', 'how do i fill an ecl?']	,
['how can i start filling the ecl form?', 'how do i complete an ecl?', 'how do i fill an ecl?', 'how do i fill in an ecl?', 'how do i fill out my ecl?', 'how do you fill out an ecl?', 'how can i fill out an ecl?', 'how can i fill out an ecl?']	,
['how can i get an ecoaching log?', 'how can i fill out the ecoaching log?', 'how can i fill out an e-coaching log?', 'how can i fill out an ecoaching log?', 'how do i fill out a coaching log?', 'how do i fill out an ecoaching log?', 'How do I fill out an eCoaching Log? ']	,
['how do you fill up a journal?', 'can you list the ways to fill the ecoaching log?', 'how can i fill up an ecoaching journal?', 'how do i fill up the ecoaching log?', 'how do i fill an ecoaching log?', 'how can i fill in an ecoaching log?', 'how do i fill in an ecoaching log?']	,
['how do i write the ecoaching log?', 'how do i form a ecoaching log?', 'how do i fill the ecoaching log?', 'how do i fill out an ecoaching log?', 'how do i fill out an ecoaching log?']	,
['how do i fill out a form for an urgent term request?', 'how do i fill out a form for urgent term requests?', 'how can i fill an urgent term request?', 'how do i fill out an urgent term request form?', 'how do i fill a urgent term request?', 'how do i fill an urgent term request?', 'How do I fill out an urgent term request?']	,
['how can i request termination urgently?', 'how do i fill in a termination form?', 'how do i fill in my termination request?', 'how do i fill out a termination request?', 'how do i submit an urgent termination request?', 'how can i fill out an urgent termination request?', 'how do i fill out an urgent termination request?', 'How do I fill out an urgent termination request? ']	,
['how do you do a term request?', 'how do you fill a term request?', 'how do i fill a term request?', 'how do i fill out a term request?', 'what steps are involved in filling out a term request?', 'can you list the steps to fill in term request form?', "what's the best way to fill out a term request?", 'can you list the steps to fill out a term request form?', 'What are the steps to fill out a term request? ']	,
['how can i request a termination order?', 'how do i do a termination request?', 'how do i file a termination request?', 'how do you file a termination request?', 'how can i file a termination request?', 'how do i fill out a termination request?', 'show the steps in filling out a termination request?', 'What are the steps to filling out a termination request? ']	,
['tell me the best way to fill out timecards for a second shift?', 'how do i fill out my 2nd shift timesheet?', 'how can i fill out the time card for second shift?', 'how can i fill out a timecard for 2nd shift?', 'How do I fill out my timecard for 2nd shift?']	,
['how can i fill out my timecard after 7pm?', 'how do i fill out a timecard after 7pm?', 'How do I fill out my timecard for after 7pm? ']	,
['How to record time in deltek for 2nd shift?', 'How to record time in deltek for 2nd shift?']	,
['How to record time in deltek for second shift?', 'How to record time in deltek for second shift?']	,
['I have a night class. How do we record our 2nd shift time?', 'I have a night class. How do we record our 2nd shift time?']	,
['how do you record second shift time in class?', 'can you record a second shift time?', 'how can we record second shift time?', 'I have a night class. How do we record our second shift time?']	,
['how can i record second shift time?', 'how does the deltek time card record second shift time?', 'What do I put on my Deltek time card to record 2nd shift time? ']	,
['what would be the best time stamp i could put on my deltek time card after 7 shifts?', 'what do i have to write on my deltek time card to get an accurate time-to-save?', 'What do I put on my Deltek time card to record after 7 shift time? ']	,
['show the options for recording second shift time on a deltek - timecard?', 'is it possible to record second shift time on deltek time card?', "what is on deltek's time card?", 'what can i do to record my deltek shift time?', 'what should i put on my deltek time card?', 'can a deltek time card record second shift time?', 'how can i use my deltek time card to record the second shift?', 'What do I put on my Deltek time card to record second shift time? ']	,
['how can i print my daily training report?', 'tell me the easiest way to fill out a daily training report?', 'how can i fill in the daily training report?', 'how do i fill out a daily training report?', 'how do i fill out the daily training report?', 'How do I fill out the Daily Training Report?']	,
['how can we update our daily training report?', 'tell me the information i should include in the daily training report?', 'show the key information i should include on the daily training report?', 'what information should i include in my daily training report?', 'what information should i include in the class daily training report?', 'what information should i include on the class daily training report?', 'What information should I include on the class Daily Training Report?']	,
['how can i fill out the cit score table?', 'How do I fill out the scores and cert table in the DTR? ']	,
['how do i use the scores tab in the daily training report to fill out the certifications?', 'how can i fill out the score and certification tab on the daily training report?', 'how do i fill out the score and certification tab on a daily training report?', 'How do I fill out the scores and certification tab on the Daily Training Report? ']	,
['how do you complete a daily training report?', 'how can i complete the daily training report?', 'tell me the requirements to complete a daily training report?', 'What is required to complete the scores and certs in the Daily Training Report? ']	,
['what would be the required marks to complete the dtr?', 'what is required for completion of dtr scores and certs?', 'What is required to complete the scores and certs in the DTR? ']	,
['what do i need for a dtr score?', 'tell me the minimum certification necessary for dtr scores?', 'can we get certification under dtr?', 'can you list the requirements for dtr scores?', 'tell me the necessary documentation to get the dtr certification?', 'what is required for certification of dtr scores in india?', 'tell me the required information for dtr scores?', 'tell me the requisite information for dtr certification?', 'What is the required information for the scores and certification on DTR?']	,
['how does one figure out the nesting agenda?', 'how do i discover the nesting agenda?', 'how can i find the nesting agenda?', 'where do i find the nesting agenda?', 'how do i find the nesting agenda?', 'How do I find the nesting agenda?']	,
['how can i find an agenda for a nesting nest?', 'can you list the best ways to create a nesting agenda?', 'how do you figure out your nesting agenda?', 'can you list the possible nesting agendas?', 'can you list the ways to set up a nesting agenda?', "how do you find a good 'nesting' agenda?", 'how can i find my nesting agenda?', 'how can i find a nesting agenda?', 'how can i find a nesting agenda?']	,
['how do i get started with the nesting agenda?', 'how can we create a nesting agenda?', 'where do i find my nesting agenda?', 'how do i find my nesting agenda?', 'how do i find a nesting agenda?', 'how do i find a nesting agenda?']	,
['can you list the goals of the nest?', 'how do you find a plan on nesting?', 'how do i find a nesting agenda?', 'how do i find the nesting agenda?', 'how do i find the nesting agenda?']	,
['how do i get a daily training report?', 'how do i get the list of the classes i would like to attend and complete the daily training report?', 'how do i fill out my daily training report?', 'how do i access the entire information on my class and fill it out?', 'how do i get all the details needed for the daily training report?', 'how do i find the course information that is required to fill out my daily training report?', 'how do i find all the information necessary to fill out my training report?', 'how do i get all the information needed to fill out a daily training report?', 'How do I get all the information for my class needed to fill out my Daily Training Report?']	,
['where can i get information for dtr?', 'tell me the best way to get the details about my class and also fill out my dtr?', 'how do i get all the information on dtr?', 'how do i get a list of all the information needed to fill the dtr?', 'how do i get all of the information for my dtr to submit?', 'how can i get all the information needed for my dtr?', 'How do I get all the information for my class needed to fill out my DTR?']	,
['tell me the easiest way to pull information from a crop report?', 'how can i pull information from a crop report?', 'How do I pull information from CROP?']	,
['how can i find a list of the relevant and relevant information from the recruiters portal and make them available to me online?', 'tell me the best way to get information about any candidate who had successfully applied for the recruitment?', "how do i extract candidates' recruitment information from the recruiting portal?", 'how do i access onboarding application information using the candidate recruitment portal?', 'how can i get information on the candidate recruitment website?', 'how do i get updates from the candidate recruitment portal?', 'how do i use the candidate recruitment onboarding portal?', 'How do I pull information from the Candidate Recruitment & Onboarding Portal?']	,
['i need data from my class recruitment portal how do i find this information?', 'i need data from the candidates recruitment and operations portal for my class where do i get this information?', 'I need CandidateRecruiting and Operations Portal data for my class. Where do I find that?']	,
['where can i find a list of crop data files?', 'how can i find this kind of data for a class?', 'where can i find the crop data in my class?', 'where can i find the crop data for my class?', 'i need crop data for my class where can i get it?', 'i need crop data for my class where do i find this?', 'i need crop data for my class where do i find that?', 'I need CROP data for my class. Where do I find that?']	,
['can you list the training reports i must fill out on crop?', 'what information do i need to fill out my daily training report?', 'What information from CROP do I need to fill out my Daily Training Report?']	,
['What information from CROP do I need to fill out my DTR?', 'What information from CROP do I need to fill out my DTR?']	,
['what is the crop database i use for my classes?', "where can i find my class's crop data?", 'tell me the crop data for my class?', 'where can i find crop data for a given class?', 'where can i get crop data for my class?', 'how can i find crop data for my class?', 'where can i find all the crop data for my class?', 'where can i find crop data for my class?', 'Where can I find the CROP data for my class?']	,
['where can i find out all the information regarding the candidate recruiting and operations portal?', 'is there a online recruitment portal for my classes?', 'how can we get our class info from the candidate recruitment and operations portal?', 'how do i find candidates for the recruiting and operations portal?', 'where can i get access to the candidate recruitment portal of your class?', 'Where do I get the Candidate Recruiting and Operations Portal  information for my class?']	,
['how can i get crop info for my class?', 'where can i get crop information for my class?', 'where can i get the crop information for my class?', 'Where do I get the CROP information for my class?']	,
['how can i access class information in a daily training report?', "how do i get my instructor's class information for my daily training report?", 'where can i get my class information for a daily training report?', 'how do i find my class information and fill in daily training reports?', 'where do i access class information to fill out my daily training report?', 'Where do I pull my class information to fill out my Daily Training Report?']	,
['How do I get my missing trainees added to CROP?', 'How do I get my missing trainees added to CROP?']	,
['i need my participants to be included in the candidate recruiting and operations portal what can i do?', 'I need my participants added to Candidate Recruiting and Operations Portal. What do I do?']	,
['I need my participants added to CROP. What do I do?', 'I need my participants added to CROP. What do I do?']	,
['i need my trainee added to the crop', 'I need my trainee added to CROP.']	,
['i need my trainees to be added to the candidate recruiting and operations portal', 'i need my trainees added to the applicant recruiting and operations portal', 'i need my trainees added to the candidate recruitment and operations portal', 'i need my trainees added to the recruiting and operations portal', 'i need my trainees added to the candidate recruiting and operations portal', 'I need my trainees added toCandidate Recruiting and Operations Portal.']	,
['i have a new hire who is missing from the portal for recruiting  operations? what should i do?', 'what can i do if one of my new hires is not found in the candidate recruiting and operations portal?', 'my new employee is not listed on the candidate recruiting and operations portal what should i do?', 'if i am missing one of my new hires from the recruiting and operations portal what should i do?', 'One of my new hires is missing from Candidate Recruiting and Operations Portal. What do I do?']	,
['why am i missing out on my new hire?', 'is this a missing crop person?', 'One of my new hires is missing from CROP. What do I do?']	,
['what do i do if i lose someone from my course?', 'what should i do when i am missing one of my participants in the candidate recruitment and operations portal?', 'how do i get one of my participants to the candidate recruiting and operations portal?', 'One of my participants is missing from Candidate Recruiting and Operations Portal. What do I do?']	,
['what should i do if one of my participants is not in the crop program?', 'i missed one of the participants at crop what can i do?', 'One of my participants is missing from CROP. What do I do?']	,
['one of my trainees is missing from the candidate recruiting and operations portal what should i do?', 'One of my trainees is missing from Candidate Recruiting and Operations Portal. What do I do?']	,
['what should i do to help a trainee who is missing from the crop program?', 'when a trainee is missing from the crop program what should i do?', 'one of my trainees is missing from the crop how can i help them?', 'One of my trainees is missing from CROP. What do I do?']	,
['can i add participants?', 'is it possible to add some participants on the candidate recruiting and operations portal?', 'can i add participants on the candidate recruiting and operations portal?', 'can i add some participants on the candidate recruiting and operations portal?', 'Some of my participants are missing from Candidate Recruiting and Operations Portal. Can I get them added?']	,
['can i add participants to my crop group?', 'i have a few participants missing from the crop can i get them added?', 'Some of my participants are missing from CROP. Can I get them added?']	,
['can i add a trainee to my recruits page?', 'can i have a trainee from the recruitment department added?', 'can i add a trainee who is not added to the candidate recruiting and operations portal?', 'can i add trainees on the candidate recruiting and operations portal?', 'some of my trainees are not listed in the candidate recruitment and operations portal can i get them added?', 'Some of my trainees are not in Candidate Recruiting and Operations Portal. Can I get them added?']	,
['Some of my trainees are not in CROP. Can I get them added?', 'Some of my trainees are not in CROP. Can I get them added?']	,
['how can i find someone missing from the candidate recruitment portal?', 'What do I do if someone is missing from Candidate Recruiting and Operations Portal? ']	,
['is it possible to contact an individual on crop when they are missing something from the list?', 'What do I do if someone is missing from CROP? ']	,
["how do i know if i'm submitting the right ticket?", 'how can i know if i submitted a ticket correctly and correctly?', 'how can you know if you submitted a ticket correctly and appropriately?', 'how do i know if i submit a ticket correctly and appropriately?', 'how can i know if i am submitting a ticket correctly and appropriately?', 'How do I know if I am submitting a ticket correctly and appropriately (IT, LSA, Maxnet, CCO Learning, etc.)?']	,
['how can i know if i have tickets for cherwell?', 'How do I know what catergory to submit tickets on Cherwell? ']	,
['how do you determine which catergorie to create a ticket for?', 'how do you determine which catergory to create a ticket for?', 'How do I know which catergory to create a ticket? ']	,
['how can i submit tickets to it?', 'how do i submit tickets to it?', 'How do I submit Tickets to IT?']	,
['tell me the best way to submit a ticket?', 'how do you submit tickets?', 'how do you submit a ticket?', 'how do i submit a ticket?', 'How do submit a ticket?']	,
["how do you know if you're sending the correct ticket?", "How do we know if we're sending the correct ticket too IT\\LSA is there a reference material and if we can send tickets for trainees that cant access outlook, or Maxnet for the first time?"]	,
['what are ticket forms? how do you submit tickets?', 'how should i submit a ticket?', 'how do i submit a ticket?', 'how do i send a ticket? what information do i need to submit it?', 'How do you properly submit a ticket and what information is needed?']	,
['can i get job aid for ticket submission?', 'which is the best place to get job assistance in the selection of catergories for ticket submission?', 'are there any free jobs to help choose catering options for a ticket issuance?', 'is there any job help for selecting catergories for ticket submissions?', 'is there any job aid in selecting catergories for tickets submissions?', 'is there any job aid to help in selecting catergories for ticket submissions?', 'is there any job aid to help in choosing catergories for ticket submissions?', 'is there a job aid to help in choosing catergories for ticket submission?', 'is there a job aid to help in choosing catergories for ticket submissions?', 'Is there a job aid to help in choosing catergories for ticket submissions?']	,
['is there a way to use to send a ticker to cherwell?', 'Is there a resourse for us to use in choosing a catergory to submit a ticker through Cherwell? ']	,
["Ticket creation - We received a great 'how to' on creating tickets but can there be a reference material or SOP to know what falls under which category?", "Ticket creation - We received a great 'how to' on creating tickets but can there be a reference material or SOP to know what falls under which category?"]	,
['how can i open a ticket to a local system administrator?', 'which catergorize should i use to create the ticket for the local system administrator?', 'which type of ticket should i use to send to a local system administrator?', 'what sort of ticket should i use for ticketing to the local system administrator?', 'which catergory should i use for tickets to the local system administrator?', 'What catergory should I I use for tickets to Local System Administrator? ']	,
['list the most efficient methods of submitting a ticket?', 'tell me the best way to submit a ticket?', 'which categoria should i use to submit a ticket?', 'which catergory should i use to submit a ticket?', 'What catergory should I use to submit a ticket?']	,
['what kind of ticket should i use to submit a ticket for the it office?', 'what ticket are you using to submit an it ticket?', 'tell me the best ticket that i can use when i submit a ticket?', 'which ticket should i use when i submit a ticket to it?', 'what ticket should i use when i submit a ticket to it?', 'what ticket do i use when i submit a ticket to it?', 'What ticket do I use when I submit a ticket to IT?']	,
['where can i get a list of catergories for admission into itlsa?', 'Where can I find the catergories to submit tickets to IT/LSA? ']	,
['how do i know whether a virtual or online class i attend is real or virtual?', 'how do i know whether a class is a virtual class or an on-site class?', 'how do you know if a class is online?', 'how do i know if a class is online or virtual?', 'how do i know if my class is online or virtual?', 'how do i know if a class is online or online?', 'how can i know if my classes are virtual or on-site?', 'how do i know whether my class is virtual or on site?', 'how do i know if my class is virtual or on-site?', 'How do I know if my class is virtual or on site?']	,
['how do i know when training is going on?', 'how can i know where i am training?', 'how do i know if i am in a class?', 'how can i tell when and where i am training for a class?', "how do i know when and where i'm training?", 'how do i know when and where i am training?', 'How do I know when and where I am training a class?']	,
['how long does it take to receive medicare coverage from medicare-supply?', 'tell me the difference between medicare and marketplace?', 'how long is a medicaremedicaid exchange marketplace plan?', 'how long does a medicare to marketplace dual class last?', 'How long is a Dual Medicare to Marketplace Class?']	,
['How long is a FFM New Hire Class?', 'How long is a FFM New Hire Class?']	,
['how long does a new hire marketplace take?', 'how long does a marketplace new hire class last?', 'How long is a Marketplace New Hire Class?']	,
['how long does medicare run between the medicare and marketplace classes?', 'how long can medicare be part of the marketplace?', 'how long does it take medicare to become a marketplace class?', 'how long is a medicare marketplace class?', 'How long is a Medicare to Marketplace Class?']	,
['how long does it take for a fbm to get registered?', 'how long will a transfer to the ffm class take?', 'how long does it take to transfer to ffm class?', 'how long can i transfer to ffm class?', 'how long is a transfer to the ffm class?', 'How long is a transfer to FFM Class? ']	,
['how long does a transfer in a medical or pharma school take?', 'how long does it take to transfer to gen med class?', 'How long is a transfer to Gen Med Class? ']	,
['how much time does it take to get to the marketplace class?', 'how long does it take to move in marketplace class?', 'how long does it take to transfer to marketplace class?', 'how long does a transfer to marketplace class take?', 'How long is a transfer to Marketplace Class? ']	,
['how long does it take for a medicare student to transfer to another class?', 'how long does it take to convert medicare to the medicare class?', 'how long does medicare class transfer?', 'how long does it take to transfer to the medicare-based class?', 'how long does medicare classes last?', 'how long does it take to transfer in medicare?', 'how many days should i go in a medicare class?', 'how long does it take to transfer in a medicare class?', 'how long does it take to transfer to medicare class?', 'How long is a transfer to Medicare Class? ']	,
['how long does it take for class?', 'how long are my classes?', 'how long is the class?', 'how long is a class?', 'how long is my class?', 'How long is my class?']	,
['tell me the duration of dual class?', 'how long does dual class last?', 'how long does a dual class last?', 'how many days does the dual - class last?', 'how many days does a class last?', 'how many days will a dual class last?', 'how many days does the dual class last?', 'how many days do a dual class last?', 'how many days does a dual class last?', 'How many days does a Dual class last?']	,
['how many days does medicare marketplace do?', 'how many days does medicare-maryland - marketplace - class last?', 'how many days does a medicaremedicaid dual  marketplace class last?', 'how many days does a dual medicare to marketplace class last?', 'How many days does a Dual Medicare to Marketplace Class last?']	,
['how many days does it take to complete a medicare dual enrollment?', 'how many days in a medicare dual enrollment class?', 'how many days do people spend in a dual medicare class?', 'how many days does a dual medicare class take?', 'how many days a dual medicare class is?', 'how many days is a medicare class?', 'how many days is a dual medicare class?', 'How many days is a Dual Medicare class?']	,
['how many days does medicare require?', 'how many days does a medicare new hire class last?', 'how many days will a medicare new hire be?', 'how many days is a medicare re-hire class?', 'how many days is a medicare new hire class?', 'How many days is a Medicare New Hire Class?']	,
['how long is the first marketplace hiring class?', 'How many days is Marketplace New Hire Class?']	,
["how do i know if i'm training with a professional?", "how do i know if i'm training with a real person?", 'how do you know which people you are training for?', 'how can i tell who i am training with?', "how do i know who i'm training with?", "How will I know who I'm training with?"]	,
['show me the current training schedule?', "what's the current training schedule?", 'can you list the current and anticipated training schedule?', 'show me the current and expected training schedule?', 'tell me the current and future training schedule?', 'tell me the current and upcoming training schedule?', 'what is the current and upcoming training schedule?', "what's the current and upcoming training schedule?", 'What is Current and Upcoming Training Schedule? ']	,
['what is cuts? what are its characteristics?', 'tell me the meaning of the term cut?', 'tell me the significance of a cut?', 'what are cuts?', 'what is cut?', 'What is CUTs?']	,
['show me the upcoming courses and schedules?', 'where can i find planned classes?', 'where can i find upcoming classes?', 'where can i find scheduled classes?', 'where can i find scheduled upcoming classes?', 'Where do I find scheduled upcoming classes?']	,
['how do i find the id number of my class?', "where can i find my school's class id?", 'where is the class identification number?', 'where can i find the class id?', 'how can i find a class id number?', 'how do i find a class id number?', 'how do i find my class id number?', 'how do i find the class id number?', 'where do i find the class id number?', 'Where do I find the class ID number?']	,
["where can i find my class's id?", 'where can i find my class id?', "what's my class id?", 'where can i see my class id?', 'what is my class id?', 'where is my class id?', 'Where is my class ID?']	,
['who do i share my tutor with?', 'who is my co-instructor?', 'Who is my co-instructor?']	,
['is that my coach?', 'who is my trainer?', 'who is my co-trainer?', 'Who is my co-trainer?']	,
['how should i prepare for a school class?', 'how should i prepare for a class?', 'what should one do to prepare for class?', 'how do i prepare for a class?', 'how do i prepare for classes?', 'How do I prepare for class?']	,
['what should i do with my class?', 'how do i prepare for a class next week?', 'what should i do now that i have a class up?', 'what should i do now that i have a class coming up?', 'i have a class coming up what should i do?', 'i have a class coming up what should i do now?', 'I have a class coming up, what do I do now?']	,
['how can i prepare for training?', 'how do i prepare for class?', 'how do i prepare for an a level training?', 'how do i get my floor ready for class?', 'how do i get the floor ready for a class?', 'how do i get the floor ready for class?', 'how can i get the training area ready for class?', 'what can i do to get my training area ready for class?', 'What do I do to get the floor or training area ready for class?']	,
['how do i prep for a training class?', 'how do i get my training room ready for class?', 'how can i get the training room ready for class?', 'how do i get the training room ready for class?', 'What do I do to get the training room ready for class?']	,
['is it necessary to have a class?', 'what do i need to prepare for class?', 'what should i do for a class?', 'what should i do for class?', 'what should i have for class?', 'what do i need in class?', 'what do i need for a class?', 'what do i need for class?', 'What do I need for class?']	,
['recommend some things that a teacher needs to get ready for a class?', 'list the necessary items to train a class?', 'what items do i need in order to train a class?', 'what items do i need to train my students?', 'What items do I need to train a class?']	,
['list some good tips and tricks for preparing for an upcoming class?', 'tell me the best plan i can follow for preparation of class?', 'what should i do when i prepare for my class?', 'what should one do when preparing for a class?', 'what should i do when preparing for class?', 'what should i do when preparing for a school class?', 'When preparing for class, what should I have?']	,
['where can i find a list of preparation checklists for my upcoming class?', 'how do i find a checklist for class preparation?', 'where can i find a checklist to prepare for class?', 'how do i find a student preparation checklist?', 'how do i find class preparation checklists?', 'how do i find a class preparation checklist?', 'Where can I find a class preparation checklist?']	,
['how do i prepare for exams in finals?', 'tell me the best way to prepare for the final assessment?', 'how do i prepare for the final exam?', 'how should i prepare for final assessment?', 'how can i prepare for the finals assessment?', 'how do i prepare for final assessment?', 'how should i prepare for the final assessment?', 'how do i prepare for the final assessment?', 'how do i prepare a class for the final assessment?', 'How do I prepare my class for the final assessment?']	,
['tell me the best way to prepare for a final exam?', 'how can i prepare for final exam?', "what's the best way to prepare for the final exam?", 'how do i prepare for final exams?', 'how do i prepare for final exam?', 'how do i prepare for the final exams?', 'How do I prepare my class for the final exam? ']	,
['how can i get some high scores for level 1?', 'is there a way to track and pull down level 1 results?', 'how do i take a level 1 search?', 'how should you pull a level 1 result?', 'how can i pull out my level 1 results?', 'How do I pull my Level 1 results? ']	,
['how can i get into level 1?', 'how do i retrieve level 1 results?', 'how can i pull a level one test results?', 'how can i pull a level 1 result?', 'how do i pull a level one test?', 'how do you pull a level one result?', 'how do i score my level one results?', 'How do I pull my Level One results?']	,
['what can i do to request a term?', 'How do I request a term properly? ']	,
['how can i request a termination of my account?', 'how can i request a termination?', 'How do I request a termination properly? ']	,
['how do i begin a termination process?', 'how do i start the process of terminating an employee?', 'how do i start the process of terminating a new employee?', 'how do you initiate the process of terminating a hire?', 'how do i start the process of terminating a new hire?', 'how should i start the process of terminating a new hire?', 'What are the proper steps to start the process of terminating a new hire? ']	,
['how can i get a termination request and answer it?', 'how can i process a termination request?', 'how should i process a termination request?', 'how do i process a termination request?', 'how do you process a termination request?', 'what steps should i follow to process a termination request?', 'show the steps to process a termination request?', 'show the steps to properly process a termination request?', 'What are the steps to properly process a termination request? ']	,
['how do i stop a trainee?', 'how do i terminate a trainee?', 'what should i do if i want to terminate my trainee?', 'What do I need to do to get the process started to terminate a trainee?']	,
['can i fire a new hire?', 'list the forms that should be filled out to fire a trainee?', 'what forms do you need to fill out to fire a trainee or new hire?', 'what forms should i follow to fire a new hire trainee?', 'what forms do i need to fill out to fire a new hire?', 'what forms do i need to fill out to fire a new hire trainee?', 'What forms do I need to fill out to fire a new hire trainee? ']	,
['list the forms required to separate a new hire trainee from a former trainee?', 'can you list the forms required to separate a new hire from a trainee?', 'what are the forms i should complete to separate a new hire from a trainee?', 'what forms do i need to fill out to separate a trainee from a new hire?', 'what forms do i need to fill out in order to separate a new hire trainee?', 'what forms do i need to fill out to separate a new hire trainee?', 'What forms do I need to fill out to separate a new hire trainee? ']	,
['how do i terminate a trainee?', 'which form should i fill out in order to terminate a trainee?', 'what forms can i fill out if i want to terminate a student?', 'tell me the proper form to terminate a new hire trainee?', 'what forms should i fill out to terminate a traineetrainer?', 'what forms do i need to fill out to terminate a new hire trainee?', 'What forms do I need to fill out to terminate a new hire trainee? ']	,
['how do i get a separation?', 'which information is necessary for a separation request?', 'tell me the details required for a separation?', 'What is the required information to properly request a separation? ']	,
['what information should be used to request a term?', 'what information is required to request a term?', 'tell me the required information required to request a term?', 'What is the required information to properly request a term? ']	,
['what documentation are required to request termination?', 'tell me the information required to request a termination?', 'What is the required information to properly request a termination? ']	,
['how do i process an edi form on behalf of an organisation?', 'how do i process a term request?', 'What were the proper steps to process a term request EDI form completion etc ']	,
['how do i proceed with the edi form process?', 'how can i complete a term request edi form?', 'what steps need to be taken to complete the term request edi form?', 'how do i process a term request edi form?', 'what are steps to complete a term request form using edi?', 'tell me the proper steps to complete a term request edi form?', 'What were the proper steps to process a term request EDI form completion?']	,
['how can i fix a windows accessibility issue in my class?', 'how do i fix windows access in a newer class?', 'how do i resolve a windows access issue?', 'how do i resolve windows accessibility issues in a school?', 'how can you resolve your windows access issue in a class?', 'how do i resolve windows issues for my class?', 'how can i resolve a windows access issue in a class?', 'how do i resolve a windows access issue in a class?', 'How do I resolve Windows access issues in my class?']	,
['how do you submit a ticket for an lan id lockout?', 'how can i submit a ticket for lan id lockouts?', 'How do I submit a ticket for LAN ID lockouts?']	,
['how can i get my windows unlockable by submitting a ticket?', 'how do i create a ticket for unlocking a windows account?', 'how do i submit a ticket to unlock my windows live account?', 'how can i submit a ticket to unlock my windows account?', 'how do you submit a ticket to unlock a windows account?', 'How do I submit a ticket to unlock a Windows account?']	,
['how can i get a ticket for a lockout on a computer?', 'how do you ticket someone who is locked out of their computer?', "how do i submit a ticket if i'm locked out of my computer?", 'how do i submit tickets when trainees are locked out of their computer?', 'how do i submit a ticket when a trainee has been locked out of the computer?', 'How do I submit a ticket when a trainee is locked out of their computer?']	,
['tell me the best way to unlock an acd id?', 'how can i unlock my acd id?', 'what should i do to unlock my acd id?', 'how do i unlock an acd id?', 'How do I unlock an ACD ID?']	,
['tell me the best way to get your trainees to work with windows?', 'how do you get new trainees out of windows?', 'what if my trainees are locked out of windows?', 'what should we do when your trainees are locked out of windows?', 'what should i do when my trainees are locked out of windows?', 'what should i do if my trainees are locked out of windows?', 'What should I do if my trainees are locked out of Windows?']	,
['how do you start creating virtual training courses?', 'how can i set up a virtual training program?', 'how do i setup a virtual training system?', 'how do i get virtual training?', 'how can you set up virtual training?', 'how do i set up virtual training?', 'How do I set up virtual training?']	,
['how do you take online courses?', 'how do i get started in the virtual world?', 'how do you do a virtual training program?', 'how can i go about virtual training?', 'list some ways to do virtual training?', 'how can i do virtual training?', 'how do i do virtual training?', 'how do i do virtual training?']	,
['how can i start a virtual training program?', 'how do i prepare for the virtual training?', 'how do i arrange for virtual training?', 'how do i start virtual training?', 'how do i set up virtual training?', 'how do i set up virtual training?']	,
['what are your instructions for submitting a ticket with reset password and user name?', 'how do you reset a password?', 'what should i do to reset my password for my ticket?', "what's the best way to submit a password reset ticket?", 'How do I submit a reset password ticket correctly?']	,
['can i get a ticket with a forgotten password?', 'I forgot my password, what catergory should I submit the ticket under?']	,
["i need to reset my password but i don't need to submit a ticket to change my password will i be able to do so?", 'i forgot my password but i still want to get a ticket do i need to reactivate it?', 'i forgot my password after i submitted a ticket to them do i have to submit my ticket again?', 'how do i reset my password?', 'do i need a reset password to enter a ticket?', 'i need a reset of my password do i have to submit a ticket?', 'i need to reset my password do i need to submit a ticket?', 'do i need a password reset to submit a ticket?', 'i need a password reset do i need to submit a ticket?', 'I need a password reset. Do I need to submit a ticket?']	,
['how can i get a trainee back into the computer if he is locked out?', 'tell me the best way to get a trainee back in the pc?', 'how do you get your trainee back on a computer?', 'how do i get my trainee back on the computer?', 'how do i get my trainee back into the computer?', 'how can i get my trainee back to the computer?', 'My trainee is locked out. How can I get them back into the computer?']	,
['how do i reset my password?', 'tell me the best ticket to reset a password?', 'What catergory of ticket should I submit for a password reset?']	,
['how do i change my password?', 'how do you reset your password?', 'what are ticket types that can reset your password?', 'what should i do to reset my password?', 'What ticket should I submit to reset a password?']	,
['how can i unlock a computer from another person?', 'how can i submit a ticket for opening a computer?', 'how do i submit a ticket for unlocking a computer?', "how can i submit a ticket to unlock a trainee's computer?", "How do I submit a ticket to unlock a trainee's computer"]	,
["my new hire doesn't know how to access cco learning?", 'can a new hire access cco learning?', 'how can i help my new hires access cco learning?', 'can someone in my office not access cco learning?', 'is there anyone who can access cco learning? what should i do?', 'One of my new hires cannot access CCO Learning.  What do I do?']	,
['how do i access cco learning?', 'what should i do if i cannot access cco learning?', 'how do you contact a participant who cannot access cco learning?', 'One of my participants cannot access CCO Learning.  What do I do?']	,
["what should i do if i can't find any course material in cco learning?", 'how do i make sure that someone can access cco learning?', "what should i do if i can't access cco learning?", "what should i do if one of my trainees can't access cco learning?", "what can i do when one of my trainees can't access cco learning?", 'One of my trainees cannot access CCO Learning.  What do I do?']	,
['can you list the numbers of help desks?', 'tell me the helpdesk number?', 'tell me the help desk number?', 'can you list the helpdesk number?', 'which is the helpdesk number?', 'what is a help desk number?', 'tell me the nearest help desk phone number?', 'what is an help desk phone number?', 'what is a help desk phone number?', 'What is the Help Desk phone number?']	,
['how can i find out who is doing the issue and which department to go to?', 'how can we identify the best departments or departments?', 'how do you find the best departments in a complex situation?', 'how do i find the right department to contact?', 'what department should i go to for help?', 'how do we know which department to reach out for?', 'How do we know what department to reach out to for different issues?']	,
['can you list the departments to whom you need to reach?', 'how can we know which department to contact?', 'can you list the best departments to reach out to?', 'how do i find out which department i should contact?', 'how do i know which department i should contact first?', 'how can i know what departments to reach out to?', 'show some good ways to find out which department we can reach?', 'how can i find the department we should reach out to?', 'how can we find out which department we should reach out to?', 'is there a good way to know which department we should reach out to?']	,
['where can i find help?', 'how do i find out which department i should contact?', 'which are the best services to reach out to?', 'which is the best department to talk to?', 'how do i find the best department?', 'how do i find the best department to reach out to?']	,
['how can i find the support office to call if an issue is important?', 'how do i reach out to the right department?', 'how do you find the right department to contact?', 'can you list the best departments to contact for an issue?', 'how do you know which department to reach out to if you have an issue?', 'how do you know which department to contact for an issue?', 'how do you know which department to reach out to for your issues?', 'how do you know which department to reach out to for an issue?', 'how do you know which department to reach out to for an issue?']	,
['how should i complete an edi?', 'how can i fill an edi?', 'what can be required to complete an edi?', 'how do i fill an edi?', 'how can i fill out an edi?', 'how do i fill an edi and receive the required information?', 'how do you fill an edi?', 'how do i fill out an edi?', 'how do you submit an edi form?', 'How do you properly fill out an EDI? What information is required?']	,
['what can you do to complete the disbursement information form?', 'tell me the best way to fill a discharge information form?', 'can i complete an employee discharge information form?', 'How do you properly fill out Employee Discharge Information Form? ']	,
['can you list the required information to fill in an edi form?', 'what should i bring to fill the edi form?', 'what information should i provide to fill out an edi form?', 'what information do i need for filling an edi form?', 'what information is needed to fill out the edi form?', 'What information is needed to fill out the EDI Form?']	,
['what is required to fill out the employee discharge information form?', 'what information should i provide to fill out the employee discharge information form?', 'tell me the main information that is required to fill out the employee discharge information form?', 'What information is needed to fill out the Employee Discharge Information Form?']	,
['how can you remove a course from cco learning?', 'how can we remove the curriculum from cco learning?', 'how can i remove a curriculum from cco learning?', 'how can i remove curriculum from cco learning?', 'how do you remove curriculum from cco learning?', 'How do you remove curriculum from CCO Learning? ']	,
['how do i delete content from a cco learning plan?', 'can i remove materials from my cco learning plan?', 'can you remove material from a cco learning plan?', 'how do you remove materials from cco learning plans?', 'how do you remove materials from a cco learning plan?', 'How do you remove materials from a CCO Learning Plan?']	,
['how can i remove the material in my class?', 'how do i remove the material from my class?', 'how do i remove the wrong materials from my class?', 'i have added the wrong materials for my class how can i remove the material?', 'I added the wrong materials for my class. How can I remove the material? ']	,
['my class was assigned a wrong curriculum', 'my class was assigned the wrong curriculum', 'my class had a wrong curriculum', 'i was assigned the wrong curriculum to my class', 'i assigned the wrong curriculum to my class', 'the wrong curriculum was assigned to my class', 'The wrong curriculum was assigned to my class. ']	,
['is it safe to remove the wrong materials in my class?', 'do i have to remove the wrong materials from a class?', 'i got all my wrong material pushed out in class how can i remove them?', 'The wrong materials were pushed out to my class. Can I remove these? ']	,
['how can i complete a level 1 class?', 'what are ways to get a class to complete level 1s?', "How does my class complete Level 1's?"]	,
['what was the best way to complete level 1?', "How does my class complete Level One's?"]	,
['how do i finish a level 1 survey?', 'how do i complete a level 1 survey?', 'how do i complete a level one survey?', 'how can i complete a level one survey?', 'what are steps for level one surveys?', 'show the steps to complete a level one survey?', 'What are the steps for completing Level One Survey? ']	,
['what is a level 1 quiz? how do i submit it?', 'how do i complete a level 1 survey?', 'how do i complete a level i survey?', 'how can i complete the level i survey?', 'what steps must be taken to complete the level i survey?', 'tell me the procedure for completing the level i survey?', 'list the steps for completing a level 1 survey?', 'What are the steps for completing the Level I Survey?']	,
['how do i fill out the time card for trainees?', 'how should i fill out my trainees time cards?', 'what is required to fill the time card?', 'how do trainees fill out their time card?', 'How does my trainees fill out their time card?']	,
['how is my career as a trainer going?', "what's been your job as a trainer?", 'how do i perform as a trainer?', 'how have i been doing as a trainer?', 'How have I been doing as a Trainer?']	,
['is there a way to view level 1 assessment results?', 'how can i view a full level 1 evaluation?', 'show some of my level 1 evaluations?', 'can i see my level 1 evaluations?', 'how do i view level 1 evaluations?', 'how do i view the level 1 level 1 evaluations?', 'how do i view my level 1 evaluations?', 'how can i view my level 1 evaluations?', 'where can i view my level 1 evaluations?', 'Is there a place I can view my Level 1 Evaluations?']	,
['is it possible to view my level one evaluations at your site?', 'how do i view a level one evaluation?', 'how can i view a level one evaluation?', 'can i view level one level one evaluations?', 'how do i view my level one evaluations?', 'is there a site where i can view my level one evaluations?', 'is there a place i can see the level one evaluations?', 'is there a place to view my level one evaluations?', 'Is there a place I can view my Level One Evaluations?']	,
['how do i get feedback about trainees?', 'how do i get feedback from my trainees?', 'how can i get feedback on my trainees?', 'where can i get feedback from trainees?', 'how can i find feedback from my trainees?', 'where can i find feedback from trainees?', 'Where can I find feedback from my trainees?']	,
['where are my level 1 evals?', 'where is my level 1 eval?', 'where are my level one evals?', 'Where is my Level One Evals?']	,
['where do i get my level 1?', "where are my level 1's?", "Where is my Level One's?"]	,
['how long can you retain a capa citation from your last capa?', 'how long will capa stay in your record?', 'how long does a capa stay in my account?', 'how long do capas stay on my records?', 'how long does the capa stay on my record?', 'how long did a capa stay on my record?', 'how long does capa stay on my record?', 'how long does a capa stay on my record?', 'How long does a CAPA stay on my record?']	,
['how long will it take to submit a corrective action?', 'how long does preventative action keep on my record?', 'how long will a corrective action from a prior action be on the op record?', 'how long does a corrective action be on my record?', 'how long does correctional action preventative action stay in your file?', 'how long does correction or preventative action stay in the record?', 'how long does corrective action preventative action stay on my record?', 'how long does a corrective action preventative action stay on my record?', 'How long does a Corrective Action Preventative Action stay on my record?']	,
['how many csrs are there in my class who can pass the final assessment?', 'how many csrs have to pass the final assessment?', 'how many csrs in a class must pass the final assessment?', 'how many csrs in class have to pass the final assessment?', 'how many csrs in a class have to pass the final assessment?', 'how many csrs in my class have to pass the final assessment?', 'how many csrs in my class have to pass the final assessment? ', 'How many CSRs in my class have to pass the final assessment? ']	,
['what proportion of csrss have to pass final examination?', 'how many students do i need to pass my final exam?', 'how many csrs in class pass the final examination?', 'how many students in my class pass final exam?', 'how many csrss do i need to pass the final exam?', 'how many csrss in my class have to pass final exams?', 'how many csrss in a class have to pass the final exam?', 'how many csrss in my class have to pass the final exam?', 'How many CSRSs in my class have to pass the final exam?']	,
['how many participants can pass the final exam in a class?', 'how many students will pass the final assessment?', 'how many students can pass the final assessment?', 'how many students in the same class have to pass the final assessment?', 'how many people in the class have to pass the final assessment?', 'how many people in a class have to pass the final assessment?', 'how many participants do i have to pass the final assessment?', 'how many participants in a class must pass the final assessment?', 'how many participants in a class have to pass the final assessment?', 'How many participants in my class have to pass the final assessment? ']	,
['how many students have passed the final exam?', 'how many people have to pass my final exam?', 'how many participants will pass my final exam?', 'how many students in a class can pass the final exam?', 'how many participants have to pass the final exam?', 'How many participants in my class have to pass the final exam?']	,
['how many trainees need to pass my class?', 'how many trainees should pass my final assessment?', 'how many trainees do i have to pass my final assessment?', 'how many trainees have to pass the final assessment?', 'how many trainees in a class must pass the final assessment?', 'how many trainees in a class have to pass the final assessment?', 'how many trainees in my class have to pass the final assessment?', 'How many trainees in my class have to pass the final assessment? ']	,
['how many students should pass final exam?', 'what number of trainees need to pass the final exam?', 'how many students in my class will pass the final exam?', 'how many trainees in a class in my class have to pass the final exam?', 'how many trainees are in class who have to pass the final exams?', 'how many trainees in my class pass the final exams?', 'how many trainees in my class have to pass my final exam?', 'how many trainees in my class have to pass the final exam?', 'How many trainees in my class have to pass the final exam?']	,
['can a csr miss an hour during training?', 'how many hours does a csr miss during training?', 'how many hours can a csr miss during training?', 'how many hours can a csr miss while training?', 'how many hours can a csr miss while in training?', 'How many hours can a CSR miss while in training? ']	,
['how many hours of training should a new hire miss?', 'how many hours in training do new hires miss?', 'how many hours should a new hire miss during training?', 'how many hours can a new hire miss during training?', 'how many hours can a new hire miss while training?', 'how many hours can a new hire miss while in training?', 'How many hours can a new hire miss while in training? ']	,
['how many hours a person misses during a training session?', 'how many hours does a participant have to miss while doing training?', 'how many hours can a participant miss during the training?', 'how many hours can a participant miss during training?', 'how many hours can one participant miss while training?', 'how many hours can a participant miss in training?', 'how many hours can a participant miss while in training?', 'How many hours can a participant miss while in training? ']	,
['how many hours can a trainee miss out on?', 'how many hours will a trainee miss in training?', 'how many hours can a trainee miss during their training?', 'how many hours a traineee can miss while in training?', 'how many hours can a trainee miss during training?', 'how many hours can a traineee miss during training?', 'how many hours can trainees miss while in training?', 'How many hours can a traineee miss while in training? ']	,
['can a new employee have an attendance policy?', 'what attendance policies do new hires go by while training?', 'what policies do new hires go by while they are in training?', 'what attendance policy does a new hire follow while training?', 'what attendance policies do new hires go by while they are training?', 'What attendance policy does a new hire go by while they are in training? ']	,
['what policy do participants follow during training?', 'what is the attendance policy of a student during the training?', 'what are attendance policies for a participant while training?', "what policy does a participant follow while he is in training? ''", 'tell me the attendance policy of a particiant who is in training?', 'what attendance policies do participants go by while in training?', 'what is a attendance policy for our particpants while they are in training?', 'What attendance policy does a particpant go by while they are in training? ']	,
["what is a trainee's attendance policy?", 'how is attendance policy enforced when trainees go through training?', 'tell me the attendance policy a trainee should follow in training?', 'show the attendance policies a trainee must follow while in training?', 'tell me the attendance policy a trainee follows when in training?', 'what attendance policy does a trainee follow during training?', 'what attendance policy does the trainee follow while in training?', 'what attendance policy does a trainee follow while in training?', 'tell me the attendance policy of a trainee while they are training?', 'What attendance policy does a trainee go by while they are in training? ']	,
['tell me the attendance policy in a class?', 'tell me the attendance policy of my class?', 'tell me the attendance policy for my classes?', 'tell me the attendance policy for my class?', "what's the attendance policy for my class?", 'What is the attendance policy for my class?']	,
['can new employees attend training?', 'what is the attendance policy for new employees while training?', 'list the attendance policy for new hires while training?', 'tell me the attendance policy for new hires while in training?', 'what is the attendance policy for new hires while in training?', 'can you list the attendance policy for new hires while they are training?', 'what is attendance policy for new hires while they are training?', 'tell me the attendance policy for new hires while they are in training?', 'What is the Attendance Policy for new hires while they are in training? ']	,
['where can i find a sample sop for the attendance policy?', 'can you list the sops that outline training attendance policies?', 'list the sops that outline the training attendance policy?', 'what are the sops that outline the training attendance policy?', 'where are the sops that outline the training attendance policy?', 'Where are the SOPs that outline the training attendance policy?']	,
['where can i find the attendance policy for new hires?', 'where can i find an attendance policy for new hires in training?', 'how can i find the attendance policy for new hires in training?', 'where can i find the attendance policy for new hires in training?', 'Where can I find the Attendance Policy for new hires in training?']	,
['how many observations should i take from the instructor?', 'what is a mandatory instructor observation number?', 'tell me the number of instructor observations that i have to make?', 'how many observations must i complete?', 'how many instructor observations should i take?', 'how many instructor observations should i complete?', 'how many instructor observations are necessary to complete?', 'how many instructor observations do i have to complete?', 'How many instructor observations am I supposed to complete?']	,
['how many peer audits are necessary?', 'how many peer audits should i take?', 'how many peer audits do i have to complete each year?', 'how many peer audits do i have to do?', 'how many peer audits should i complete?', 'how many peer audits do i have to complete?', 'how many peer audits am i supposed to complete?', 'How many peer audits am I supposed to complete?']	,
['how many peer observations should i do in a given year?', 'how many peer observations should i take?', 'how many peer observations should i make?', 'how many peer observations can i complete?', 'how many peer observations should i complete?', 'how many peer observations am i supposed to complete?', 'How many peer observations am I supposed to complete? ']	,
['tell me the requirement for complete audits?', 'why do i have to do audits?', 'why do i have to complete audits?', 'why do i have to complete an audit?', 'Why do I have to complete an audit?']	,
['tell me the requirement to do observation?', 'tell me the point of doing an observation?', 'why are we required to complete observation?', 'why do i have to complete an observation?', 'Why do I have to complete an observation?']	,
['how many questions do you have in level 1 evaluation?', 'how many questions are included in level 1 evaluations?', 'how many questions are there in the level 1 evaluation?', 'how many questions are in level 1 evaluation?', 'how many questions are on the level 1 evaluation?', 'how many questions are in the level 1 evaluation?', 'How many questions are in the Level 1 Evaluation?']	,
['how many questions are included in the level 1 exam?', 'how many questions in level one evaluation are there?', 'how many questions are there in level 1 evaluation?', 'how many questions are in level 1 evaluation?', 'how many questions are there in the level one evaluation?', 'how many questions are in level one evaluation?', 'how many questions are in the level 1 evaluation?', 'how many questions are in the level one evaluation?', 'How many questions are in the Level One Evaluation?']	,
['what is level 1 evaluation? what is it?', 'how does a level 1 evaluation work?', 'what is a level 1 appraisal?', 'tell me the value of level 1 evaluation?', 'what are level 1 evaluations?', 'what is level 1 evaluation?', 'what is level 1 evalution?', 'What is a Level 1 Evalution?']	,
['how do i start to know what a level 1 evaluation is?', "what's a level one appraisal?", 'what does level one evaluation mean?', 'how does a level one evaluation work?', 'tell me the level one valuation?', 'what is a level 1 evaluation?', 'what is level one evaluation?', 'What is a Level One Evalution?']	,
['when do i complete a level 1 evaluation?', 'when do i have to complete class level 1 evaluation?', 'when do i have to complete my grade one evaluations?', 'how do i get my class to complete the level one evaluations?', 'when do i have to complete level 1 evaluations?', 'when do i have to get my class to complete level one evaluations?', 'when do i have to have my class complete a level one evaluation?', 'when do i have to have my class complete level one evaluations?', 'When do I have my class complete Level One Evaluations?']	,
['who conducts my level 1 class?', 'who is responsible for helping me in my class to complete level 1?', 'who facilitates my classes during the level 1 exams?', 'Who facilitates my class completing Level 1s?']	,
['how do i find a suitable facilitator for level one?', 'who facilitates my classes when i want to complete level 1?', 'who guides my class in completing the level one?', 'who will facilitate my class completing level ones?', 'Who facilitates my class completing Level Ones?']	,
["what's the point of doing the level 1 level 1 for my class?", 'why must a class get level 1 for me?', 'why does my class require me to complete a level 1 test?', 'why do my classes have to complete level 1?', 'why does my class have to complete level 1 of my class?', 'Why does my class have to complete a Level 1 on me?']	,
['why is it for my class to have a level 1?', 'why is my class required to complete a level 1?', 'Why does my class have to complete a Level One on me?']	,
['how many hours am i allowed to teach?', 'How much OT am I allow to have as a instructor? ']	,
['how many hours in an ot do you have as a trainer?', 'How much OT am I allow to have as a trainer? ']	,
['can i work overtime at my school?', 'how much overtime is allowed in a teaching assignment?', 'how much overtime can i get as a faculty member?', 'how much overtime can i have as an instructor?', 'how many overtime am i allowed to have as a lecturer?', 'how much overtime can i allow as an instructor?', 'how much overtime am i allowed to have as a teacher?', 'how much overtime am i allowed as an instructor?', 'how much overtime am i allowed to have as an instructor?', 'How much overtime am I allow to have as a instructor?']	,
['tell me the overtime you can add as a trainer?', 'how much overtime i am allowed as a trainer?', 'how much overtime do i have to work as a trainer?', 'how much overtime am i allowed as a trainer?', 'how much overtime am i allowed to have as a trainer?', 'How much overtime am I allow to have as a trainer?']	,
['how long am i allowed to leave class for administrative tasks?', 'how much time do i have to spend after class for administrative tasks?', 'can i leave a class just after that to complete my administrative duties?', 'how much time do i have to stay after class to do administrative work?', 'how much time do i have after class to finish my administrative duties?', 'how much time am i allowed to stay after class for administrative duties?', 'how much time am i allowed to stay after class to finish my administrative duties?', 'how much time am i allowed to stay after class to complete my administrative duties?', 'How much time am I allowed to stay after class to complete my administrative duties?']	,
['is there time for new hires?', 'how much time do new hires have?', 'how much time should a new hire miss?', 'how much time a new hire trainee should miss?', 'how much time a new hire can miss?', 'how much time can a new hire trainee miss?', 'How much time can a new hire trainee miss?']	,
['how do i fire a guy who has been hired?', 'how do i fire someone after their first test?', 'how can i fire a new hire for failing?', 'If a new hire misses, can I fire them?']	,
['how do i fire a participant who missed this challenge?', 'how do you fire a participant who misses the question?', 'can i fire a participant when they miss?', 'can i fire a participant if he misses?', 'If a participant misses, can I fire them?']	,
['tell me the best way to fire a student?', 'can you fire a trainee who misses work?', 'if i miss a trainee how can i fire him?', 'If a trainee misses, can I fire them?']	,
['if a new hire is missing time for training will that mean they miss out?', "should i term a new hire when they don't leave time to train?", 'what do i do to a new hire if they have missed training?', 'how do you rename a new hire if they miss training?', 'how do i describe a new hire if he misses time in training?', 'how do i term a new hire if they miss training?', 'should i call a new hire if they miss time in training?', 'how do i term a new hire if they miss time in training?', 'Should I term a new hire if they miss time in training?']	,
['why is the term participant used to describe a person who missed training?', 'how do i describe a participant as missing time at training?', 'how should i name a participant if they miss time in training?', 'Should I term a participant if they miss time in training?']	,
["what's the best way to describe a trainee if they miss a lot of time during their studies?", 'should i describe a trainee in a trainee-type sentence as missing time in training?', 'should i describe trainees as missed training time?', 'tell me the best way to term a trainee when he or she is missing time in training?', "should i call my trainees when they're missing time in training?", 'should i term trainees when they miss training?', 'should i call a trainee if he missed the training?', 'how should i refer to a trainee if they miss their training?', 'should i call a trainee if they miss time in training?', 'Should I term a trainee if they miss time in training?']	,
['should i terminate a new hire?', 'why would you have to terminate someone if they missed a period of training?', 'how can i terminate someone who missed time on training?', 'should i terminate a fresh hire who has missed training?', 'is there a reason to terminate a job if they miss time in training?', 'should i terminate a new hire who misses time for training?', 'should i terminate a new hire if they miss training?', 'can you terminate a new hire if they miss time in training?', 'should i terminate a new hire if they miss time in training?', 'Should I terminate a new hire if they miss time in training?']	,
['is there any way to terminate a participant who misses time during the training?', 'should i terminate a student if he does not attend training?', 'should i remove a participant when they miss training?', 'should i terminate a participant who missed a deadline in training?', 'should i terminate the participant if they have missed training?', 'should i remove a participant if they miss their time in training?', 'should i terminate a participant if they miss training time?', 'should i terminate a participant if they miss training?', 'Should I terminate a participant if they miss time in training?']	,
['can i fire a trainee if they miss time in the training?', 'should i terminate a trainee if he misses class time?', 'what should i do if i terminate a trainee after missing the training?', 'should i terminate a trainee if he or she misses time in training?', 'should i terminate a trainee if he misses training?', 'should i terminate a trainee who misses time in training?', 'should i terminate a trainee if he misses time in training?', 'should i terminate my trainee if they miss time in training?', 'should i terminate a trainee if they miss time in training?', 'Should I terminate a trainee if they miss time in training?']	,
['what if a team fails to show up?', "what should i do if a participant doesn't show up?", "what do i do if all of my participants don't show up?", "what happens if all participants don't show up?", "what if all participants don't show up?", "what should i do if all my participants don't show up?", "what if all participants didn't show up what do i need to do?", "What if all my participants don't show up, what will I need to do?"]	,
['how much time should csrs spend on day 1 and day 0 courses?', 'how much time should csrs spend on day 0 and day 1 courses?', 'How much time should my CSRs spend on Day 0 and Day 1 courses?']	,
['how much time should i spend doing day zero or day one?', 'how much time should a csr spend on completing a day zero and day 1 course?', 'how much time should a csr spend on day zero and day one training?', 'how much time should csrs spend on day zero and day one?', 'how much time should the csrs spend on day zero and day one?', 'how much time should csrs spend on day zero and day one courses?', 'How much time should my CSRs spend on Day Zero and Day One courses?']	,
['what kind of time should my new hires spend on day 1 and day 0 training?', 'how much time should new hires spend on day 0 and day 1 courses?', 'how much time should my new hires spend on day 0 and day 1 courses?', 'How much time should my New Hires spend on Day 0 and Day 1 courses?']	,
['how much time should one person spend on day 1 and day zero courses?', 'how much time should new hires spend on day one courses?', 'how much time should my new hires spend on their day 1 and day zero courses?', 'how much time should new hires spend on day one and day zero courses?', 'how much time can a new hire spend on day zero and day one courses?', 'how much time should my new hires spend on day zero and day one courses?', 'How much time should my New Hires spend on Day Zero and Day One courses?']	,
['how much time does a student should spend on day 1 courses?', 'what time should i spend on day 0 and day 1 courses?', 'what is time should participants spend in the day 1 and day 0 courses?', 'how much time should i spend on day 1 and day 0 courses?', 'How much time should my particpants spend on Day 0 and Day 1 courses?']	,
['how much time should an undergraduate student spend on the day course?', 'how much time should participants spend in a day one or day zero course?', 'how much time should students spend on day zero or day one courses?', 'how much time should my students spend on day zero and day one courses?', 'How much time should my particpants spend on Day Zero and Day One courses?']	,
['how much time does your trainee have to spend on training in the first day?', 'how much time does a trainee need for day 1?', 'how much time should trainees spend on day 1 and day 0?', 'how much time should trainees spend in day 0 and day 1?', 'how much time should trainees spend on day 0 and day 1 courses?', 'how much time should my trainees spend on day 0 and day 1 courses?', 'How much time should my trainees spend on Day 0 and Day 1 courses?']	,
['how much time do i want my trainees to spend on training before they can go on day 1?', 'what time can i spend as a trainee on day 1 and day zero courses?', 'how much time should a day-0 trainee spend on their first day?', 'how much time should trainees spend on day one?', 'how much time should my trainees spend on course day zero?', 'how much time should my trainees spend on day one and day zero training?', 'how much time should my trainees spend in day one and day zero?', 'how much time should my trainees spend on day zero and day one?', 'how much time should my trainees spend on day zero and day one courses?', 'How much time should my trainees spend on Day Zero and Day One courses?']	,
['what must i do on the first day of a class in medicare as a new hire?', 'what should i do on the first day of my medicare new hire class?', 'what should the class do the first day of its medicare new hire class?', 'what should my class be supposed to do on the first day of the medicare new hire class?', 'What is my class supposed to do the first day of Medicare new hire class?']	,
['how much time is necessary to complete the compliance training in my medicare class?', 'what time is best to spend completing medicare compliance training?', 'can medicare students spend enough time completing compliance training?', 'how much time should medicare class participants spend on compliance training?', 'how many hours should a medicare compliance class spend in compliance training?', 'how much time should i spend with my medicare class to complete compliance training?', 'how much time should the medicare classes spend on compliance training?', 'what amount of time should my class spend in compliance training?', 'what amount of time should my medicare class spend completing compliance training?', 'What is the amount of time should my Medicare class spend completing compliance training?']	,
['how can i get the agenda for medicare day zero?', 'where can i find the day-to-day medicare agenda?', 'where can i find the medicare agenda for day zero?', 'where can i find the agenda for medicare day zero?', 'where should i find the agenda for day zero of medicare?', 'where can i find the agenda for day zero of medicare?', 'Where can I find the agenda for Day Zero of Medicare?']	,
['where can i find a new employees handbook?', 'how do i get in the first two days of medicare?', 'how do i get the handbook for medicare new hire?', 'where can i find a handbook for the first two days of the medicare new hire program?', 'can you list the handbooks for the first two days of medicare new hire?', 'Where can I find the handbook for the first two days of Medicare new hire?']	,
['tell me the best way to get people to discuss payroll incentives?', 'how do you address a question about payroll?', 'how do i get answers to my questions about pay and incentives?', 'how do i answer questions about payroll incentives?', 'how do i ask questions about payroll and incentives?', 'how can i address questions about payroll and incentives?', 'How to address questions about payroll and incentives?']	,
['tell me the best way to answer the question about incentives?', 'my class has questions about incentives and payroll?', 'does my class have questions about incentives and payroll?', 'My class has questions about incentives and payroll? ']	,
['which is the best site to find out how incentives are used?', 'what are your answers to any questions regarding incentives?', 'where can i find a complete answer to any question about incentives?', 'where do i find the answers for my questions about incentives?', 'how do i find the answers for questions about incentives?', 'where can i find answers to your questions about incentives?', 'where do i find answers to questions about incentives?', 'Where do I find answers to questions about incentives? ']	,
["what's the best way to ask a question regarding csr incentivebonus?", 'how do you answer questions about csr incentivebonus?', 'how do i answer the questions of csr incentive bonuses?', 'how do i answer all my incentives questions on csr?', 'list the easiest ways to answer any incentive bonus questions?', 'how do i answer a csr incentive bonus question?', "How to answer any csr's incentive\\bonus questions?"]	,
['is there a site where i can find answers to common questions about incentives?', 'where can i get helpful answers for questions about incentives?', 'where can i find answers to my questions about incentives?', 'can you help me with questions about incentives?', 'where can i find out about incentives?', 'where can i find faqs about incentives?', 'is there a place where i can find faqs for incentives?', 'is there a place i can find faqs for incentives?', 'Is there a place I can find FAQs for incentives? ']	,
['my class has questions about incentives', 'my class asks questions about their incentives', 'my class has questions about the incentives', 'my class has questions about their incentives', 'My class has questions about their incentives. ']	,
['how can i make a new employee get any incentives?', 'which incentives are provided to all new hires?', 'show the incentives for a new hire?', 'how much incentive can a new hire earn?', 'which incentives will my new hire class receive?', 'what incentives will my new hire class receive?', 'What incentives will my new hire class receive? ']	,
['how can i organize the essential tasks of a day to be prioritized?', 'how do you organize tasks affecting the entire day in a prioritized manner?', 'how can i arrange all my primary tasks in a systematic manner', 'how do you organize all the essential tasks of a day in a prioritized manner?', 'how to arrange the everyday essential tasks in a prioritized manner', 'how to arrange all day essential tasks in a prioritized manner?', 'How to arrange all day one essential tasks in a prioritized manner.']	,
['how do i prioritize everything in my day?', 'how do i prioritize my day tasks?', 'how do i prioritize what i do the most?', 'how do i prioritize tasks for a day?', 'what should be done to organize my daily essential tasks?', 'how do i assign tasks to priorities?', 'show some ways to organize an important task in a logical manner?', 'how do i organize every day tasks in a prioritized manner?', 'how do i arrange all day essential tasks in a prioritized manner?']	,
['how do i speed up ticket processes on my site?', 'how do you resolve issues that occur in a local administration?', 'how does one track access to a ticket via a local administrator?', 'how can i speed up a ticket through a site administrator?', 'how can i fast track ticket processing through a local site administrator?', 'how can i track tickets online through local site administrators?', 'how can i quickly track tickets with local site administrators?', 'how can i fast track tickets through site administrators?', 'How to fast track tickets through Local Site Administrators ?']	,
['tell me the fastest way to book lsa tickets fast?', 'how can i book a fast track lsa flight?', 'how can i fast track a ticket through lsa?', 'how do i get fast track tickets through lsa?', 'How to fast track tickets through LSA ?']	,
['is there a way to prioritize a ticket?', 'is there any way to have my ticket prioritized?', "i've been waiting long for a ticket addressed by the local site admin is there a way to prioritize it?", 'i have been waiting for years to have my ticket addressed by local site administrators how can i prioritize it?', "i've been waiting for a long time to have my ticket addressed by the local site administrator is there a way to have it prioritized?", 'i have been waiting for a long time to get my ticket addressed by a local site administrator is there a way to have it prioritized?', 'i have been waiting for some time to have my ticket addressed by the local site administrator is there a way to have it prioritized?', 'I have been waiting a long time to have my ticket addressed by Local Site Adminstrator. Is there a way to have it prioritized? ']	,
['how do i prioritize a ticket?', 'how do you make your ticket priority?', 'how do i make my ticket prioritized by lsa?', 'how can i have my ticket prioritized?', 'how do i have my tickets prioritized by the lsa?', 'i have been waiting a long time for my ticket to be addressed by lsa is there a way to prioritize it?', 'i have been waiting for a long time to have my ticket addressed by lsa is there a way to prioritize it?', 'i had been waiting for a long time to have my ticket addressed by lsa is there a way to have it prioritized?', 'I have been waiting a long time to have my ticket addressed by LSA. Is there a way to have it prioritized? ']	,
['how do you address a ticket?', 'what should be done when your ticket is rushed?', 'what should i do if i need to go to my ticketing service sooner?', 'how do i get my ticket resolved quicker?', 'how can i get a ticket faster?', 'how can i get my ticket dealt with faster?', 'how do i get the ticket addressed quicker?', 'how can i get the ticket answered faster?', 'how do i get my ticket addressed quicker?', 'Is there a way to get my ticket addressed faster? ']	,
['how do you fast track a lsa ticket?', 'how do i speed up my lsa ticket?', 'how can i accelerate my lsa ticket?', 'how do i fast track an lsa ticket?', 'how do i make a lsa ticket work more efficiently?', 'My LSA ticket is a priority, how can I fast track it? ']	,
['how do i handle unexpected internet setbacks?', 'how do i react to unexpected webex and internet problems?', 'how do you manage a webex emergency situation in a day?', 'how can one deal with unexpected internet problems?', 'how do i deal with unexpected webex and internet problems?', 'how do i deal with unexpected webex and internet disruption?', 'how do you deal with unexpected setbacks in webex?', 'How to properly deal with WebEx and Internet unexpected setbacks that occur throughout the day?\xa0']	,
["how can i address unexpected internet problems while i'm still on the internet?", "can you list the ways of dealing with unexpected issues with the internet while i'm still web-based?", 'how can i deal with internet problems while still web-based?', 'how can i deal with unexpected internet issues while online?', "how do i deal with unexpected internet problems while i'm still using the web?", "how do i deal with unexpected internet issues while i'm still web-based?", "how do i deal with unexpected internet issues while i'm still web-based?"]	,
['how do i deal with internet glitches?', 'how do you handle unexpected problems with your internet connection?', 'how do you deal with unexpected internet and webex outages?', 'show the best ways to overcome unexpected webexinternet downtime?', 'how do i deal with unexpected webex and internet failures?', 'how do i deal with unexpected webex and internet setbacks?', 'how do i handle unexpected webex or internet setbacks?', 'how do i handle unexpected webex and internet setbacks?']	,
['list some ways to deal with unexpected issues with webex?', 'can you list the ways to deal with unexpected setbacks with webex?', 'tell me the best way to overcome unexpected setbacks with webex?', 'can you tell me the best way to deal with unexpected setbacks?', 'tell me the best way to deal with unexpected loss with webex?', 'tell me the best way to cope with unexpected setbacks in webex?', 'show me the best ways to deal with unexpected setbacks with webex?', 'tell me the best way to deal with unexpected setbacks with webex?', 'tell me the best way to deal with unexpected setbacks with webex?']	,
['how can i submit my tickets to ticketmaster and check the status of my tickets?', 'how can i submit tickets and check on the status of them or list the people we can reach out to about them?', 'How to properly submit tickets and check the status of them or have a list of people we can reach out\xa0to regarding them.\xa0']	,
['how can i submit tickets online and check the status of them or have a list of people that we can contact about them?', 'How to properly submit tickets and check the status of them or have a list of people we can reach out\xa0to regarding them.\xa0']	,
['how do you follow the day 1 and day 2 agendas?', 'how do i follow the agenda for day 1 and day 2?', 'how can you understand and follow the agenda for day 1 and day 2?', 'how can i understand and follow the agenda for day 1 and day 2?', 'how do i understand and follow the agenda for day 1 and day 2?', 'How to understand and follow the agenda for Day 1 and Day 2?']	,
['how do you understand the day one and day two agenda?', 'how do i follow and understand the agenda for day 1?', 'how to prepare for day one and day two?', 'how do you prepare for day one and day two?', 'what is the agenda for day one and day two?', 'How to understand and follow the agenda for Day One and Day Two?']	,
['i have trouble understanding the agenda for day 1 and day 2', 'i have trouble understanding the agendas for day 1 and 2', 'i am having trouble understanding the agendas for day 1 and 2', "i'm having trouble understanding the day 1 and 2 agendas", 'I am having trouble understanding the day 1 and 2 agendas.']	,
["i'm having trouble understanding the plans for the first and second day", "i'm having difficulty understanding the first and second agenda", 'i have trouble understanding the agenda of day two', 'i have trouble understanding the agenda of day 1', "i'm having difficulty understanding the two day agendas", 'i have a hard time understanding the first and second agendas', 'i have trouble understanding the day two - one agenda', "i'm having trouble comprehending the day one and two agenda", "i'm having trouble understanding the first and two agendas", 'I am having trouble understanding the day one and two agendas.']	,
['i have questions about the first and second day', 'i have questions about day 1 and day 2', 'i have questions about the day 1 and the day 2 agenda', 'i have questions about the day 1 and day 2 agenda', 'I have questions about the Day 1 and Day 2 agenda. ']	,
['i have questions about the agenda for day one and day two', "i have questions about the first day and second day's agenda", 'i have questions about the upcoming day one and day two', 'i have questions about day one and day two', 'i have questions about the day one and day two agenda', 'i have questions about the day one and day two agendas', 'I have questions about the Day One and Day Two agenda. ']	,
['i need help with the itinerary of day 1 and day 2', 'i need help with the program on day 1 and day 2', 'i need help with the agenda of day 1 and day 2', 'i need help with completing the agenda on day 1 and day 2', 'i need assistance in completing the agendas on day 1 and day 2', 'I need assistance with completing the agenda on Day 1 and Day 2. ']	,
["i need assistance with the day's agenda", 'i need help with the planning of day one and day 2 -', 'i need assistance in completing the agenda for day 1 and day 2', 'i need help with completing the agenda on day one and day two', 'i need assistance with completing the agenda on day one and day 2', 'I need assistance with completing the agenda on Day One and Day Two. ']	,
['tell me the best way to request a day off?', 'tell me the easiest way to request a day off?', 'if i want to take a day off how do i request one?', 'I am need a day off, how do I request a day off?']	,
['i want a holiday next friday which day should i ask for?', 'I am needing next Friday off, how do I request this day off?']	,
['how can i get a company to give me days off?', 'what should i do if i want some time off?', 'what are ways to request time off?', 'do i need to request time off?', 'can i request time off? what is the process?', 'I need to request time off.  What is the process?']	,
['will i get assigned to a co-instructor?', 'i am the co-instructor of a class will i get to have the co-instructor assigned to me?', 'I am the co-instructor of a class. Will I have the co-instructor assigned to me? ']	,
['will i be assigned any trainees?', 'will i have the trainees assigned to me as my co-trainer?', 'I am the cotrainer of a class. Will I have the trainees assigned to me? ']	,
['tell me the difference between teaching a class and cotraining?', 'tell me the difference between being the trainer of a class with a cotrainer?', 'how do you differentiate between a trainer and co-trainer?', 'what is difference between a trainer and a co-trainer?', "what's the difference between trainer and cotrainer?", 'tell me the difference between a trainer and a co-trainer?', 'tell me the difference between a co-trainer and a trainer?', 'when i am coaching a class with a cotrainer what is the difference?', 'I am the trainer of a class with a cotrainer.  What is the difference? ']	,
['tell me the difference between co-trainer and trainer?', 'tell me the difference between a co-trainer and a trainer?', 'tell me the difference between trainer and co-trainer?', 'tell me the difference between a trainer and a co-trainer?', 'tell me the difference between a trainer and cotrainer?', 'What is the difference between a trainer and cotrainer?']	,
['i am a coinstructor so where is the form?', 'I have just completed an instructor observation of my coinstructor. Where is the form?']	,
['I have just completed an instructor observation of my trainer. Where is the form?', 'I have just completed an instructor observation of my trainer. Where is the form?']	,
["i just completed my observation for the coinstructor where's the form?", 'I have just completed an observation of my coinstructor. Where is the form?']	,
['I have just completed an observation of my cotrainer. Where is the form?', 'I have just completed an observation of my cotrainer. Where is the form?']	,
['what are peer audit forms?', 'how can we conduct a peer audit?', 'how does one conduct a peer audit?', 'what forms should one take to conduct a peer audit?', 'what forms do i need to file for peer audits?', 'what is a peer audit? what forms are used to complete a peer audit?', 'what form do i use to complete a peer audit?', 'What form do I use to complete a peer audit?']	,
['how do i prepare for peer instructor classes?', 'how can i become a peer instructor?', 'what is a form used to fill out a peer instructor interview?', 'is there a way to do peer instructor obervation?', 'what do i need to do to complete a peer instructor interview?', 'what form do i use to complete a peer instructor program?', 'What form do I use to complete a peer instructor obervation?']	,
['how can i do peer iof?', 'what are forms for peer iof?', 'what form can i use to complete peer iof?', 'which form do i use to complete peer iof?', 'which form do i use to complete a peer iof?', 'What form do I use to complete a peer IOF? ']	,
['how do i do peer observation?', 'how does an observer complete a peer observation?', 'what forms must i use to complete a peer observation form?', 'what form can i use to complete a peer observation?', 'what forms do i use to complete a peer observation?', 'What form do I use to complete a peer observation?']	,
['who reads my feedback?', 'who reads feedback on training materials?', 'who reads my feedback on the training materials?', 'I just submitted feedback on the training materials. Who reads my feedback? ']	,
['who reviews my class surveys?', 'who will review the daily surveys?', 'how do i know who is answering my daily surveys?', 'My class is completing the daily surveys. Who reviews these?']	,
['What is a Training Operations Lead?', 'What is a Training Operations Lead?']	,
['can i get feedback after reviewing?', 'who get feedback from reviewing material?', 'who gets the feedback on the review?', 'how do i provide feedback to someone about a material review?', 'who gets feedback from a material review?', 'who receives the feedbacks on a review?', 'who receives feedback on a material review?', 'who receives the feedback when i review a material i am reviewing?', 'who receives feedback when i complete a material review?', 'Who receives the feedback when I complete a material review?']	,
['can you help me fix the daily training report?', 'what should i do if i have a messed up training report?', "how do i fix a problem with my training report if i can't fix it?", "do i have to make another daily training report if i can't fix it?", "I messed up my Daily Training Report.  Do I have to make another one if I can't fix it?"]	,
["can i make another one of these if i can't fix it?", "I messed up my DTR.  Do I have to make another one if I can't fix it?"]	,
["if i've messed up my daily training report will i have to make a new report?", 'i just messed up my training report do i need to create a new one?', "i've just messed up my daily training report do i need to create a new one?", 'Ooops!  I just messed up my Daily Training Report.  Do I need to create a new one?']	,
['Ooops!  I just messed up my DTR.  Do I need to create a new one?', 'Ooops!  I just messed up my DTR.  Do I need to create a new one?']	,
["if i can't modify my training report i'll be prompted to create a new daily training report", "i'm going to create a daily training report will i have to create a new report?", 'will i need to create a new daily training report if i mess up my system?', 'will i need to create a new daily training report if i mess up?', 'Will I need to create a new Daily Training Report if I mess mine up?']	,
['how do you create a new dtr if your dtr is broken?', 'do i have to create a new dtr after a previous one?', 'will i need to create a dtr if i mess up my dtr?', 'will i need to create a new dtr if i mess up my own dtr?', 'will i need to create a dtr if i messed up?', 'will i need to create a new dtr if i mess up my dtr?', 'will i need to create a new dtr if i mess up mine?', 'Will I need to create a new DTR if I mess mine up?']	,
['where can i find an application form for a call cert?', 'how do i get a form to fill for a ca?', 'where can i find my call certificate form?', 'I need a Call Cert form. Where do I find a form?']	,
['tell me the best template i can use when writing a callcert form?', 'what templates can i use to create my own call certificate form?', 'tell me the best site for online forms of certs?', 'where can i find a template for a callcert form?', 'how do i create a call cert form?', 'I need a Call Cert form. Where do I find a template?']	,
['what should i do if i need a form for call certification?', 'how do i get a form for a certification call?', 'where can i find a form to call certification?', 'what should i do to request a certification form?', 'which is the best website for a call certification form?', 'how do i fill out a call certification form?', 'can i get a call certification form?', 'how can i get a call certification form?', 'I need a Call Certification form. Where do I find a form?']	,
['where can i find a template for a call certification form?', 'tell me the template i need for my call certification forms?', 'how do i get a template for my call certification letter?', 'tell me the format of a call certification form?', 'how do you format a certificate form for call certification?', 'i need a call certification form where can i find a template for it?', 'i need a sample call certification form where can i find a template?', 'i need a call certification form where can i get a template?', 'I need a Call Certification form. Where do I find a template?']	,
['what can be the easiest way to apply for a call certification?', 'tell me the best form for calling certification?', 'tell me the best form to give for certification?', 'what form should i use for call certification?', 'what forms do i need for call certification?', 'What form do I use for call certification?']	,
['tell me the best way to process certification?', 'tell me the best csr form to certify?', 'how do i call a csr?', 'how can i certify a csr?', 'how do i certify a csr?', 'which form can i use to call for certification of csr?', 'tell me the best way to certify a csr?', 'what forms do i use to sign or register a csr?', 'which form should i use to certify a csr?', 'which form do i use to call certify a csr?', 'What form do I use to call certify a CSR?']	,
['list the forms required for calling a new hire?', 'which form should you use when certifying a new hire?', 'what are good ways to certify a new hire?', 'what is a form i could use to call certify a new hire?', 'What form do I use to call certify a new hire? ']	,
['how do you find an automated certification form?', 'where can i get a template call certification form?', 'where can i find a sample certification form?', 'tell me the best way to get a call certification form template?', 'where can i download a call certification form template?', 'Where do I find a Call Certification Form template?']	,
['i need ideas for icebreakers', 'i need a few ideas for icebreakers', 'i need some ideas for icebreakers', 'I need some ideas for icebreakers. ']	,
['i want to make a icebreaker with my class', 'i want to do an icebreaker for my class', 'i want to do an icebreaker with my class', 'I want to do an icebreaker with my class. ']	,
['send me icebreakers for my class', 'send me some icebreakers for my class', 'send me some icebreakers to use in my class', 'send me icebreakers to do with my class', 'send me some icebreakers to do with my class', 'Send me some icebreakers to do with my class. ']	,
['can you list the best icebreakers for the class?', 'can you list the best icebreakers for class?', 'can you list the best icebreakers for my class?', 'show some good icebreakers for class?', 'recommend some good icebreakers for a class?', 'list some good icebreakers for my class?', 'What are some good icebreakers for my class?']	,
['where can i find a good icebreaker for my class?', 'where can i find icebreakers for my class?', 'Where can I find some icebreakers for my class? ']	,
['i need ideas for revision activities', 'i need ideas for review activities', 'i need some ideas for review activities', 'I need some ideas for review activities. ']	,
['what is a daily review activity for my class?', 'is there a daily review activity for my class?', 'Is there a daily review activity for my class? ']	,
['can i write a daily review on what i know about my class?', "what's a daily review of my class?", 'what is a daily review that you can do in my class?', 'is there a daily review i can do with my class?', 'Is there a daily review I can do with my class? ']	,
['show the things you will review with your class before the final assessment?', 'can i review certain items with my class before the final assessment?', 'list the things i should review first with my class before the final assessment?', 'can you list the items that i need to review with my class before the final assessment?', 'list the items i should review in my class before the final assessment?', 'list the items that i should review with my class before the final assessment?', 'is there anything i should review with my class before the final assessment?', 'are there certain things i should review with my class before the final assessment?', 'is there specific item i should review with my class before the final assessment?', 'Is there specific items I should review with my class before the final assessment? ']	,
['show the relevant items for a final exam and what are the ones to review?', 'can you list the things i should prepare for my final exam?', 'list the essential items i have to review in my class before final examinations?', 'show the various things i should review in class before the exam?', 'can you list the things i need to review in my class before final exam?', 'can you list the things i should study in my class before the final exam?', 'is there something that i should review in my class before the final exam?', 'what items i should review with my class before the final exam? ', 'Is there specific items I should review with my class before the final examt? ']	,
['can you list the review guides that i should follow?', 'is there a review guide?', 'is there a review guide that we can use for the final assessment?', 'how do i get a guide to review before the final assessment?', "i'm getting ready for the final assessment what guide can i use?", "i'm preparing for my final assessment is there a review guide i can use?", 'i am getting ready for the final assessment is there a review guide i can use?', 'We are getting ready for the final assessment. Is there Review Guide I can use? ']	,
['which book review guide should i use if i am getting ready for the final exam?', 'can you recommend me the right review guide for the final exam?', 'i am in preparation for the finals exam can i use your guide to review the papers?', "i'm ready to face final exam can you list the reviews i can use?", "i'm preparing for the final exam is there a guide for revising?", "i'm getting ready for final exam is there any review guide i can use?", 'i am getting ready for the final exam is there a review guide i can use?', 'We are getting ready for the final exam. Is there Review Guide I can use? ']	,
['What do I find a daily review activity for my Gen Med? ', 'What do I find a daily review activity for my Gen Med? ']	,
['can you find a daily review activity for your medicare new hire class?', 'What do I find a daily review activity for my Medicare New Hire class? ']	,
['how can i do a daily review activity for my marketplace new hire class?', 'Where can I find a daily review activity for my Marketplace New Hire class? ']	,
['how do i get a review guide?', 'where can i get review guides for classes?', 'where can i find reviews for my class?', 'where can i find all review guides for classes?', 'where can i found review guides for my classes?', 'where can i get the review guides for a class?', 'where can i find review guides for a class?', 'where can i find the review guide for my class?', 'Where can I find the Review Guides for my class?']	,
['how can i do social activities from sharepoint?', 'list the best sites to use for social interactions in sharepoint?', 'what can i do if i need sharepoint for social learning?', 'can someone develop a sharepoint site for social activities in the classroom?', 'is there a central sharepoint site for social activities in the classroom?', 'is there a sharepoint web site for socially defined activities?', 'where can i find a sharepoint site for in class social activities?', 'Is there a central SharePoint site for in-class (socially dinstanced) activities?']	,
['list the best sharepoint apps for classroom social activities?', 'can you list the best sharing sites in classroom?', 'which is the best sharepoint platform to conduct social activities in classroom?', 'can you list the best sites to share social activities in school?', 'recommend some good sharepoint sites for social activities in a classroom?', 'show the best sharepoint sites for social activities in a classroom?', 'list the best sharepoint sites for social activities in a classroom?', 'can you list the best sharepoint sites for social activities in a classroom?', 'can you list the best sharepoint sites for social activities in a classroom?']	,
['is there a shared  shared point site for social activities?', 'can you list the sharepoint places for socialized activities in a class?', 'is there a central site for shared activities in class?', 'is there a central sharepoint site for social activity in class?', 'is there any central sharepoint site for socially ausstanced activities in the class?', 'is there a central sharepoint place for socially dinstanced activities in the class?', 'is there a central sharepoint for socially dinstanced activities in class?', 'is there a central sharepoint site for socially-disstanced activities in class?', 'is there a central sharepoint site for socially dinstanced activities in class?', 'is there a central sharepoint site for socially dinstanced activities in class?']	,
['does anyone know of a sharepoint site?', 'is there any sharepoint site?', 'is there a sharepoint site?', 'can you list the social sharepoint sites in this thread?', 'is there a sharepoint-based site?', 'is there a sharepoint site that caters for social activities?', 'do you have a sharepoint system in mind for socially dissistanced activities?', 'is there a sharepoint site for social activities?', 'is there a sharepoint site for socially dinstanced activities?']	,
['can you list the sharepoint points for doing socially dissectional activities in class?', 'can you list the sharepoints to share for socially disinterested activities in class?', 'can you list the sharepoints of social activities?', 'what is a sharepoint for socially discouraging activity in class?', 'can you list the sharepoints you use for socially depressed in class activities?', "there's a sharepoint for in-class social activity?", 'can you list the sharepoints for in-class social activities?', 'is there a sharepoint for in-class socially dinstanced activities?']	,
['can you recommend a list of steps that should be followed in your classroom for improvement?', 'can you list the steps to be followed in a classroom?', 'how do i get started in classroom management?', 'show the steps of classroom management?', 'list the steps to follow to achieve effective classroom management?', 'what is a proper checklist for classroom management?', 'how do i use a checklist for classroom management?', 'tell me the checklist i should use for classroom management?', 'how can i follow a checklist of steps for classroom management?', 'what is a checklist to use for classroom management?', 'Is there a checklist to use with the proper steps for classroom management?']	,
['how can i manage my classroom?', 'how can i ensure that i am managing my own classroom?', 'how do i maintain a good classroom?', 'how do you make good classroom management decisions?', 'how can i ensure a proper classroom management?', 'what should be the best way to do proper classroom management?', 'how can you ensure proper classroom management?', 'how do i ensure proper classroom management?', 'What steps should I take to ensure proper classroom management?']	,
['can you list the locations where the training sops are available?', 'where do we access sops?', 'is there any place where i can easily access the training sops?', 'where can i get access to training sops?', 'is there a place where you can access training sops?', 'is there a place to access training sops?', 'is there a place where i can access training sops?', 'Is there a place we can access Training SOPs?']	,
['what are training standard operating procedures? is there any training support?', 'do i need to access a websitewebsite where i can access the training standards of operating procedures?', 'can we access the training standard operating procedures?', 'where can i access training standard operating procedures?', 'is there any way to access training standards operating procedures?', 'is there a place where we can access training standardized operating procedures?', 'is there any place to access training standard operating procedures?', 'Is there a place we can access Training Standard Operating Procedures?']	,
['where can i find all of the td sops?', 'Is there one place to find all the TD SOPs? ']	,
['where is the td standard operating procedure list?', 'is there a single place for all td standard operating procedures?', 'Is there one place to find all the TD Standard Operating Procedures? ']	,
['is there any website where i can get basic information on training and development?', 'is there an easy site to get an overview of training and development procedures?', 'is there a single place for training and development policies?', 'can you find a repository of training and development sops?', 'can you list the training and development sops?', 'is there a place to find all the training and development sops?', 'is there one place to find all the training and development sops?', 'Is there one place to find all the Training and Development SOPs? ']	,
['where can i find a list of all the standard operating protocols for training and development?', 'can i locate all standard operating procedures in this category?', 'Is there one place to find all the Training and Development Standard Operating Procendures? ']	,
['list some of the best training documents for aits?', 'Where can I find SOPs for Training? ']	,
['where can i find a detailed training delivery sop?', "Where can we locate training delivery SOP's?"]	,
['how do you schedule day 1?', 'is there a day 1 schedule?', 'is there a schedule for day 1?', 'are there any plans for day 1?', 'is there any agenda for day 1?', 'is there an agenda for day 1?', 'Is there an agenda for Day 1?']	,
["what's your first day plan?", "what will be the day's agenda for day 1?", "what's your agenda for day 1?", 'can you list the agenda for day 1?', 'can you list the agendas for day one?', "what's the agenda for day one?", 'is there any agenda for day one?', 'is there an agenda for day one?', 'Is there an agenda for Day One?']	,
['what will we do first?', 'how do i get ready for day 1?', 'what should i do to succeed on day 1?', 'what are we supposed to do on day 1?', 'what do we need to do for day 1?', 'What all do we need to do on Day 1?']	,
['what would we do on the first day?', 'what should we do in the first day?', 'what will we do on the first day?', 'what should we do on day 1?', 'what should i do on day one?', 'What all do we need to do on Day One?']	,
['how will i start my first day of classes?', 'list the steps to start the first day of class?', 'which are the steps for the first day?', 'tell me the procedure for the 1st day of class?', 'show the steps for the first day of class?', 'can you list the steps for the 1st day of class?', 'What are all the steps for the 1st day of class?']	,
['recommend some good steps to take to start a class on the first day?', 'what are the steps to be taken in a first day of school?', 'list the steps to be taken on the first day of class?', 'list the steps of a first day of class?', 'show the steps in the first day of class?', 'list the steps for the first day of classes?', 'can you list the steps for the first day of class?', 'tell me the steps for the first day of class?', 'what are the steps for the first day of class?', 'What are all the steps for the first day of class?']	,
['tell me the best thing to do on 1st day?', 'how would you spend day 1?', 'what should i do on day 1?', 'what should we do on day 1?', 'what do we do on day 1?', 'What do we do on Day 1?']	,
['what should one do on their first day?', 'what should i do at the start of my day?', 'what should we do in the first day?', 'what are we supposed to do on day 1?', 'what are we going to do on day 1?', 'what should i do on day one?', 'what do we do day one?', 'what do we do on day 1?', 'what should we do on day one?', 'What do we do on Day One?']	,
['what does this 1 day lesson look like?', 'what should i do during my first day of class?', 'what is your routine on the 1st day of a class?', 'what can i do in one day of class?', 'what can i do on one day of class?', 'what do we do on the first day of class?', 'what do we do for the 1st day of class?', 'What do we do on the 1 day of class?']	,
['what do we do in a typical ffm class?', 'what should i do on the first day of ffm class?', 'what will you do on the first day of ffm class?', 'what should we do on a day of ffm class?', 'What do we do on the 1 day of FFM class?']	,
["what we're going to do the first day of gen med class?", 'what will we do in the first day of gen med class?', 'What do we do on the 1 day of Gen Med class?']	,
['show some of the things we can do at the marketplace class?', 'what are you able to do on the first day of a marketplace class?', 'what should we do in the 1 day marketplace class?', 'What do we do on the 1 day of Marketplace class?']	,
['what should i do during medicare classes?', 'what do we do during the day of the medicare class?', 'what do you do for a 1 day medicare class?', 'What do we do on the 1 day of Medicare class?']	,
['how do i prepare for a first day of school?', 'what we do in a first day of the classes?', 'what should we do on our first day of school?', 'what can i do on the first day of classes?', "what's what we do on first day of class?", 'what should i do on the first day of class?', 'what should we do on the first day of class?', 'what do we do on the first day of class?', 'What do we do on the first day of class?']	,
['how do you prepare for ffm class?', 'what should we do on the first day of ffm?', 'what can we do on the first day of a ffm class?', 'what do we do the first day of ffm class?', 'what should we do on the first day of ffm class?', 'what do we do on the first day of ffm class?', 'What do we do on the first day of FFM class?']	,
['what should i do on the first day of gen med?', 'what should i do on the first day of gen med school?', 'what should we do on the first day of the gen med class?', 'what do we do on the first day of the gen med class?', 'what should i do on the first day of gen med class?', 'what should we do on the first day of gen med class?', 'what do we do on the first day of gen med class?', 'What do we do on the first day of Gen Med class?']	,
['what does the first day of marketplace class look like?', 'what should i do first day in marketplace class?', 'what would we do if it were our first day of marketplace class?', 'what does it take on the first day of marketplace class?', 'what should i do on the first day of marketplace class?', 'what should we do on the first day of marketplace class?', 'what do we do on the first day of marketplace class?', 'What do we do on the first day of Marketplace class?']	,
['how do i start a medicare class on the first day?', "what's my first day of class in medicare?", 'how do you prepare for medicare class?', 'how do we prepare for medicare class?', 'what should i do the first day of a medicare class?', 'What do we do on the first day of Medicare class?']	,
['what is my class supposed to do on the first day of my ffm new hire class?', 'What is my class supposed to do the 1st day of FFM new hire class?']	,
['what should i do on the first day of gen med new hire class?', 'What is my class supposed to do the 1st day of Gen Med new hire class?']	,
['What is my class supposed to do the 1st day of Marketplace new hire class?', 'What is my class supposed to do the 1st day of Marketplace new hire class?']	,
['how does the first day of the medicare new hire class work?', 'what are class requirements for the first day of medicare new hire class?', 'what is my class supposed to do in the first day of my medicare new hire class?', 'What is my class supposed to do the 1st day of Medicare new hire class?']	,
['what should our new hire class do on the first day?', 'what should i do in class on first day of an ffm new hire class?', 'What is my class supposed to do the first day of FFM new hire class?']	,
['what can we expect in gen med med new hire class?', 'what do i need to do for the first day of a new hire class?', 'What is my class supposed to do the first day of Gen Med new hire class?']	,
['what should i do on the first day of the marketplace?', 'what will my class do on the first day of my marketplace new hire class?', 'What is my class supposed to do the first day of Marketplace new hire class?']	,
['what is day zero and what is its focus?', 'what will be the day zero agenda?', 'show me the scheduled date for day zero?', 'tell me the future plan for day zero?', 'tell me the plan for day zero?', "what's the schedule for day zero?", 'show me the plans for day zero?', "what's the plan for day zero?", 'tell me the agenda for day zero?', 'what is the agenda for day zero?', 'What is the agenda for Day Zero? ']	,
['where can i find the agenda for marketplace day 1?', 'where can i find the agenda for day one of the marketplace?', 'Where can I find the agenda for Day One of Marketplace?']	,
['where is the handbook of day 0?', 'where is the day 0 manual?', 'where is the day 0 handbook?', 'Where is the Day 0 Handbook?']	,
['which is the manual for day 1?', 'where is the handbook for day 1?', 'where can i find a day 1 handbook?', 'where is the day 1 manual?', "where's the day 1 handbook?", 'where is the day 1 handbook?', 'Where is the Day 1 Handbook?']	,
['where is the day one manual?', "what's the day one handbook?", 'how do i get the day one handbook?', 'where is day one handbook?', 'which one is the day one handbook?', 'where is the day one handbook?', 'Where is the Day One Handbook?']	,
['where is the zero day handbook?', 'where is the day zero manual?', 'where is the day zero handbook?', 'Where is the Day Zero Handbook?']	,
['how do i format a daily training report?', 'show some examples of daily training reports?', 'tell me the most appropriate example of the daily training report i can follow?', 'can you list the examples of a daily training report i can follow?', 'is there an example of a daily training report that i can follow?', 'is there an example of daily training reports i can follow?', 'is there an example of a daily training report i can follow?', 'Is there an example of a Daily Training Report I can follow?']	,
['tell me the dtr example?', 'show some examples of dtr?', 'show some of the examples of a dtr i can follow?', 'is there any example of a dtr i can follow?', 'is there an example of a dtr i can follow?', 'Is there an example of a DTR I can follow?']	,
['show some of the best daily training reports?', 'how can i get a sample daily training report for free?', 'how do i find a sample training report?', 'where can i get samples of daily training reports?', 'where can i find an example of a daily training report?', 'where can i find a sample daily training report?', 'Where can I find a sample Daily Training Report?']	,
['can you list the sample dtrs for new hire classes?', 'Where can I find a sample DTR for new hire class?']	,
['where can i find a dtr sample?', 'where can i find samples of dtr?', 'where can i get a sample dtr?', 'where can i find sample dtrs?', 'where can i find a sample dtr?', 'Where can I find a sample DTR?']	,
["how do you submit a ticket to my program for production if i don't have a production id?", 'does a participant not have a production id?', 'My participant does not have a production ID.  Do I submit a ticket?']	,
["can you send me a ticket even if i don't have production id?", "will you send me a ticket if you don't have a production id?", "is it ok to send a ticket if you don't have a production id?", "can you list the ways to send a ticket if i don't have an production id?", "should i send a ticket if i don't have the production id?", "should i send a ticket if i don't have a production id?", "why should i send a ticket if i don't have a production id?", "should i send a ticket if i don't have a production id?"]	,
["what will happen if the participant's production id is not logged in?", 'what happens if a participant in my production is not logged in?', 'what happens if my participant is not logged in?', "what happens if my participant doesn't have production id?", 'what happens if the participant is not logged into my production id?', 'what happens if a participant is not logged into my production id?', 'what happens if my participant is not logged into my production id?', 'what happens if my participant is not logged in to my production id?', 'what if my participant is not logged in to my production id?']	,
["what should i do if the participant doesn't have a production id?", 'tell me the best way to identify a participant who does not have a production id on a ticket?', "what do you do if a participant doesn't have production id when you submit a ticket?", "what happens if a participant doesn't have a production id when submitting a ticket?", 'what do i do if a participant does not have a production id when submitting a ticket?', 'what can i do if a participant does not have a production id when submitting a ticket?']	,
['why is the cco learning teaching material and curriculum not being put to the proper use?', 'should cco learning be the main focus when it comes to curriculum?', 'should cco learning provide a curriculum?', 'does cco learning provide the right curriculum?', 'why is cco learning not teaching the proper curriculum?', 'how should i integrate the curriculum in cco learning?', 'how are you pushing out the proper curriculum in cco learning?', 'Pushing out the proper curriculum in CCO Learning? ']	,
["does cco learn an example of a curriculum that isn't required for academic classes?", 'is cco learning an example of a curriculum not required for academic courses?', 'is cco learning an example of a curriculum not required for the proper academic course?', 'is learning in cco an example of a curriculum that is not required for the proper academic courses?', 'is cco learning a good example of a curriculum which is not required for the proper academic courses?', 'is cco learning a good example of a curriculum that is not required for the proper academic courses?', 'is cco learning a good example of a curriculum that is not required for the proper academic courses?']	,
['will this be a major change in the cco program?', 'will it be possible to have cco learning more or less than the curriculum?', 'will cco learning grow?', 'how can ccf enlightenment go beyond curriculum?', 'will cco learning expand beyond course work?', 'will cco learning be more than just curriculum?', 'does cco learning need to move beyond a curriculum?', 'cco learning goes beyond curriculum?', 'how does cco learning develop beyond curriculum?', 'can cco learning change beyond a curriculum?', 'will cco learning move beyond a curriculum?']	,
['We had a fire drill.  What Deltek Charge code should me class use for the time spent?', 'We had a fire drill.  What Deltek Charge code should me class use for the time spent?']	,
['tell me the deltek charge code?', 'which deltek charge code is used?', 'what deltek charge code does my class use?', 'What Deltek Charge Code does my class use?']	,
['how often should you do adminstrative tasks daily?', 'how many daily administrative duties should i undertake in one day?', 'list the administrative duties that i should do daily?', 'which administrative tasks should i do daily?', 'which administrative duties should i perform every day?', 'what administration duties should i do daily?', 'what administrative duties should i do daily?', 'What adminstrative duties should I complete daily?']	,
['how do you write an expectation list for delivery in a classroom?', 'list the delivery expectations in the classroom?', 'how do you figure out delivery expectations for the classroom?', 'where can i find out if there are delivery expectations for the classroom?', 'how can i find a list of delivery expectations for the classroom?', 'where can i find a list of delivery expectations for the classroom?', 'Where can I find a list of delivery expectations for the classroom?']	,
['how do i have an easy checklist to follow while implementing administrative expectations?', 'tell me the best way to follow an administrative expectations checklist?', 'tell me the best checklist to use when training a class?', 'where do i find a checklist of administrative expectations for training?', 'where can i find a checklist of administrative expectations that i can follow while training my class?', 'where can i find a checklist of administrative expectations when taking a training class?', 'where can i find a easy checklist of administrative expectations for training a course?', 'where can i find an easy checklist of administrative expectations to use while training a class?', 'Where can I find an easy checklist of administrative expectations to use while training a class?']	,
['tell me the best thing you should do if your csr says he or she wants to resign?', 'tell me the best way to handle an csr resignation?', 'tell me the best way to deal with a csr who is resigning?', 'what do i do when a csr says they want to resign?', 'what should i do when a csr has resigned?', 'what do i do when a csr says they resigned?', 'what do i do when the csr tells me they resigned?', 'how do i do something when a csr tells me they resigned?', 'What am I supposed to do when a CSR tells me they resign?']	,
['when a new hire tells me they want to quit what should i do?', "how do i respond when someone says i'm leaving?", 'i am a new employee what do i do if someone resigns?', 'what should i do if i get hired and the new employee resigns?', 'what should i do if the new hire resigns?', 'what should i do when a new hire tells me they are leaving?', "what should i do when a new hire tells me he's resigning?", 'What am I supposed to do when a new hire tells me they resign?']	,
['tell me the best way to deal with resigning participants?', "when a participant tells me she's quit how do i get the answer?", 'what do i do if a participant quits?', "what's the best thing to do if a participant has resigned?", 'what do i do if a participant tells me he is resigned?', 'what will you do when a participant says they resign?', 'what should i do when a participant tells me they resign?', 'What am I supposed to do when a participant tells me they resign?']	,
['when a trainee says he resigned what do i do?', 'is it acceptable to lose a trainee?', 'what should you do if a trainee says they have resigned?', 'what should i do if a trainee says they have resigned?', 'how can i deal with a trainee who says they resigned?', 'what can i do when a trainee tells me to quit?', 'what should i do if a trainee tells me he resigns?', 'what should i do if a trainee tells me to resign?', 'What am I supposed to do when a trainee tells me they resign?']	,
['i have a trainee who i want to get out of this company what should i do if she wants to get out of this company?', 'when a trainee announces that they want to quit your company what should i do?', 'tell me the right thing to do after an aspiring trainee tells me to leave the company?', 'how do i contact a trainee if they want to resign from the company?', 'what should i do if a trainee told me that they would like to leave the company?', 'how do i move on when a trainee tells me they want to resign from the company?', 'what should i do when a trainee tells me they want to leave the company?', 'what should i do when a trainee tells me she wants to resign from the company?', 'What should I do when a trainee tells me that they want to resign from the company?']	,
['what is the role of a nesting poc? what do i expect from a nesting poc?', 'tell me the role of a nesting poc?', 'what should i expect when i am a nesting poc?', 'What are my expectations when I am a Nesting POC?']	,
['can you list the expectations in a nesting poc?', 'tell me the value of a nesting point of contact?', 'what do i expect from a nesting point of contact?', "show some expectations when you're a nesting point of contact?", 'What are my expectations when I am a Nesting Point of Contact?']	,
['what is the role and responsibilities of a leader at nesting?', 'what is my role in nesting?', 'how can i meet the expectations for nesting?', 'what should i expect when i go for nesting?', "what are my expectations when i'm hired by nesting?", "what are my expectations when i'm assigned to nesting?", 'What are my expectations when I am assigned to Nesting?']	,
['show the expectations of a person working in an ict course?', 'tell me the most important aspect of training?', 'what is the expectations of a teacher while training?', 'what should i expect during a training session?', 'what are my expectations for a class?', 'What are my expectations while I am training a class?']	,
['what can i do when i am not in class?', "what do i do when i'm not in a class?", 'can you list the responsibilities of being in a non-class?', "what's my responsibilities when i'm not assigned to a class?", 'what are my responsibilities when i am not assigned to classes?', 'what are my responsibilities when i am not assigned to a class?', 'What are my responsibilities when I am not assigned to a class?']	,
["what is my training team's standard of work?", 'show the standards we must follow as a training team?', 'what should we expect from a training team?', 'what are our goals as a training team?', 'what are our expectations as a training team?', 'What are our expectations as a training team?']	,
['show expectations for training personnel?', 'can you list the expectations for training?', 'show us the expectations for the training staff?', 'can you list the expectations for training staff?', 'show the expectations of training staff?', 'show the expectations for training staff?', 'show the expectations for training staff?']	,
['what do u expect in training?', 'what should i expect in training?', 'how much should i expect in the training team?', 'what should i expect as a training team?', 'what should be my expectation in training?', 'what can we expect from a training team?', 'what can be my expectation as a training team?']	,
['What are the main courses that need to be completed on Workday and MyLearning for Day 1, like mandatory?', 'What are the main courses that need to be completed on Workday and MyLearning for Day 1, like mandatory?']	,
['what is mandatory on day 1?', 'list the mandatory classes that must be completed on the first day?', 'what courses need to be completed on day 1?', 'can you list the mandatory courses that must be completed on day 1?', 'which are the mandatory courses to be completed on day one?', 'list the mandatory courses that must be completed on day one?', 'show the mandatory courses that must be completed on day one?', 'what are mandatory courses which need to be completed on day one? a', 'What are the mandatory courses that need to be completed on Day One? ']	,
['list the courses which must be completed on the first day?', 'list the courses required on day one?', 'can you list the courses that must be completed by day 1?', 'list the courses to be completed on day 1?', 'list the required courses for day 1?', 'list the mandatory courses to complete on day one?', 'can you list the required courses for day 1?', 'list the classes required to complete on day 1?', 'What are the required courses to complete on Day 1? ']	,
['tell me the course required on day one?', 'can you list the required courses for day 1?', 'can you list the courses you have to complete on day one?', "what's the required course to complete on day one?", 'can you list the required courses to complete on day one?', 'What are the required courses to complete on Day One? ']	,
['list the courses which have to be completed on day 1?', 'which courses should be completed on day 1?', 'what courses must be completed on day 1?', 'what courses need to be completed on day 1?', 'What courses need to be completed on Day 1?']	,
['tell me the best way to choose an alias name?', 'what criteria should be met when selecting a alias name?', 'what are requirements to know about choosing a alias name?', 'show the basic requirements to pick an alias name?', 'what are requirements for an alias name?', 'list the requirements of using an alias name?', 'list the requirements to pick an alias name?', 'tell me the requirement for creating an alias name?', 'can you give me the requirements for picking an alias name?', 'What are the requirements for picking an alias name?']	,
['how do you get alias names from the agents in new hire training?', 'where can you find rulesexpectations for proper alias names when getting alias names from agents in the new hire training?', 'where do you find the rulesexpectations for proper alias names when getting alias names from the agents in the new hire training?', 'Where can you find the rules/expectations for proper Alias names when getting Alias names from the agents in new hire training?']	,
['tell me the procedure to follow if a trainee is sick?', 'what should a covarid trainee do if they say that they are sick?', "tell me the best treatment when a trainee says he's sick?", 'what happens to a trainee after she has come out sick for a procedure of covid?', 'how can i find out who is sick and how should i respond if the trainee tells me that they are sick during a covid procedure?', 'when i ask a trainer if they are sick in class what do i do?', 'what should be done when a trainee says they are sick in class?', 'What are the steps when a trainee tells me that they are sick in class (COVID procedures)?']	,
['how can a trainee get better during a clear2work assessment?', 'tell me the first thing you do if your trainee is failing the clear2work health assessment?', 'what should i do if my trainee fails the clear2work health assessment?', 'what do i do when a trainee fails the clear2work health assessment?', 'what steps should i take when a trainee fails the clear2work health assessment?', 'What steps am I supposed to take when a trainee fails the Clear2Work Health Assessment?']	,
['tell me the last day of class?', "what's the last day of class?", "what's the final day in class?", 'what should i do on the last day of the class?', 'what should we do on the last day of class?', 'what should i do on the last day of class?', 'what should we do on the final day of our class?', 'what do we need to do on the final day of class?', 'what are we supposed to do on the final day of school?', 'What are we supposed to do on the final day of class?']	,
['what do we do on our final day of school?', 'what should we do on the last day of school?', 'what should we do on the last day of classes?', 'what do we do on the last day of class?', 'what should we do on the last day of class?', 'what should you do on the last day of class?', 'what we have to do on the last day of class?', 'what do we have to do on the last day of class?', 'what are we supposed to do on the last day of class?', 'what are we supposed to do on the last day of class', 'What are we supposed to do on the last day of class']	,
['what duties can be completed on the final day of training for new hires?', 'What end of training class duties are due to be completed on the last day of a new hire class?']	,
['how does an instructor manage attendance policies?', "how should i adhere to my instructor's attendance policy?", "what are instructors' attendance policies?", 'tell me the attendance policy of an instructor?', 'what attendance policy should i follow as an instructor?', 'what attendance policy do i have as an instructor?', 'tell me the attendance policy i am responsible for as an instructor?', 'What attendance policy am I responsible for as a instructor?']	,
['tell me the attendance policy of a trainer?', 'tell me the attendance policy for a trainer?', 'what attendance policies should i be responsible for as a trainer?', 'What attendance policy am I responsible for as a trainer?']	,
['how are instructor observation catergories defined?', "what do instructors' observation catergories mean?", 'what do instructor observation catergories mean?', 'what do all instructor observation catergories mean?', 'What do all the instructor observation catergories mean?']	,
['how do observation catergories mean?', 'what do observation catergories mean?', 'what do all observation catergories mean?', 'What do all the observation catergories mean?']	,
['what do trainees in their personal files look for?', 'what do you want your trainee to include in their folder?', "list some of the things that should be included in the trainee's folder?", "what can be added to the trainees' folder?", "list the items in the trainee's folder?", "what should i include in my trainee's file?", "what should i include in a trainee's folder?", "what should i include in the trainee's folder?", "What do I include in the trainee's folder?"]	,
['show the documents that are included in a certification folder of a csr?', 'list the documents which go in a csr certification folder?', "list some documents in csr's certification folder?", 'what documents go in a certification folder?', "list the documents in a csr's certification folder?", "what documents are in a csr's certification folder?", "what documents go in a csr's certification folder?", "What documents go in a CSR's certification folder?"]	,
['how do i include the paperwork of employees in their work folders?', "can you list the documents an employee may need in a 'employee' folder?", "what document should be included in an employee's documents?", "how can you save paperwork in an employee's employee folder?", "which paperwork should i put in my employee's file?", "what paperwork should be on an employee's file?", "what paperwork can i include in an employee's employee folder?", "what paperwork should i place in my employee's folder?", "what paperwork do i have in my employee's folder?", "What paperwork do I include in an employee's folder?"]	,
['tell me the csr certification procedure i can follow?', 'what is a csr certification application form?', 'how do i obtain csr certification?', 'tell me the best form to apply for a csr certificate?', 'how do i apply for csr certification?', 'What form do I fill out for CSR Certification?']	,
['what is csr certification and what are the steps required to get it?', 'how do i get a csr certificate?', 'how do you get a csr certificate?', 'what is a csr certification?', 'what is csr certification?', 'what is csr certification? how do i get it?']	,
['can you list the various forms of csr certification?', "what's in csr certification?", 'tell me the csr certification form?', 'what are csr certification forms?', 'what is csr certified certification forms?', 'what are csr certification certification forms?', 'what are csr certification certification forms?']	,
['list some requirements to pass the csr certification?', 'list some dfa forms required for csr certification?', 'which forms should be completed in order to get the certificate from csr?', 'what forms need to be completed to gain csr certification?', 'list the correct forms to complete to get csr certification?', 'which forms must be completed for scr certification?', 'which form should be filled to get csr certification?', 'which certification forms should be completed to get csr certification?', 'which forms should be completed for csr certification?', 'which forms should be completed to get csr certification?']	,
['show me the formality of certification of a csr?', 'tell me the certification process for csr certification?', 'show the terms and conditions of the csr certification?', 'show me the criteria for csr certification?', 'tell me the process for csr certification?', 'can you list the formalities for csr certification?', 'tell me the formalities of csr certification?', 'tell me the formalities for csr certification?', 'tell me the formality for csr certification?', 'tell me the formality for csr certification?']	,
['what happens to my training report if i forgot to do it every day?', "what will happen if i don't read the training daily every day?", 'what would happen if you forgot to do the daily training report?', 'what should i do when i forget to check the daily training report every day?', 'what if i forgot to get a daily training report every day?', 'what will happen if i forget to do the daily training report every day?', 'what happens if i forget to do the daily training report every day?', 'What happens if I forget to do the Daily Training Report every day?']	,
["what should i do if i don't do dtr every day?", 'what can happen if i forget to perform dtr?', 'what happen if i forget to perform the dtr each day?', 'what should i do if i forget to do the dtr every day?', 'what will happen if i forget to do dtr every day?', 'what happens if i forgot to take the dtr every day?', 'what happens when i forget to do the dtr every day?', 'What happens if I forget to do the DTR every day?']	,
['what if my level 1 score is bad?', 'what should i do if i get a bad score on a level 1?', 'what happens if someone scores 1st level badly?', 'what will happen to the student who has a bad level 1 score?', 'what happens if a level 1 score is bad?', 'what should you do if you have a bad level 1 score?', 'what happens if you get a low level 1 score?', 'what will happen if i got a bad level 1 score?', 'What happens if I get a bad Level 1 score?']	,
['i have a bad score on level one what should i do?', 'what happens if you have a bad score in level one?', 'what happens if i get a bad grade at the level 1?', 'what happens when i get a bad score on level one?', 'what happens to a level one score?', 'what happens to a low level one score?', 'what happens if i get a bad level 1?', 'what should i do if i get a bad level one score?', 'What happens if I get a bad Level One score?']	,
['tell me the training milestones?', 'tell me the training milestone?', 'how do you identify a training milestone?', 'what exactly is a training milestone?', 'what is a training milestone?', 'What is a Training Milestone?']	,
['tell me the milestones in the training?', 'show me the defining moment of training?', 'can you explain the training milestones?', 'show me the training milestone?', 'tell me the training milestone?', 'tell me the training milestone?']	,
['what does training milestone mean?', 'tell me the training milestone?', 'what are training milestones?', "what's a training milestone?", 'what is a training milestone?', 'what is a training milestone?']	,
['what does contingency mean?', 'tell me the term contingency?', 'what is contingency?', 'What is contingency?']	,
['tell me the definition of contingency?', 'what is contingency and what are the definitions of contingency?', 'can you list the terms contingency?', 'what exactly is contingency?', 'what are contingencies?', 'what is contingency?', 'how is contingency defined?', 'what are contingencies? what are their terms?', 'what is contingency? what are its terms?', 'what is contingency? what are its terms?']	,
['what do contingencies mean?', 'what is contingency?', 'what are contingencies?', 'what are contingencies?']	,
['how do you define contingencies?', 'what does a contingency mean?', 'what exactly is contingency?', 'what is contingency?', 'what is contingency?']	,
['what does day 0 mean in medicare?', 'tell me the day zero of medicare?', 'what is day zero in medicare?', 'what is day 0 of medicare?', 'What is Day 0 of Medicare?']	,
['what is a day zero medicare? what do you think of it?', 'what do you think of day zero in medicare?', 'what is day zero for medicare?', 'what is day zero in medicare?', 'what is day zero of medicare?', 'What is Day Zero of Medicare?']	,
['how is information collected from the status report updated?', 'tell me the purpose of the weekly status report?', 'what do you think about the information provided in our weekly status report?', 'what happens to information you get in our weekly status report?', 'what would be done with the data in our status report?', 'what can i do with the information in the weekly status report?', 'what can i do with the information in our weekly status report?', 'what is done with the information in our weekly status report?', 'What is done with the information in our Weekly Status Report?']	,
["what's the effect on information stored in our wsr?", 'what is done with data in wsr?', 'what should i do with information on wsr?', 'what is done with the information we have in our wsr?', 'how do you deal with the information in our wsr?', 'what can i do with the information in our wsr?', 'what is done with the information in the wsr?', 'What is done with the information in our WSR? ']	,
['what is a weekly status report and why is it used?', 'tell me the purpose of the weekly status report?', 'what is a weekly status report used for?', 'what is the weekly status report used for?', 'What is the Weekly Status Report used for?']	,
['tell me the purpose of wsr?', 'what does wsr mean?', 'what can a wsr do?', 'how is the wsr used?', 'why is the wsr used?', 'what is wsr used for?', 'what is a wsr used for?', 'what is the wsr used for?', 'What is the WSR used for?']	,
["what's expected as a poc?", 'what should be expected in a class poc?', 'what is expected as a poc?', 'what are expected of a class poc?', 'what is expected in a class poc?', 'What is expected as a class POC?']	,
['tell me the basic definition of point of contact in a class?', 'what are points of contact in a class?', 'what is expected of a point of contact in a class?', 'what are expected from a point of contact?', 'what is a point of contact? what is expected as a class point of contact?', 'what is expected as a class point of contact?', 'What is expected as a class Point of Contact?']	,
['how much time should i spend on compliance training during my marketplace classes?', 'how much time should i spend completing marketplace compliance training?', 'tell me the time i should spend in a marketplace course on compliance training?', 'how much time should a marketplace class take to complete cpa training?', 'how much time should my marketplace class spend on compliance training?', 'tell me the amount of time a marketplace class should spend completing compliance training?', 'how much time should a marketplace class spend completing compliance training?', 'what amount of time should my marketplace class spend on compliance training?', 'What is the amount of time should my Marketplace class spend completing compliance training?']	,
['can you tell me how much time do i want to spend on compliance training for marketplace?', 'can you tell me how much time i should spend on compliance training for marketplace?', 'show me the time i should spend on compliance training for marketplace?', 'can you tell me how much time i should spend completing compliance training for the marketplace?', 'tell me the time i need to spend completing a compliance training for marketplace?', 'can you tell me how much time i should spend completing compliance training for marketplace?', 'tell me the time i should spend completing compliance training for marketplace?']	,
['how much time should i put into training for compliance?', 'how much time should we spend on training for compliance?', 'how much time should i spend on training for compliance?', 'how much time should i spend for compliance training?', 'how much time should i spend on compliance training?', 'how much time should i spend on compliance training?']	,
['how much time can i spend in a market training program?', 'how much time does it take to complete marketplace class compliance training?', 'how much time should we spend in compliance training?', 'how much time is needed to complete compliance training?', 'how much time should a classroom invest for compliance training?', 'how much time does a marketplace class need for compliance training?', 'how much time should a marketplace class have to complete the training?', 'how much time should a marketplace class have to complete compliance training?', 'how much time should a marketplace class have to complete compliance training?']	,
['can you list the minimum time it takes to spend in a compliance class?', 'what time a marketplace course should spend on compliance training?', 'tell me the amount of time a marketplace class should spend on compliance training?', 'what amount of time should a class spend on compliance training?', 'what amounts of time should a business class spend on compliance training?', 'how much time should a marketplace class spend in compliance training?', 'how much time should a marketplace class spend on compliance training?', 'what amount of time should a marketplace class spend on compliance training?']	,
['tell me the dress code for a particular class?', 'show the dress codes for a school class?', 'tell me the dress code in class?', 'tell me the dress code of class?', 'tell me the dress code for the class?', 'tell me the dress code for a class?', 'tell me the dress code for my class?', 'What is the dress code for my class?  ']	,
['what dress code is appropriate for a class?', 'which dress code is appropriate for my class?', 'list the dress codes in class?', "what's the dress code of my class?", 'show the dress codes for my class?', 'which dress code for school is appropriate?', 'which dress code in my class is appropriate?', 'what dress code for my class is appropriate?', 'which dress code for my class is appropriate?']	,
['tell me the dress code?', 'tell me the dress code of class?', 'show me the dress code of the class?', 'give me the dress code for the class?', 'show me the dress code for the class?', 'tell me the dress code for this class?', 'tell me the dress code for the class?', 'please let me the dress code for the class', 'tell me the dress code for the class?']	,
['can you tell me what my dress code is in my class?', 'can you list the dress code for the class?', 'show me the dress code of my class?', 'show me the dress code for my class?', 'tell me the dress code for the class?', 'tell me the dress code for class?', 'can you tell me the dress code for the class?', 'tell me the dress code for my class?', 'please tell me the dress code for my class?', 'tell me the dress code for my class?']	,
['tell me the dress code of the course?', 'tell me the dress code of a class?', "what's the dress code of a class?", "what's the dress code in my class?", "what's the dress code of my class?", "what's the dress code for a class?", 'what is dress code for a class?', 'what is a dress code for my class?', 'what is dress code for my class?', 'what is dress code for my class?']	,
['tell me the dtr naming convention?', 'tell me the naming convention of dtr?', 'tell me the name convention of a class dtr?', 'tell me the naming convention for a class name dtr?', 'tell me the naming convention for the class dtr?', 'What is the naming convention for my class DTR?']	,
["what's a good dtr name convention?", 'tell me the proper dtr naming convention?', 'can you list the correct name convention of dtr new hire classes?', 'what are best naming conventions for new hire classes?', 'show the best naming conventions for new hire classes?', 'tell me the appropriate dtr naming convention for new hire classes?', 'tell me the proper dtr naming convention for new hire classes?', 'What is the proper DTR naming convention for new hire classes?']	,
['what is ecl? how do i teach a trainee?', 'how do i prepare for delivering ecl?', 'how do i deliver my ecl to a trainee?', 'how do i deliver an ecl to a trainee?', 'show the methods for giving an ecl to a trainee?', 'tell me the procedure to deliver an ecl to a trainee?', 'tell me the process of delivery of ecl to a trainee?', 'tell me the process for giving an ecl to a trainee?', 'tell me the process of delivering an ecl to a trainee?', 'What is the process for delivering an eCL to a trainee?']	,
['how can i prepare for ece?', 'how can i prepare for ecl?', 'how do i prepare my student for the eclo?', 'how can i prepare my trainee for ecl?', 'how should i prepare my trainee for ecl?', 'how do i prepare my trainee for the ecl?', 'how do i prepare my trainee for an ecl?', 'how do i prepare my trainee to do an ecl?']	,
['can you list the processes of giving the eclcpl to the trainee?', 'can you list the stages of giving eclcpl to a trainee?', 'can you list the timelines and processes of giving eclcpl to a trainee?', 'can you list the processes and timelines for giving eclcpl to a trainee?', 'can you list the process and timelines of giving eclcpl to a trainee?', 'can you list the processes and timelines of giving eclcpl to a trainee?', 'can you list the processes and timelines of giving eclcpl to a trainee?']	,
['how do you teach english to a trainee?', 'how do you teach ecl to a trainee?', 'how do you deliver ecl to a trainee?']	,
["what is tat's team?", 'what is a tat team??', 'what is a tat team?', 'What is the TAT team? ']	,
['What is the Training ApplicatonsTeam team? ', 'What is the Training ApplicatonsTeam team? ']	,
['what link should i send to level 1 students?', 'tell me the link that goes with the class for level 1?', 'tell me the best way to send a class to complete level 1?', 'what link should i send to my classes to complete level 1?', 'What Link do I send my class to complete Level 1s?']	,
['what link can i send for students to complete level i?', 'what link can you send to a class to complete level ones?', 'What Link do I send my class to complete Level Ones?']	,
['how does a school assess for level 1?', 'where do i go for level 1 evaluation?', 'tell me the best place to do a level 1 evaluation?', 'what sites can i go to in level 1 evaluation?', 'which site can i go to to complete my level 1 evaluation?', 'where does my class go to complete a level 1 evaluation?', 'what site should my class go to complete a level 1 evaluation?', 'What site does my class go to complete a Level 1 Evaluation?']	,
['how do i complete a level one assessment?', 'which site will my class go to for the assessment of a level 1 project?', 'which site should i go to for a level one assessment?', 'what site can i go to do my level one exam?', 'which site is your class going to to complete a level 1 evaluation?', 'what sites do students go to to complete a level 1 evaluation?', 'what site do my students go to to complete level one evaluation?', 'which site does my class go to complete a level 1 evaluation?', 'What site does my class go to complete a Level One Evaluation?']	,
['what are service levels agreed upon when i train a class?', 'what service level agreements should i sign when i train a class?', 'what service level agreements do i have to create when i train a class?', 'list the service level agreements i should be responsible for when i train for a class?', 'what are service level agreements i should be responsible for when i train a class?', 'what are service level agreements i am responsible for when i train a class?', 'what service level agreements am i responsible for when i train a class?', 'What Service Level Agreements am I responsible for when I train a class? ']	,
['what slas do i have to be responsible for?', 'list the different slas i must follow when i teach a class?', 'What SLAs am I responsible for when I train a class?']	,
['when a trainee goes over the time allowed in training what do i do?', 'what should i do if a trainee goes over the amount allowed in the training session?', 'what should i do if my trainee goes over the time allowed in training?', 'what should i do when a student goes over the time permitted to miss in training?', 'what should i do if a trainee goes over the time allowed in the training?', 'what should i do when a trainee goes over the time allowed in training?', 'What should I do when a trainee goes over the time allowed to miss in Training?']	,
['how do i check for unplanned absences?', 'how do i get a checklist for an unplanned absence in class?', 'how does the checklist help when a trainee has over 8 hours of absence in class?', 'Where can I find the checklist to use when a trainee goes over 8.0 hours of unplanned absence in class?']	,
['what should i include in writing a written disciplinary action?', 'how do i write a disciplinary action for a cell phone violation?', 'what should i include in my disciplinary action letter for a cell phone violation?', 'what should i include in my disciplinary letter for a cell phone violation?', 'what can i include when writing disciplinary action for a cell phone violation?', 'what should i include when writing disciplinary action for cell phone violation?', 'what should i include when writing up a disciplinary action for a cell phone violation?', 'what should i include when writing a disciplinary action for a cell phone violation?', 'what should i include when writing up disciplinary action for a cell phone violation?', 'What should I include when writing up disciplinary action for a cell phone violation']	,
['how can i find verbiage in the classroom regarding cell phone violations?', 'how do you describe cell phone violations?', 'how do i report cell phone violations in class?', 'where is the verbiage for cell phone violations in the classroom?', 'where do i find verbiage for cell phone violations in the classroom?', 'where can i find verbiage for cell phone violations in the classroom?', 'Where can I find verbiage for cell phone violations in the classroom?']	,
['how do you approach an ecl presentation?', 'tell me the best way to do ecls for trainees?', 'tell me the process of delivering an ecl to a trainee?', 'what steps should i take while giving ecl to trainees?', 'what steps should i take when i deliver ecl to trainees?', 'what steps should i take when delivering an ecl to a trainee?', 'What steps should I take when delivering an eCL to a trainee?']	,
['how do i prepare for ecl?', 'how do i prepare for an ecl?', 'how do i prepare an ecl for a trainee?']	,
['how do i tell a trainee about my ecl?', 'how do you convey an ecl?', 'how do i deliver ecl to a trainee?', 'how do i deliver my ecl to a trainee?', 'how do i deliver my ecl to a trainee?']	,
['what should i do in order to do an ecl for a trainee?', 'can you list the steps to take when giving the ecl?', 'can you list the steps to follow to get an ecl for a trainee?', 'can you list the steps to follow when giving ecl to a trainee?', 'recommend some steps to take when giving an ecl to a trainee?', 'recommend some steps to take when giving an ecl to a trainee?']	,
['what should a trainee do if they need to have ecl?', 'recommend some of the steps to deliver ecl to a trainee?', 'show the steps needed for delivering ecla to trainees?', 'can you list the steps to taking to deliver ecl to a trainee?', 'show some of the steps you should take in delivering ecl to trainee?', 'what should one do when delivering ecl to a trainee?', 'what steps should be taken in delivering ecl to a trainee?', 'what steps should we take when delivering ecl to trainee?', 'what steps should one take when delivering ecl to a trainee?']	,
['tell me the daily tracker? what is it for?', 'how can i find daily trackers?', 'how can i find a daily tracker?', 'what is a daily tracker?', "what's the daily tracker?", 'Whats the daily tracker?']	,
['what should be my daily activity?', 'how do you track your daily activities?', 'how do you track your daily activity?', 'how do i keep track of my activity?', 'how do you keep track of your daily activity?', 'how do i keep track of my daily activities?', 'how can i keep track of my activity?', 'how do i keep track of my daily activity?', 'how can i keep track of my daily activities?', 'how can i get track of my daily activity?']	,
['what is a daily tracking program?', 'what is a daily tracker?', "what's your daily tracker?", "what's the daily tracker for?", "what's the daily tracker?"]	,
['when does maximus need to complete a course in policies and procedures?', 'when do maximus policies and procedure trainings have to be completed?', 'When do all of the Maximus policies and procedure trainings need to be completed?']	,
['can you list the maximus training courses?', 'how many maximus trainings do you need?', 'what maximus training you need to do to get your company to adhere to policies and procedures?', 'how many maximus courses do you need to complete for policy and procedure?', 'how many maximus training courses do you need to complete for the policies and procedures?', 'how many maximus trainings do you need to complete for the policies and procedures?']	,
['how do i find the roster for a class?', 'how do you find class rosters?', 'where can i find class rosters?', 'where can i find my class roster?', 'where is my class roster?', 'where are my class rosters?', 'Where are my class rosters?']	,
['tell me the credentials used by a csr to log in and retrieve login credentials?', 'how do i find login credentials for csr?', "how do i get csr's login credentials?", 'where can i find a csr login username?', 'where can i find csr login credentials?', 'how can i find a csr log in credentials?', "how can i find a csr's login credentials?", 'where can i find a csr log in credentials?', "where can i find csr's log in credentials?", "Where can I find a CSR's log in credentials?"]	,
['show some ways to view class rosters online?', 'how can i find a complete class list?', 'where can i see the full class roster?', 'how do i find the class roster?', 'how do i find my class roster?', 'where do i find my class roster?', 'where can i find my class roster?', 'Where can I find my class roster?']	,
["how do i find my participant's id for the acdcdw session?", 'where can i get acd id?', 'how do i get an acd id from a participant?', 'how can i find my acd id?', "how do i find a participant's acd id?", "where can i find my participant's acd id?", "Where do I find my participant's ACD ID?"]	,
['how do i find an employee number?', "how can i find my participant's employee number?", "how do i find my participant's employee number?", "Where do I find my participant's employee number?"]	,
['where can i get my lan identification number?', 'can someone give me a lan id?', "how can i find an individual's lan id?", "where can i find my user's lan id?", "where can i find the participant's lan id?", "Where do I find my participant's LAN ID?"]	,
['how can i find out if an individual is in the group pgs or not?', 'how do i find the pgs number?', 'how do i find my pgs number?', "which is my participant's pgs number?", "how can i find the participant's pgs number?", "how can i find my participant's pgs number?", "how do i find my participant's pgs number?", "Where do I find my participant's PGS number?"]	,
['where do i find the acd id of a trainee?', 'how can i find an acd id?', 'tell me the acd id for my trainee?', 'where can i find an acd id?', "how do i find my trainee's acd id?", "Where do I find my trainee's ACD ID?"]	,
["is there a simple way to find a trainee's employee number?", "how do i get a trainee's employee number?", 'how can i find my trainee employee number?', 'how do i find my trainee employee numbers?', "how do i find my trainee's employee number?", "Where do I find my trainee's employee number?"]	,
['how can i find my lan id?', "tell me the easiest way to get a trainee's lan id?", 'how do i find my lan id?', "Where do I find my trainee's LAN ID?"]	,
['how do i find my next generation id in a production software?', 'where can i find my next generation production id?', "where can i find the next generation delegate's production id?", 'where can i find my next generation desktop production id?', "where can i find a trainee's next generation  desktop production id?", "where can i find my trainee's next generation desktop production id?", "Where do I find my trainee's Next Generation Desktop Production ID?"]	,
['where can i find my ngd production id?', 'where can i find the ngd production id?', 'how do i find the ngd production id?', "how do i find my trainee's ngd production id?", "Where do I find my trainee's NGD Production ID?"]	,
['how do i find the production id of my trainee?', "how can you find a trainee's production id number?", "tell me the best way to get a trainee's production id?", "how do you find your trainee's production id?", "how do i get my trainee's production id?", "how can i find my trainee's production id?", "Where do I find my trainee's production ID?"]	,
["where do you find a trainer's windows username?", "how can i find my trainee's windows username and password?", "how do i find my student's login username for windows?", "how can i find my trainee's username for windows?", "where can i find my trainee's username for windows?", "Where do I find my trainee's username for Windows?"]	,
['where can i find a participant list?', 'where are the participant lists?', 'how do i access my participant list?', "where's the participant list?", 'where is my participant list?', 'Where is my participant list?']	,
['tell me the user information or login data for a new hire?', 'how do you log into a job for the first time?', 'where do you find the log in information for new hires?', 'show me the login log info of a new hire?', "can i verify my new hire's log in information?", "how can i get my new hire's login information?", "Where is new hire's log in information?"]	,
['tell me the login information of trainees?', "where can i find the trainee's log in information?", "Where is trainee's log in information?"]	,
['who does my class assign me to?', 'who assigned me to classes?', 'who is assigned to a class?', "who's assigned to my class?", 'who is assigned to my class?', 'Who is assigned to my class?']	,
['can you list the instructor guides for marketplace?', 'Where are my instructor guides for Marketplace?']	,
['show the course guides for the marketplace?', 'can you list the best instructors for marketplace?', 'show some of the instructor guides for marketplace?', 'can you list the instructor guides for marketplace?', 'which are the instructor guides for marketplace?']	,
['which are some good - online - training manuals for medicare?', 'where can i find the medicare instructor guide?', 'where can i find medicare instructor guides?', 'where can i get a guide on medicare?', 'where can i find the course guide for medicare?', 'where can i get an instructor guide for medicare?', 'where can i find my instructor guide for medicare?', 'where are our instructor guides for medicare?', 'Where are my instructor guides for Medicare?']	,
["what's the best way to get a handbook on medicare?", 'how do i download a guide for medicare?', 'how can i get a guide about best practices in medicare?', "what's the best practice guide for medicare?", 'how do i get a good knowledge manual on medicare?', 'how do you get a best practice guide for medicare?', 'how do you get a well practice manual for medicare?', 'how do i get a best practice manual for medicare?', 'how do i get a best practice manual for medicare?']	,
['where can i find information on medicare?', 'where can i find a medical training manual for medicare?', 'where can i buy an instructor manual for medicare?', 'where can i find the instructional manual on medicare?', 'where can i find an instructional manual on medicare?', 'where can i find an instructional manual for medicare?', 'where can i find an instructional manual for medicare?']	,
['Where are the participant guides for Marketplace?', 'Where are the participant guides for Marketplace?']	,
['can you list the guides for the participant market?', 'can you list the guides for the participants in the marketplace?', 'can you list the participants manuals for the marketplace?', "can you list the participants' guides to the marketplace?", "can you list the participants' guides for the marketplace?", 'can you list the participants guides for the marketplace?', 'can you list the participants guides for the marketplace?']	,
['are there any participant guide books or information guides available on medicare?', 'where can i get participant guides on medicare?', 'where are the participants guides for medicare?', 'where are the participant guides for medicare?', 'Where are the participant guides for Medicare?']	,
['where can i find a list of eligible participants for the medicare program?', 'how do i get entry on medicare?', 'where can i find my application list to enroll in medicare?', 'where can i find an entry list for participants in medicare?', 'where can i get entry forms for the medicare participants?', 'where can i find the list of medicare participation?', 'where can i find a list of medicare participants?', 'where can i find entry lists for medicare participants?', 'where can i find an entry list for medicare participants?']	,
['can you list the participation guide in medicare?', 'how can i get the participant guide in medicare?', 'how do you get a participant guide to medicare?', 'how do you get participant guides for medicare?', 'how do you get participant guides for medicare?']	,
['can you list the participants in medicare?', 'can you list the participant guide for medicare?', 'can you list the participants guides for medicare?', 'can you list the participant guides for medicare?', 'can you list the participant guides for medicare?']	,
['have you found a list of participant guides for the medicare?', 'where can i find participant guides for medicare?', 'can you list the participant guides for medicare?', "where are the participant's guide to medicare?", 'where are the participants guides for medicare?', 'where are the participant guides for medicare?', 'where are the participant guides for medicare?']	,
['where can i find a template of daily training reports?', 'where can i download a daily training report template?', 'where can i find a free training report template?', 'which is a best training report template to download?', 'where can i find a daily training report template?', 'where can i find a daily training report template to download?', 'Where can I find a Daily Training Report template to download? ']	,
['show the best dtr templates?', 'where can i find a template for a dtr?', 'where can i find a template of a dtr?', 'where can i find a template of dtr?', 'where can i get the dtr template?', 'how do i find a dtr template?', 'where can i find a dtr template?', 'Where can I find a DTR Template? ']	,
['Where can we download a new DTR template for new hire classes and nesting?', 'Where can we download a new DTR template for new hire classes and nesting?']	,
['where can i find the free training report template?', 'where can i download a daily training report template?', 'where can i download the daily training report template?', 'where is the training daily report template that i can download?', 'where is the daily training report template i can download', 'where is the daily training report template that i can download', 'Where is the Daily Training Report template that I can download']	,
['where can i get an example daily training report?', 'tell me the template of the daily training report?', 'where can i get a free printable daily training report template?', 'tell me the daily training report template?', 'where is the daily training report template?', 'Where is the Daily Training Report template? ']	,
['where can i download a template of a dtr?', 'where can i download the dtr template?', 'where is the dtr template i can download', 'where is the dtr template that i can download?', 'where is the dtr template that i can download', 'Where is the DTR template that I can download']	,
['where can i find sops and best practices to help me with my classroom problems?', 'is there any sop i can use to address the classroom problem?', 'where can i find sops for classroom issues?', 'where can i find an sop when dealing with the classroom issue?', 'where can i find sops when dealing with classroom problems?', 'Where can I find SOPs when dealing with classroom issues?']	,
['where can i find sops in a training program?', 'how do i learn sops?', 'where can i find an sop?', 'Where do I find Training SOPs?']	,
['show the best templates for a daily training report?', 'tell me the format for daily training reports?', 'how can i get the daily training report template?', 'where can i get a training report template?', 'is there a daily training report template?', 'how do i get a daily training report template?', 'where can i find a daily training report template?', 'Where do I find a Daily Training Report Template?']	,
['which is the best dtr template for your website?', 'how do i find a template for dtr?', 'where can i download a dtr template?', 'where can i find dtr templates?', 'where can i find a dtr template?', 'where do i find a dtr template?', 'Where do I find a DTR Template?']	,
['i need compliance training from where i live where can i get it?', 'how do i complete compliance training?', 'where can i get effective compliance training?', 'where can i find compliance training?', 'where can i find the compliance training in my class?', 'where do i find the compliance training i need in a classroom?', 'where do i find compliance training for my class?', 'where can i find the compliance training a class needs to complete?', 'Where do I find compliance training my class needs to complete?']	,
['can you list the best certification training videos for compliance?', 'which training materials is the best for compliance certification?', 'tell me the best and best training material for compliance?', 'recommend some best training materials for compliance certifications?', 'list the best training materials for compliance certification?', 'can you list the best training materials for compliance certification?', 'which is the best training material for compliance certification?', 'which are the best training materials for compliance certification?']	,
['can you list the best websites to learn compliance?', 'where can i get more info about compliance and compliance?', 'where can i find out more about compliance?', 'where can i learn more about compliance?', 'where can i get information on compliance?', 'where can i get more details about compliance?', 'where can i find more information about compliance?', 'where can i get more information about compliance?', 'where can i get more information about compliance?']	,
['what are compliance training programs available in the us?', 'where can i get professional help on compliance training?', 'where can i get guidance for compliance training?', 'where can i get assistance with compliance training?', 'where can i get advice on compliance training?', 'where can i get help with compliance training?', 'where can i get help with compliance training?']	,
['how do i get correct compliance training?', 'where can i get necessary training about compliance?', 'where can i get training for the compliance section of my class?', 'can you list the good places to get mandatory compliance training for my class?', 'where can i get necessary compliance training?', 'where can i get necessary compliance training for my class?', 'where can i get necessary compliance training for my class?']	,
['is a compliance training required in business class?', 'what should i follow for a compliance course for business class?', 'what should i expect from a compliance training in a business class?', 'what is an mandatory compliance training for business class?', 'what are the requirements of compliance training for business class?', 'list the requirements for compliance training for a business class?', 'what are requirements of compliance training for a business class?', 'what is required for a compliance training for a business class?', 'what is a requirement for compliance training for a business class?', 'what is a requirement for a compliance training for a business class?']	,
['how can i review week 1 of marketplace training online?', 'what are review guides for a marketplace class?', 'where do i find reviews for marketplace class?', 'where can i find review guides for my marketplace class?', 'Where do I find Review Guides to use for my Marketplace class to review week 1 of Marketplace training?']	,
['can you review week 1 of marketplace course?', 'i want to review week one of marketplace training where can i find review guides to use?', 'where can i get reviews for my marketplace class?', 'where can i find the review guide for marketplace training class?', 'where can i find a review guide for my marketplace class?', 'how do i find review guides for my marketplace class?', 'what do you look for in a review guide to review week 1 of marketplace training?', 'Where do I find Review Guides to use for my Marketplace class to review week one of Marketplace training?']	,
['which is the best review guide for marketplace class?', 'how do i review week 2 of marketplace training?', 'how do i find review guides for my marketplace class?', 'where can i find review guides for my marketplace training?', 'Where do I find Review Guides to use for my Marketplace class to review week 2 of Marketplace training?']	,
['how can i get a review guide for my weekly marketplace class?', 'which are the best review guide resources for online training on the marketplace?', 'where can i find a review guide that i can use for my marketplace course?', 'where can i find review guides for marketplace class?', 'where can i find a review guide for my marketplace course for week two of marketplace training?', 'where can i find review guides for my marketplace class for week 2 of marketplace training?', 'Where do I find Review Guides to use for my Marketplace class to review week two of Marketplace training?']	,
['where can i find a review guide for my training on medicare?', 'show the best book reviews for my first week of medicare class?', 'Where do I find Review Guides to use for my Medicare class to review week 1 of Medicare training?']	,
['is there any review guide in a medicare class?', 'where can i find review guides for my medicare training classes?', 'can you list the review guides that i can use for my medicare classes to review week 1 of training?', 'where can i find review guides for my medicare class for week one of training?', 'where can i find review guides for medicare classes for week one of medicare training?', 'Where do I find Review Guides to use for my Medicare class to review week one of Medicare training?']	,
["i'm planning on attending week 2 of medicare training in march where can i find review guides?", 'who would make the best guide for reviewing a medicare class?', 'how do i review review guides for a medicare class?', 'where do i find review guides for my medicare classes?', 'how can i use the review guide for my medicare class to review week 2 of my medicare training?', 'Where do I find Review Guides to use for my Medicare class to review week 2 of Medicare training?']	,
['how do i find a review guide that helps me with my homework in medicare class?', 'can you list the review guides for my medicare class on week two of medicare training?', 'Where do I find Review Guides to use for my Medicare class to review week two of Medicare training?']	,
['which is the best review guide for medicare training?', 'is there a manual for review of medicare training?', 'where can i find reviews for my medicare class?', 'how do i find review guides for my week 3 course?', 'how do i use review guides for my medicare class?', 'where can i find review guides for my medicare training?', 'where do i find review guides for my medicare class?', 'can i make a review guide for my third week of medicare training?', 'Where do I find Review Guides to use for my Medicare class to review week 3 of Medicare training?']	,
['where can i find medicare review guides for my medicare class?', 'where can i find a review guide for my medicare class?', 'how do i find review guides for my medicare class?', 'where can i find reviews for week three of medicare training?', 'Where do I find Review Guides to use for my Medicare class to review week three of Medicare training?']	,
['where does my class find daily surveys?', 'Where does my class find the daily surveys?']	,
['where can i get assignments for my daily quiz?', 'where can i find a lot of daily surveys for my class?', 'where can i find daily surveys in my class?', 'where can i find my daily surveys in my class?', 'where can i get daily assignments in my class?', 'where can i find daily surveys and assignments in my classes?', 'where can i find all daily surveys and assignments in my class?', 'where can i find the daily surveys and assignments in my class?', 'where can i find the daily surveys and assignments in my class?']	,
['where is the termination procedure outlined?', 'where is the termination policy?', 'where is the termination schedule or policy on termination?', 'where is the sop or policy detailing the termination steps?', 'where is the sop or policy outlining the termination steps?', 'Where is the SOP or Policy outlining the termination steps?']	,
["how do you describe the termination steps in a company's operating procedure?", 'tell me the standard operating procedure or policy?', 'tell me the standard operating procedure outline for termination?', 'tell me the standard operating procedure of termination steps?', 'what is a standard operating procedure or policy that outline termination steps?', 'where is the standard operating procedure and policy stating the termination steps?', 'where is the standard operating procedure or policy outlining termination steps?', 'where is the standard operating procedure or policy outlining the termination steps?', 'Where is the Standard Operating Procedure or Policy outlining the termination steps?']	,
['who gets class folders after certification?', 'who gets the class file after certification?', 'who gets the class folder after certification?', 'who gets the class folders after certification?', 'Who gets the class folders after certification?']	,
["who gets the certificated trainees' folders?", "who gets the trainee's document after certification?", "who gets the trainees' folders after certification?", "Who gets the trainee's folders after certification?"]]

In [13]:
# now run all intents and make 0 succsess 1 fail on a medoum set
results = []
sorry =0
index =0
for x in questions:
    answer = answers[index]
    texts = get_para(x[0], 1, len(text))
    #texts.append(i)
    print(texts)
    for text in texts:
        res = qa_model_input(text, questions, answers)
        #print('AI :', res)
        #print('answer :', answer)
        if(answer == res or res == 'Sorry I do not understand the question, can you please try asking again.'):
            results.append(0)       
        else:
            results.append(1)
        if(res == 'Sorry I do not understand the question, can you please try asking again.'):
            sorry+=1  

    index +=1
    print(results)

print('')
print('these are the final results')
print(results)
print('please try again count =',sorry)

['a csr in my class cannot access their cco learning']
[0]
['A CSR in my class locked themselves out of their application.  How will they complete the daily health assessment? ']
[0, 0]
['A CSR quit but their timecard is wrong.  How can I get their timecard corrected? ']
[0, 0, 0]
['A step-by-step guideline on how to fill in specific sections of the daily training report  on the daily basis.']
[0, 0, 0, 0]
['a student has questions about the forms']
[0, 0, 0, 0, 0]
["i have a student who's had his cell phone go off in class what can i do?"]
[0, 0, 0, 0, 0, 1]
['am i completing a daily training report correctly?']
[0, 0, 0, 0, 0, 1, 0]
['why are we required to take calls during our classes?']
[0, 0, 0, 0, 0, 1, 0, 0]
['what is the priority of day one nesting?']
[0, 0, 0, 0, 0, 1, 0, 0, 0]
['can we use cco learning to load training classes?']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1]
['show some different deltek charge codes to use on days 1 and 2?']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1]
['Are there re

[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
['how can i solve windows access problems in my class?']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['how can i create training online?']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['How do I submit a reset password ticket correctly?']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
["How do I submit a ticket to unlock a trainee's computer"]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,

[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
['I need a Call Cert form. Where do I find a form?']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]
['i need some ideas for icebreakers']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
['i need some ideas for review activities']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

['how do i submit a form for csr certification?']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]
['What happens if I forget to do the Daily Training Report every day?']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
['what happens if i get a minus level 1?']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0

['where are my class rosters?']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0]
['Where are my instructor guides for Marketplace?']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]
['where can i find the instructor guides for medicare?']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
#target_set
# now run all intents and make 0 succsess 1 fail
results = []

index =0
targets_index =0
for x in questions:
    print('cuurently at '+str(index)+' out of'+str(len(questions)))
    answer = answers[index]
    for i in x:
        texts = targets_set[targets_index]
        targets_index +=1
        print(texts)
        for text in texts:
            res = qa_model_input(text, questions, answers)
            #print('AI :', res)
            #print('answer :', answer)
            if(answer == res):
                results.append(0)
            else:
                results.append(1)
    index +=1
    print(results)

print('')
print('these are the final results')
print(results)

In [9]:
# now run all intents on an easy set and make 0 succsess 1 fail
results = []

index =0
for x in questions:
    answer = answers[index]
    for i in x:
        texts = [i]
        print(texts)
        for text in texts:
            res = qa_model_input(text, questions, answers)
            #print('AI :', res)
            #print('answer :', answer)
            if(answer == res):
                results.append(0)
            else:
                results.append(1)
    index +=1
    print(results)

print('')
print('these are the final results')
print(results)

['A CSR in my class cannot access their CCO Learning ']
['A CSR in my class cannot get into their CCO Learning. ']
['A CSR in my class cannot log into their CCO Learning ']
['A participant in my class cannot access their CCO Learning ']
['A participant in my class cannot get into their CCO Learning. ']
['A participant in my class cannot log into their CCO Learning ']
['A trainee in my class cannot access their CCO Learning. ']
['A trainee in my class cannot get into their CCO Learning.  ']
['A trainee in my class cannot log into their CCO Learning. ']
['How do I submit a ticket to report my trainee in class cannot get into CCO Learning? ']
['Who do I reach out to if a trainee cannot access CCO Learning?']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['A CSR in my class locked themselves out of their application.  How will they complete the daily health assessment? ']
['How to handle the clear2work app if a trainee gets locked out of the app?']
['One of my new hires is locked out of the Health Ass

['How to push out on CCO Learning for your assigned participants?']
['How to push out on CCO Learning for your assigned trainees?']
['How to push out/assign on CCO Learning for your assigned trainees?']
['I need to push the materials out to my class.  How do I do that?']
['Show me how I push the materials out to my class?']
['What are the steps to assign the materials to my class?']
['What steps do I take to assign the materials for my class? ']
['What steps do I take to push cources on CCO Learning to my class?']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['Are there different deltek charge codes we are supposed to use on days 1 and 2? ']
['Are there different Deltek project codes we are supposed to use on days 1 and 2?

['Can we play calls other than those in the lessons for hands on activities? ']
['Can we pull calls other than those in the materials for hands on activity? ']
['Do we only use the calls in the lessons? ']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['Can we use different calls to listen during class not only the ones on the instructor resources list?']
['I am excellent on calls.  Can I play my own calls for my class? ']
['I found some great calls to play for class!']
['The calls in the materials are not as good as mine. 

['I need my class to be moved over to me in CCO Learning. ']
['I need to assign materials to my class, but they are not rolling up to me in CCO Learning.']
['My class is not under me in CCO Learning. How can I get that fixed? ']
['What do I do if I need my class moved to me in CCO Learning so I can assign the curriculum to them? ']
['What do I do if the new hires are not rolling up to me in CCO Learning?']
['What do I do if the participants are not rolling up to me in CCO Learning?']
['What do I do if the trainees are not rolling up to me in CCO Learning?']
['What ticket should I submit to get me class moved over to me to assign curriculum to them? ']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

['How do I correct errors on my Daily Training Report? ']
['How do I correct errors on my DTR? ']
['How do I go about uploading my daily training report? ']
['How do I prepare the class Daily Training Report? ']
['How do I prepare the class DTR?']
['How do I record information in the Daily Training Report?']
['How do I record information in the DTR?']
['How do I upload a Daily Training Report?']
['How do I upload a DTR?']
['How do I upload the DTR for class? ']
['How do I Upload the DTR for either Class Training or Nesting?']
['How much information should I record in my Daily Training Report?']
['How much information should I record in my DTR?']
['How to fix errors on Daily Training Report']
['How to fix errors on DTR']
['I messed up my Daily Training Report.  How do I fix it?']
['I messed up my DTR.  How do I fix it?']
['I need to fix errors on my Daily Training Report. ']
['I need to fix errors on my DTR. ']
['I need to upload my Daily Training Report.']
['I need to upload my DTR.']


...

['can you list the participants guides for the marketplace?']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1

['Where do I find Training SOPs?']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1

['Where do I find Review Guides to use for my Marketplace class to review week one of Marketplace training?']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1

['Where do I find Review Guides to use for my Medicare class to review week two of Medicare training?']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1

['Where is the Standard Operating Procedure or Policy outlining the termination steps?']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0

## Save the data to CSV

now we take the results and save them to a csv format 


In [3]:
results =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0]
print(results)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 

In [14]:
import csv

header = ['index', 'pred', 'expected']
data = []

index = 0
for x in results:
    item = [index,x,0]
    data.append(item)
    index +=1
    
    

#with open('csv data/BERT_data.csv', 'w', encoding='UTF8', newline='') as f:
with open('csv data/BERT_data_medium.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

print('done all data writen to csv')

done all data writen to csv
